---
title: "Meta Learning in Neural Networks — A Survey"
date: 2025-11-13       
description: "메타 러닝 관련 논문 요약 및 주요 내용"
categories: [MetaLearning, Survey, Review]
author: "김한울"
---


# Paper Review: Meta Learning in Neural Networks: A Survey

``` 
@article{hospedales2021meta,
  title={Meta-learning in neural networks: A survey},
  author={Hospedales, Timothy and Antoniou, Antreas and Micaelli, Paul and Storkey, Amos},
  journal={IEEE transactions on pattern analysis and machine intelligence},
  volume={44},
  number={9},
  pages={5149--5169},
  year={2021},
  publisher={IEEE}
}
```

`업데이트 내역`

||||
|-|-|-|
|2025-11-14|태스크 분포 관점 업데이트||
|2025-11-15|Bilevel optimization View 업데이트||
|2025-11-16|2.2-2.3 업데이트|배고프군|
|2025-11-17|4.1 업데이트||

# Abstract

`Learning-to-learn`이라고도 불리는 메타-러닝 분야는 최근 몇 년간 관심이 급격히 증가해왔습니다. 
고정된 학습 알고리즘을 사용하여 처음부터 과제를 해결하는 기존의 AI 접근 방식과는 대조적으로, 
메타-러닝은 다수의 학습 에피소드 경험을 바탕으로 **학습 알고리즘 자체를 개선하는 것을 목표**로 합니다. 

이 패러다임은 데이터 및 계산 병목 현상, 그리고 일반화 성능 등 딥러닝의 여러 기존 난제들을 해결할 기회를 제공합니다. 

본 서베이는 현대 메타-러닝의 전반적인 동향을 기술합니다. 

+ 먼저 메타-러닝의 정의를 논하고, 전이 학습(transfer learning) 및 하이퍼파라미터 최적화(hyperparameter optimization)와 같은 관련 분야와 비교하여 그 위치를 정립합니다. 
+ 다음으로, 오늘날의 메타-러닝 방법 공간을 더 포괄적으로 분석하는 새로운 분류 체계를 제안합니다. 
+ 또한 퓨샷 러닝(few-shot learning) 및 강화 학습(reinforcement learning)과 같은 메타-러닝의 유망한 응용 분야와 성공 사례들을 살펴봅니다. 
+ 마지막으로, 아직 해결되지 않은 과제들과 향후 연구를 위한 유망한 영역들에 대해 논의합니다.

# Introduction

메타러닝이란, "머신러닝 모델"이 "공부하는 법"자체를 배우게 하는 새로운 패러다임이라고 할 수 있음

> Meta-learning provides an alternative paradigm where a machine learning model gains experience over multiple learning episodes – often covering a distribution of related tasks – and uses this experience to improve its future learning performance.

1. provides an alternative paradigm?

* 기존 방식?    
  기계에게 '고양이 사진 분류'라는 숙제 하나를 주고, 그 숙제만 잘 풀도록 처음부터 끝까지 가르침. 다른 숙제(예: '개 사진 분류')를 주면 또 처음부터 새로 배워야 함.   
* 대안(메타러닝):    
  기계에게 숙제 하나만 주는 게 아니라, **"스스로 공부하는 법"**을 터득하게 만들자!   

2. a machine learning model gains experience over multiple learning episodes

* 비유를 해보자면    
  사람에게 수학만 가르치는 게 아니라, 수학, 과학, 국어 등 **여러 과목의 문제(관련된 과제들)**를 풀게 하는 행위랑 비슷함.   
  이 과정에서 모델은 단순히 개별 문제를 푸는 법을 배우는 게 아니라, 문제들 사이의 공통점이나 문제 해결의 **'요령(패턴)'**을 깨닫게 됨. 이것이 바로 **"경험"**이라고 할 수 있음.
* `learning episodes`: 문제 하나하나를 풀어보는 경험 한 번 한 번을 의미.


보통의 경우에, 그리고 역사적으로 보면, 

1. 머신러닝   
+ 기계가 데이터를 잘 이해할 수 있도록 데이터의 핵심적인 특징(feature)을 사람 전문가(human-expert)가 직접 추출함.
  + hand-engineered features   
+ 그 후, 사람이 뽑아낸 특징 데이터를 입력으로 받아서, 기계가 정답을 맞히는 패턴을 학습.
  + 이미지 문제를 예로 들었을 때, 기계는 원본 이미지를 보고 있는 것이 아니라, 사람이 가공해서 준 특징 값들만 보게 됨.   
+ 즉, 머신러닝에서 **특징 추출 단계**와 **모델 학습 단계**는 분리 되어 있음.



2. 딥러닝   
+ 위 머신러닝의 두 단계인 **특징 추출 단계**와 **모델 학습 단계**를 하나의 단계로 통합해버림.    
  + 특징과 모델의 공동 학습 (Joint feature and model learning)    
+ 예전 처럼 특징을 정성스럽게 추출할 필요가 없음.
  + 원본 데이터(이미지 형태 그대로!) 모델에 그대로 입력하면,   
  + 모델이 내부의 여러 계층(layer)을 거치면서 자동으로 특징을 찾아내고, 동시에 그 특징을 이용해 분류하는 방법까지 한꺼번에 학습함.   
+ 고양이 사진 분류하는 모델을 예로 들어보면...   
  + 초반 계층: 이미지의 가장 기본적인 특징(선, 색상, 명암 대비 등)을 스스로 학습.   
  + 중간 계층: 초반 계층에서 학습한 기본 특징들을 조합하여 더 복잡한 특징(눈, 코, 귀의 형태 등)을 학습.   
  + 후반 계층: 중간 계층의 복잡한 특징들을 다시 조합하여 최종적으로 "이것이 고양이다"라고 판단하는 방법을 학습.
+ 어떤 특징이 중요한지를 기계가 데이터로부터 직접 배우는 것.    
+ 사람이 "귀가 중요해, 수염이 중요해"라고 알려줄 필요가 없게 됨. 
+ 이처럼 딥러닝에서는 특징을 배우는 과정과 그 특징으로 정답을 맞히는 과정이 '공동으로(jointly)', 즉 '동시에' 최적화 됨.   

3. 신경망에서의 메타러닝?
+ 특징 추출 단계, 모델 학습 단계, 그리고 `알고리즘`을 하나의 단계로 통합하고자 하는 학습 방식이라고 할 수 있음.


Thrun은 '러닝-투-런'을 다음과 같이 측정 가능하게(operationally) 정의하고 있습니다. 

> Thrun [7] operationally defines learning-to-learn as occurring when a learner’s performance at solving tasks drawn from a given task family improves with respect to the number of tasks seen.   
  > *해당 책은 한 권에 40만원이라 읽어보지는 못할 것 같다. S. Thrun and L. Pratt, “Learning To Learn: Introduction And Overview,” in Learning To Learn, 1998 *

> 학습자(AI)가 관련된 과제 그룹(task family)에서 여러 과제를 풀어볼수록,    
> 즉 더 많은 종류의 과제를 경험할수록 새로운 과제를 푸는 성능이 향상되는 현상.

+ task family (과제 그룹): 서로 다르지만 관련이 있는 문제들의 묶음입니다.   
  + 예시: ('고양이/개 분류', '사자/호랑이 분류', '새/물고기 분류')는 모두 **"동물 분류"**라는 하나의 task family에 속합니다.   
  + AI가 '고양이/개 분류' 문제를 푼 다음, '사자/호랑이 분류' 문제를 풀고, 또 '새/물고기 분류' 문제를 푸는 등... 이렇게 다양한 종류의 과제(tasks)를 더 많이 경험할수록, 나중에 처음 보는 '코끼리/기린 분류' 문제도 더 잘 풀게 된다는 것입니다. 
  + 즉, 경험하는 과제의 '종류'가 늘어날수록 똑똑해집니다.

> conventional machine learning performance improves as more data from a single task is seen

+ 기존 머신러닝은 하나의 과제(a single task)에 대한 데이터가 많아질수록 성능이 향상됩니다.

**공짜 점심은 없다(no free lunch) Theorem** 에 대항하는 알고리즘 ㅋㅋ

> This perspective [27]–[29] views meta-learning as a tool to manage the ‘no free lunch’ theorem [30] and improve generalization by searching for the algorithm (inductive bias) that is best suited to a given problem, or problem family.

> 이러한 관점은 메타러닝을 '공짜 점심은 없다'는 정리(한계)에 대처하는 도구로 봅니다. 즉, 주어진 문제나 문제 그룹에 가장 적합한 알고리즘(귀납적 편향)을 탐색함으로써 일반화 성능을 향상시키는 도구라는 것입니다.

> However, this definition can include transfer, multi-task, feature-selection, and model-ensemble learning, which are not typically considered as meta-learning today.

> 하지만, 이러한 정의는 전이 학습, 다중과제 학습, 특징 선택, 모델 앙상블 학습까지 포함할 수 있는데, 이들은 오늘날 일반적으로 메타러닝으로 간주되지 않습니다.

+ "문제에 맞는 해결책을 찾는다"는 정의가 너무 광범위해서, 관련은 있지만 엄연히 다른 여러 기법들까지 전부 '메타러닝'의 범주에 포함시켜 버린다는 문제가 있다는 맥락입니다. 이 관점은 메타러닝의 철학을 이해하는 데는 도움이 되지만, 현대의 메타러닝을 다른 기법들과 구분 짓기에는 그 정의가 너무 모호하고 포괄적이라는 한계가 있습니다. 오늘날의 메타러닝은 단순히 알고리즘을 '선택'하거나 '재사용'하는 것을 넘어, '학습하는 과정 자체'를 최적화하는 더 구체적인 의미로 사용되기 때문입니다.

이 논문이 Review하려고 하는 대상 논문?

+ 본 논문에서는 현대의 신경망 기반 메타러닝에 집중하고 있습니다. 
  + 참고문헌 [27], [28]에 따라 **'알고리즘 학습'**으로 간주하지만, 특히 이 학습이 명시적으로 정의된 목적 함수(예: 교차 엔트로피 손실)의 종단간(end-to-end) 학습을 통해 달성되는 경우에 초점을 맞춥니다. 
  + 이에 더해, 우리는 단일 과제 메타러닝을 고려하고, 강건성(robustness) 및 컴퓨팅 효율성과 같은 더 폭넓고 다양한 (메타) 목적 함수에 대해서도 논의할 것입니다.

   
+ 본 논문에서는 메타러닝의 방법론과 응용 분야를 모두 다루려고 함. 
  + 먼저, 이 분야의 연구들을 이해하고 그 위상을 정립하는 데 사용할 수 있는 고수준의 문제 정형화(formalization)를 통해 메타러닝을 소개합니다
  + 그런 다음, **메타-표현(meta-representation), 메타-목적(meta-objective), 메타-optimzer(meta-optimizer)**라는 관점에서 새로운 분류 체계를 제공할 것입니다. 
  + 이 프레임워크는 새로운 메타러닝 방법을 개발하고 다양한 응용에 맞게 맞춤화하기 위한 **설계 공간(design-space)**을 제시합니다. 
  + 우리는 퓨샷 학습, 강화 학습, 구조 탐색 등 여러 인기 있고 새롭게 부상하는 응용 분야를 살펴보고, 전이 학습 및 다중과제 학습과 같은 관련 주제와 비교하여 메타러닝의 위상을 정립할 것입니다. 
  + 마지막으로, 아직 해결되지 않은 중요한 과제들과 미래 연구를 위한 유망한 영역들을 논의하며 마무리하겠습니다.


# BackGround

메타러닝은 현대 신경망 문헌 내에서조차 
다양하고 일관성 없는 방식으로 사용되어 왔기 때문에 "한 단어, 한 문장"으로 명확하게 정의하기가 어려운 상태임. 

+ 이 섹션에서는 우리의 정의와 주요 용어를 소개하고, 관련 분야와 비교하여 메타러닝의 position을 정립하고 있음.


#### 메타러닝?

**'학습하는 방법을 학습하는 것(learning to learn)'**

+ 이는 **여러 학습 에피소드에 걸쳐 학습 알고리즘을 개선하는 과정을 의미**함. 

+ 기존의 머신러닝이 여러 데이터 인스턴스에 걸쳐 모델 예측을 개선하는 것과 대조적. 


+ **기반 학습(base learning)** 중에는 **내부(inner)** (또는 하위/기반) 학습 알고리즘이 이미지 분류와 같은 과제를 해결하며, 이는 데이터셋과 목적 함수에 의해 정의됨. 흔히 inner loop라는 표현으로도 많이 쓰임    
+ **메타러닝** 중에는 **외부(outer)**(또는 상위/메타) 알고리즘이 내부 학습 알고리즘을 업데이트하여, 그것이 학습하는 모델이 외부 목적 함수를 개선하도록 만듦. 
  + 예를 들어, 이 외부 목적 함수는 내부 알고리즘의 일반화 성능이나 학습 속도가 될 수 있음. 
  + (기반 알고리즘, 학습된 모델, 성능) 튜플로 구성된 기반 과제의 학습 에피소드들은 외부 알고리즘이 기반 학습 알고리즘을 학습하는 데 필요한 인스턴스를 제공하는 것으로 볼 수 있습니다.

  > inner loop의 결과물이라고 할 수 있는 (Base Algorithm, Trained Model, Performance)을 outer loop의 알고리즘이 "학습 데이터"로 삼아서 "기반 알고리즘 자체를 좋게 만드는 방법"을 학습하게 됨.
 


> 메타 러닝은 여러 학습 에피소드에 걸쳐 학습 알고리즘을 개선하는 과정?

위에 정의된 대로라면, `Cross Validtion`을 통한 하이퍼파라미터의 무작위 탐색과 같은 많은 기존 알고리즘이 메타러닝의 정의에 포함되어 버림. 

#### 현대의 neural-network meta-learning의 두드러지는 특징

+ **명시적으로 정의된 메타 수준의 목적 함수**이 있고,    
+ 이 목적 함수에 대한 내부 알고리즘의 **end-to-end 최적화**가 이루어짐.


## 메타러닝을 공식화 해보자

### 기존 머신러닝 

기존의 지도(supervised) 머신러닝에서는 (입력 이미지, 출력 레이블) 쌍과 같은 훈련 데이터셋 $D = \{(x_1, y_1), \dots, (x_N, y_N)\}$이 주어짐. 

$\theta$로 매개변수화된 예측 모델 $\hat{y} = f_{\theta}(x)$를 다음 식을 풀어 훈련시키게 됨:


$$
\theta^* = \arg\min_{\theta} \mathcal{L}(D; \theta, \omega) \quad (1) 
$$


* $\mathcal{L}$은 실제 레이블과 $f_{\theta}(\cdot)$가 예측한 값 사이의 오차를 측정하는 손실 함수. 
* $\omega$라는 조건이 걸려 있음.
  * 이는 학습하는 방법 $\omega$에 따라 이 식(1)의 해인 $\theta$가 달라질 수 있다는 의미임.   
  * $\omega$ 예를 들어보자면 Optimizer의 선택, model의 선택이 될 수 있음.   

* 일반화 성능은 알려진 레이블을 가진 여러 테스트 포인트를 평가하여 측정됨.



##### 기존 머신러닝의 2가지 가정

(1) 최적화 과정이 모든 문제 $D$에 대해 매번 처음부터 수행됨(from scratch)   
(2) 학습 방법 $\omega$는 사전에 지정됨.   


이때, $\omega$의 **specification**, 즉 `학습 방법을 어떻게 정하느냐`는 정확도나 데이터 효율성과 같은 성능 지표에 큰 영향을 미칠 수 있음. 

+ 메타러닝은 이러한 지표를 개선하기 위해 학습 알고리즘 자체를 *사전에 지정하고 고정하는 대신* 학습 알고리즘 자체를 학습하게 됨. 

+ Specification? 학습 알고리즘의 구체적인 내용과 구성이라고 할 수 있음

  > optimzer(Optimizer)의 종류: SGD, Adam, RMSprop 등 어떤 것을 쓸 것인가?    
  > 학습률(Learning Rate): 학습률을 얼마로 설정할 것인가?   
  > 모델 구조(Model Architecture): 어떤 종류의 신경망(CNN, RNN 등)을 사용할 것인가?   
  > 정규화(Regularization) 방법: L1, L2, Dropout 등 어떤 정규화 기법을 적용할 것인가?   

### 과제 분포(Task-Distribution) 관점에서 본 메타러닝


> 메타러닝을 통해 여러 과제에 걸쳐 일반화할 수 있고,    
> 이상적으로는 새로운 과제를 접할 때마다 이전보다 더 잘 학습할 수 있게 해주는 **범용 학습 알고리즘**을 학습하자. 

**Notation**    

+ $p(\mathcal{T})$: 과제들의 분포
+ $\omega$: 어떤 학습 방법 
+ $\mathcal{T} = \{D, L\}$: 어떤 과제($\mathcal{T}$)는 데이터셋($D$)과 손실 함수($L$)의 조합이다. 
+ $D$: 데이터 셋
+ $\mathcal{L}(D, \omega)$: 데이터 셋 $D$에서 학습 방법 $\omega$를 사용해 훈련했을 때의 loss 값

위와 같이 정의 했을 때, '학습하는 법을 배우는 것'은 다음과 같이 표현할 수 있음.

$$
\min_{\omega} \mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})} \mathcal{L}(D; \omega) \quad (2)
$$


'학습 방법', 즉 $\omega$는 **과제 전반의 지식(across-task knowledge)** 또는 **메타 지식(meta-knowledge)** 이라고 할 수 있음.


**식 (2)를 실제로 해결하려면?**

* 목표: 세상의 모든 과제 분포 $p(\mathcal{T})$에 대해 평균적으로 가장 좋은 성능을 내는 만능학습법 $\omega$를 찾자!   
* 현실: 모든 과제 분포의 모든 문제인 $p(\mathcal{T})$를 다룰 수는 없음.
* 타협: 그러니까, 우리가 가진 문제는 "전체 과제들의 분포 $p(\mathcal{T})$를 어느정도 대표할 수 있는 샘플들"이야! --> source tasks(소스 과제)

**Notation 2**

$\mathcal{D}_{\text{source}} = \{(D_{\text{source}}^{\text{train}}, D_{\text{source}}^{\text{val}})^{(i)}\}_{i=1}^M$

* **메타-훈련(meta-training)에 사용할 전체 데이터셋**을 나타내는 기호. 

하나씩 뜯어보면

+ $\mathcal{D}_{\text{source}}$ : **'소스 데이터셋(Source Dataset)'**이라는 뜻. 여기서 '소스(Source)'는 메타 지식(학습 노하우)을 배우는 **원천(Source)**이 된다는 의미. 즉, **"훈련용"**이라는 뜻.
+ $M$: 우리가 가지고 있는 **훈련용 과제(task)의 총 개수**. (예: 50개의 다른 종류의 동물 분류 문제)
+ $\{ \dots \}_{i=1}^M$: 괄호 안의 내용이 **1번부터 M번까지 M개**가 있다는 뜻.
+ $( \dots )^{(i)}$: 그중에서 **i번째 과제**를 의미. (예: 50개 중 17번째 과제)
+ $(D_{\text{source}}^{\text{train}}, D_{\text{source}}^{\text{val}})$: 하나의 과제($i$)가 두 종류의 데이터로 구성되어 있다는 뜻.
  + $D_{\text{train}}^{\text{source}}$: **'훈련용' 데이터(train data)**. 이 과제를 풀기 위해 공부하는 데 사용되는 데이터. 메타러닝에서는 이것을 특별히 **서포트셋(support set)**이라고 함.
  + $D_{\text{val}}^{\text{source}}$: **'검증용' 데이터(validation data)**. 위에서 공부한 내용으로 얼마나 잘하는지 쪽지시험을 보는 데 사용되는 데이터. 메타러닝에서는 이것을 특별히 **쿼리셋(query set)**이라고 함.

> "$\mathcal{D}_{\text{source}}$란, 총 M개의 훈련용 과제 묶음인데, 각 과제$i$는 '서포트셋(훈련용)'과 '쿼리셋(검증용)'이라는 두 개의 데이터 묶음으로 이루어져 있다."

아무튼, 저런 데이터 묶음으로 뭘할거냐 하면, 식 (3)을 풀기 위함이다.

$$
\omega^* = \arg\max_{\omega} \log p(\omega|\mathcal{D}_{\text{source}}) \quad (3)
$$

또, 식을 하나씩 요소별로 뜯어보자.

+ $\omega$ (오메가): 우리가 찾고 싶은 **'학습 방법' 또는 '공부법'**
+ $\omega^{\ast}$ (오메가 스타): 수많은 가능한 공부법($\omega$) 중에서 우리가 찾아낸 **최고의(optimal) 공부법**을 의미함.
+ $\arg\max_{\omega}$: "괄호 안의 값을 **최대(max)로 만드는** $\omega$를 **찾아라(arg)**"라는 명령어.
+ $p(\omega|\mathcal{D}_{\text{source}})$: **사후 확률(posterior probability)**.
  + **"우리가 가진 훈련용 과제 데이터($\mathcal{D}_{\text{source}}$)를 관찰했을 때, 어떤 공부법($\omega$)이 가장 그럴듯한가(정답일 확률이 높은가)?"**.

> "우리가 가진 **훈련용 과제 데이터($\mathcal{D}_{\text{source}}$)를 가장 잘 설명하고 해결할 수 있는, 가장 그럴듯한(확률이 가장 높은) 학습 방법($\omega$)을 찾아서, 그것을 우리의 최종 학습법($\omega^{\ast}$)으로 삼아라!"**

이제 메타-테스트 단계에서 사용되는 $Q$개의 타겟 과제(target tasks) 집합을 
$\mathcal{D}_{\text{target}} = \{(D_{\text{target}}^{\text{train}}, D_{\text{target}}^{\text{test}})^{(i)}\}_{i=1}^Q$
로 표기하며, 각 과제는 훈련 데이터와 테스트 데이터를 모두 가집니다. 메타-테스트 단계에서는 학습된 메타 지식 $\omega^*$를 사용하여 이전에 보지 못한 각 타겟 과제 $i$에 대한 기반 모델을 훈련합니다:

$$
\theta^{*(i)} = \arg\max_{\theta} \log p(\theta|\omega^*, D_{\text{target}}^{\text{train }(i)}) \quad (4)
$$

식 (1)의 기존 학습과 대조적으로, 타겟 과제 $i$의 훈련 세트에 대한 학습은 이제 사용할 알고리즘에 대한 메타 지식 $\omega^*$의 이점을 얻습니다. 이것은 좋은 초기 파라미터의 추정치[16]일 수도 있고, 전체 학습 모델[38] 또는 최적화 전략[39]일 수도 있습니다. 우리는 각 타겟 과제의 테스트 스플릿 $D_{\text{test}}^{\text{target}}(i)$에 대한 $\theta^{*(i)}$의 성능으로 메타 학습기의 정확도를 평가할 수 있습니다.

이러한 설정은 기존의 과소적합 및 과적합과 유사한 개념인 **메타-과소적합(meta-underfitting)** 과 **메타-과적합(meta-overfitting)** 으로 이어집니다. 특히, 메타-과적합은 소스 과제에서 학습된 메타 지식이 타겟 과제로 일반화되지 않는 문제입니다. 이는 비교적 흔하며, 특히 소수의 소스 과제만 사용할 수 있는 경우에 그렇습니다. 이것은 가설 공간 $\theta$를 소스 과제의 해법 주변으로 너무 강하게 제약하는 귀납적 편향 $\omega$를 학습하는 것으로 볼 수 있습니다.




**다음 섹션 넘어가기 전 수식 해설:** 

$\mathcal{D}_{\text{target}} = \{(D_{\text{target}}^{\text{train}}, D_{\text{target}}^{\text{test}})^{(i)}\}_{i=1}^Q$

이 수식은 **메타-테스팅(Meta-Testing) 단계**, 즉 최종 실력을 평가하는 데 사용되는 **"실전 시험 문제지 묶음"** 전체를 정의합니다.

이 구조를 세 단계로 나누어 이해하면 가장 쉽습니다.

#### 1단계: $\mathcal{D}_{\text{target}}$ - 전체 시험지 묶음

*   **이름**: 타겟 데이터셋 (The Target Dataset)
*   **역할**: 메타-훈련을 통해 학습된 '만능 공부법($\omega^*$)'이 얼마나 효과적인지 최종적으로 평가하기 위한 **전체 시험 문제들의 집합**입니다.
*   **표기 규칙**: `target`이 **아래 첨자(subscript)**로 붙어, 이 데이터셋 묶음의 **소속(Group)**이 '소스(훈련용)'가 아닌 '타겟(시험용)'임을 나타냅니다.

#### 2단계: $( \dots )^{(i)}$ 와 $\{ \dots \}_{i=1}^Q$ - i번째 시험지

*   **이름**: i번째 타겟 과제 (The i-th Target Task)
*   **역할**: 전체 시험지 묶음($\mathcal{D}_{\text{target}}$)은 총 **Q개의 개별 시험 문제(과제)**로 이루어져 있습니다. 이 표기는 그중 **i번째 시험 문제** 하나를 가리킵니다.
*   **비유**: '수능 시험'이라는 전체 묶음 속의 '수학 시험지' 하나에 해당합니다.

#### 3단계: $(D_{\text{target}}^{\text{train}}, D_{\text{target}}^{\text{test}})$ - 시험지 한 장의 구성

이것이 가장 중요한 부분입니다. '수학 시험지' 한 장은 두 부분으로 구성됩니다.

##### 가. $D_{\text{target}}^{\text{train}}$: 시험지 속 "참고 예시" 또는 "<보기>"

*   **정확한 표기**: `target`은 **아래 첨자**, `train`은 **윗 첨자**.
*   **역할**:
    1.  이 데이터는 이미 학습이 끝난 **'만능 공부법($\omega^*$)'을 개선하는 데 사용되지 않습니다.**
    2.  대신, $i$번째 새로운 문제를 만난 모델이 이 데이터를 **딱 몇 번만 보고** "아, 이 문제는 이런 유형이구나!"하고 **빠르게 적응(adaptation)**하는 데 사용됩니다.
    3.  이 적응 과정을 통해 $i$번째 문제에만 특화된 모델($\theta^{*(i)}$)이 만들어집니다.
*   **비유**: 시험 문제에 나오는 **`<보기>` 자료**와 같습니다. `<보기>`를 읽고 문제의 의도를 파악하고 적응하는 것이지, `<보기>`를 읽는다고 해서 근본적인 국어 실력($\omega$)이 오르는 것은 아닙니다.

##### 나. $D_{\text{target}}^{\text{test}}$: 시험지 속 "실제 채점 문제"

*   **정확한 표기**: `target`은 **아래 첨자**, `test`은 **윗 첨자**.
*   **역할**:
    1.  위에서 `<보기>`($D_{\text{target}}^{\text{train}}$)를 보고 적응을 마친 모델($\theta^{*(i)}$)에게 이 문제를 풀게 합니다.
    2.  모델의 답과 정답을 비교하여 **최종 성능(정확도)을 채점**합니다. 이 점수가 바로 메타 학습기의 $i$번째 과제에 대한 최종 실력입니다.
*   **비유**: `<보기>`를 참고하여 풀어야 하는 **실제 문제 1번, 2번, 3번**에 해당합니다.

#### 최종 정리

| 표기법 | 소속 (아래 첨자) | 역할 (윗 첨자) | 비유 |
| :--- | :--- | :--- | :--- |
| $\mathcal{D}_{\text{target}}$ | **타겟(Target)** | (없음) | **수능 시험 전체** (최종 평가 목적) |
| $D_{\text{target}}^{\text{train}}$ | **타겟(Target)** | **훈련(Train)** | 시험지 속 **`<보기>`** (새로운 유형에 적응) |
| $D_{\text{target}}^{\text{test}}$ | **타겟(Target)** | **테스트(Test)** | 시험지 속 **채점 문제** (최종 성능 평가) |


### 이중 최적화(Bilevel Optimization) 관점에서 본 메타러닝

이전 섹션의 논의는 다중 과제 시나리오에서 메타러닝의 일반적인 흐름을 설명했지만, 식 (3)의 메타-훈련 단계를 어떻게 풀어야 하는지는 명시하지 않았습니다. 
이는 보통 메타-훈련 단계를 **이중 최적화(bilevel optimization)** 문제로 구성함으로써 수행됩니다. 이 그림은 optimzer 기반(optimizer-based) 방법론(섹션 3.1 참조)에만 정확하게 들어맞는다고 할 수 있지만, 메타러닝의 작동 방식을 더 보편적으로 시각화하는 데 도움이 됩니다. 
이중 최적화[40]는 계층적 최적화 문제로, 하나의 최적화가 다른 최적화를 제약 조건으로 포함하는 구조를 의미합니다[17], [41]. 

이 표기법을 사용하면, 메타-훈련은 다음과 같이 정형화될 수 있습니다:

$$
\omega^* = \arg\min_{\omega} \sum_{i=1}^{M} \mathcal{L}^{\text{meta}}(\theta^{*(i)}(\omega), \omega, D_{\text{source}}^{\text{val }(i)}) \quad (5)
$$

$$
\text{s.t. } \theta^{*(i)}(\omega) = \arg\min_{\theta} \mathcal{L}^{\text{task}}(\theta, \omega, D^{\text{train }(i)}_{\text{source}}) \quad (6)
$$

여기서 $\mathcal{L}^{\text{meta}}$
와 
$\mathcal{L}^{\text{task}}$
는 각각 **외부(outer) 및 내부(inner) 목적 함수**를 의미하며, 퓨샷 분류의 경우 교차 엔트로피와 같은 것입니다. 외부와 내부 수준 사이의 **리더-팔로워(leader-follower) 비대칭성**에 주목하십시오: *내부 수준 최적화인 식 (6)은 외부 수준에서 정의된 학습 전략 $\omega$에 따라 조건부로 결정되지만, 자신의 훈련 중에는 $\omega$를 변경할 수 없습니다.*

여기서 $\omega$는 비볼록(non-convex) 최적화에서의 초기 조건[16], 정규화 강도와 같은 하이퍼파라미터[17], 또는 최적화할 손실 함수 $\mathcal{L}^{\text{task}}$의 매개변수화[42]까지도 나타낼 수 있습니다. 

+ 섹션 4.1에서 $\omega$에 대한 선택의 공간을 자세히 논의합니다. 

외부 수준 최적화는 훈련 후 검증 세트에서 좋은 성능을 보이는 모델 $\theta^{*(i)}(\omega)$를 생성하도록 $\omega$를 학습합니다.

+ 섹션 4.2에서는 $\omega$를 최적화하는 방법을 자세히 논의합니다. 
+ 섹션 4.3에서는 $\mathcal{L}^{\text{meta}}$가 검증 성능, 학습 속도, 모델 강건성 등 무엇을 측정할 수 있는지 고려합니다.

마지막으로, 위의 정형화는 과제 분포라는 개념을 사용한다는 점에 주목합니다. 

+ 이는 메타러닝 문헌에서 일반적이지만, 메타러닝의 필수 조건은 아닙니다. 

더 공식적으로, 만약 단일 훈련 및 테스트 데이터셋($M=Q=1$)이 주어진다면, 우리는 훈련 세트를 분할하여 메타-훈련을 위한
$\mathcal{D}_{\text{source}} = (D^{\text{train}}_{\text{source}}, 
D^{\text{val}}_{\text{source}})$
와 메타-테스트를 위한 
$\mathcal{D}_{\text{target}} = (D^{\text{train}}_{\text{source}} \cup D^{\text{val}}_{\text{source}}, D^{\text{test}}_{\text{target}})$를 얻을 수 있습니다. 
우리는 여전히 여러 에피소드에 걸쳐 $\omega$를 학습하며, 메타-훈련 중에는 보통 다른 훈련-검증 분할이 사용됩니다.

**comment**

메타러닝의 작동 방식을 **"두 단계로 이루어진 최적화"**라는 틀로 설명합니다. 마치 회사에서 **팀장(외부 루프)**과 **팀원(내부 루프)**이 협업하는 것과 같습니다.

#### 1. 두 개의 최적화 문제: 내부 루프와 외부 루프

*   **내부 최적화 (식 6) - 팀원의 임무**:
    *   팀장은 **업무 가이드라인($\omega$)**을 줍니다. (예: "이런 방식으로 초기 모델을 설정해봐", "학습률은 0.01로 해")
    *   팀원은 주어진 가이드라인($\omega$)과 훈련 데이터($D^{\text{train}}$)를 가지고, **자신의 과제($\theta$)를 가장 잘 해결하기 위해 최선을 다합니다.**
    *   **중요한 점**: 팀원은 팀장이 준 가이드라인($\omega$)을 바꿀 수 없습니다. 그냥 따라야 합니다. 그 결과물이 바로 **최적의 모델($\theta^*$)**입니다.

*   **외부 최적화 (식 5) - 팀장의 임무**:
    *   팀장은 팀원이 과제를 수행한 결과물($\theta^*$)을 가져와서 **실전 테스트($D^{\text{val}}$)**를 해봅니다.
    *   테스트 결과(성능)를 보고, **"내가 처음에 줬던 가이드라인($\omega$)이 과연 최선이었을까?"**를 고민합니다.
    *   **목표**: 팀원의 최종 성과($\mathcal{L}^{\text{meta}}$)가 가장 좋아지도록, **최초의 가이드라인($\omega$) 자체를 수정하고 개선**합니다. 이것이 바로 팀장의 최적화, 즉 메타러닝입니다.

#### 2. $\omega$는 무엇인가? (팀장의 가이드라인)

팀장이 주는 가이드라인($\omega$)은 여러 형태일 수 있습니다.
*   **초기 조건**: "업무를 이 지점($\theta_0$)에서 시작하면 가장 빨리 끝낼 수 있을 거야." (MAML 방식)
*   **하이퍼파라미터**: "이 업무는 꼼꼼함(정규화)이 중요하니, 정규화 강도를 0.5로 설정해."
*   **손실 함수**: "이 업무의 목표는 단순히 정확도를 높이는 게 아니라, 안정성도 고려해야 해." 라며 목표 자체를 재정의해 줌.

#### 3. 꼭 여러 과제가 필요한가? No!

*   보통 메타러닝은 여러 팀(과제)의 성과를 보고 최고의 가이드라인을 찾지만, 꼭 그럴 필요는 없습니다.
*   **단 하나의 매우 중요한 프로젝트(single task)**가 있다면, 프로젝트를 여러 단계로 나누고 각 단계마다 팀원이 업무를 수행하게 한 뒤, 그 결과를 보고 팀장이 계속해서 가이드라인을 수정해주는 방식도 가능합니다. 이것이 **'단일 과제 메타러닝'**입니다.

### 피드-포워드 모델(Feed-Forward Model) 관점에서 본 메타러닝

앞서 살펴본 바와 같이, 식 (5)-(6)과 같은 명시적인 반복 최적화를 통하지 않고, 피드-포워드 방식으로 모델을 합성하는 여러 메타러닝 접근법이 있습니다. 
이들은 복잡도에 차이가 있지만, 이 접근법 계열을 이해하기 위해서는 식 (2)의 추상적인 목표를 구체화하여 선형 회귀 메타-훈련을 위한 간단한 예시[43]를 정의하는 것이 도움이 될 수 있습니다.

$$\min_{\omega} \underset{(\mathcal{D}^{tr}, \mathcal{D}^{val}) \in \mathcal{T}}{\mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})}} \sum_{(\mathbf{x}, y) \in \mathcal{D}^{val}} \left[ (\mathbf{x}^T \mathbf{g}_{\omega}(\mathcal{D}^{tr}) - y)^2 \right] \quad (7)$$


여기서 우리는 과제 분포에 대해 메타-훈련을 수행합니다. 각 과제에 대해 훈련 세트와 검증 세트가 주어집니다. 

+ 훈련 세트 $D^{\text{tr}}$은 벡터 $g_\omega$로 임베딩[44]되며, 이 벡터는 검증 세트의 예시 $x$를 예측하기 위한 선형 회귀 가중치를 정의합니다. 
+ 식 (7)을 최적화하는 것은 함수 $g_\omega$가 훈련 세트를 가중치 벡터로 매핑하도록 훈련함으로써 '학습하는 법을 배우는 것'입니다.
+ 따라서 $g_\omega$는 $p(\mathcal{T})$에서 추출된 새로운 메타-테스트 과제 $^{\text{te}}$에 대해서도 좋은 해법을 제공해야 합니다. 이 계열의 방법들은 사용되는 예측 모델 $g$의 복잡성과 서포트셋이 어떻게 임베딩되는지(예: 풀링, CNN, RNN 사용)[44]에 따라 다양합니다. 

이러한 모델들은 **상각(amortized)**[45] 모델로도 알려져 있는데, 이는 새로운 과제를 학습하는 비용이 $g_\omega(\cdot)$를 통한 피드-포워드 연산 한 번으로 줄어들기 때문입니다. 반복 최적화에 드는 비용은 이미 $\omega$의 메타-훈련 중에 지불되었습니다.

**comments**

이 섹션은 이전의 '이중 최적화' 방식과는 완전히 다른, **매우 빠르고 효율적인 메타러닝 방식**을 설명합니다.

#### 이전 방식 (이중 최적화)의 문제점

*   새로운 문제가 주어질 때마다, 내부 루프에서 **느린 최적화 과정(경사 하강법 등)을 여러 번 반복**해야 합니다.
*   비유: 학생이 새로운 수학 문제를 만날 때마다, 공책에 여러 번 계산을 반복하며 풀어야 합니다.

#### 새로운 방식 (피드-포워드 모델)의 아이디어

*   "느린 반복 계산 과정을 없애버리고, 그냥 **문제를 척 보면 바로 답이 나오는 '만능 공식 생성기'($g_\omega$)**를 만들자!"
*   비유: 학생이 문제 유형과 주어진 숫자들을 '만능 공식 생성기'에 넣으면, 계산 과정 없이 바로 그 문제에 맞는 '최적의 공식'이 튀어나오고, 그 공식으로 답을 구합니다.

#### 수식 (7) 해설

$$\min_{\omega} \underset{(\mathcal{D}^{tr}, \mathcal{D}^{val}) \in \mathcal{T}}{\mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})}} \sum_{(\mathbf{x}, y) \in \mathcal{D}^{val}} \left[ (\mathbf{x}^T \mathbf{g}_{\omega}(\mathcal{D}^{tr}) - y)^2 \right] \quad (7)$$

*   **$D^{\text{tr}}$ (훈련 세트)**: 학생에게 주어진 **'참고 예제'** 데이터입니다.
*   **$g_{\omega}(D^{\text{tr}})$**: 이것이 바로 **'만능 공식 생성기'**입니다. 이 함수($g$)는 참고 예제 데이터($D^{\text{tr}}$)를 입력으로 받아서, 이 문제를 푸는 데 필요한 **최적의 모델 파라미터(가중치)**를 **한 방에(피드-포워드 연산으로) 출력**합니다.
*   **$x^T g_{\omega}(D^{\text{tr}})$**: '만능 공식 생성기'가 만들어준 공식($g_{\omega}(D^{\text{tr}})$)을 가지고 실제 문제($x$)를 푸는 과정입니다.
*   **$( \dots - y)^2$**: 예측값과 실제 정답($y$) 사이의 오차입니다.
*   **전체 의미**: 여러 종류의 과제에 대해, 
"주어진 참고 예제($D^{\text{tr}}$)를 보고 최적의 공식($g_{\omega}(D^{\text{tr}})$)을 한 번에 만들어내는 생성기($g_\omega$)를 훈련시켜라!
이 생성기는 어떤 문제가 주어져도 항상 좋은 공식을 만들어내야 한다."

#### 상각(Amortized)이라는 용어의 의미

*   **'상각(Amortize)'**은 회계 용어로, 큰 비용을 여러 기간에 걸쳐 나누어 처리한다는 의미입니다.
*   메타러닝에서 이 용어는, **가장 비용이 많이 드는 '느린 반복 최적화' 과정을 메타-훈련 때 미리 다 해치워버리고($\omega$를 학습),** 정작 새로운 문제를 풀 때는 그 비용을 거의 치르지 않는다는 의미에서 사용됩니다.
*   **메타-훈련 (비용이 비쌈)**: 수많은 과제를 풀어보며 '만능 공식 생성기'($g_\omega$)를 만드는 데는 엄청난 시간과 계산이 필요합니다. (비용을 미리 지불)
*   **메타-테스트 (비용이 거의 0)**: 일단 생성기만 만들어지면, 새로운 문제는 그냥 함수에 데이터 한 번 넣는 것으로 끝나므로 거의 즉시 해결됩니다. (미리 지불한 비용의 혜택을 봄)

이 피드-포워드 방식은 특히 **새로운 문제에 대한 반응 속도가 매우 빨라야 하는 응용 분야**에 매우 유용합니다.

## 메타러닝의 역사적 배경

메타러닝과 러닝-투-런(learning-to-learn)은 1987년 문헌에 처음 등장합니다. 

**위르겐 슈미트후버(J. Schmidhuber)**
+ **자기 참조 학습(self-referential learning)**을 사용하여 '학습하는 방법을 학습'할 수 있는 방법론의 한 갈래를 소개했습니다. 자기 참조 학습은 신경망이 자신의 가중치를 입력으로 받아 해당 가중치에 대한 업데이트 값을 예측하도록 훈련하는 것을 포함합니다. 슈미트후버는 진화 알고리즘을 사용하여 모델 자체를 학습할 것을 제안했습니다.

메타러닝은 이후 여러 분야로 확장되었습니다. 

**요슈아 벤지오(Bengio)**
+ **생물학적으로 타당한 학습 규칙(biologically plausible learning rules)**을 메타-학습하는 방법을 제안했습니다. 
+ 슈미트후버 등은 자기 참조 시스템과 메타러닝을 계속해서 탐구했습니다. 

**세바스찬 스런(S. Thrun)**
+ 'learning to learn'이라는 용어를 더 명확하게 정의하는 데 힘썼으며, 초기의 이론적 정당성과 실용적인 구현 방법을 소개했습니다. 
+ 경사 하강법과 역전파를 사용하여 메타러닝 시스템을 훈련시키자는 제안은 1991년에 처음으로 나왔고, 2001년에 더 확장된 연구들이 뒤따랐으며 당시 문헌에 대한 개요를 제공합니다. 메타러닝은 1995년에 강화 학습의 맥락에서 사용되었고, 이후 다양한 확장 연구가 이어졌습니다.

**comments**

이 섹션은 메타러닝의 아이디어가 어떻게 시작되고 발전했는지를 시간 순서대로 보여줍니다.

*   **1987년, 슈미트후버의 시작**:
    *   **"자기 참조 학습(Self-referential Learning)"**이라는 혁신적인 아이디어를 제시합니다.
    *   **아이디어**: 보통 신경망은 외부 데이터(예: 이미지)를 입력받습니다. 하지만 슈미트후버는 신경망이 **자기 자신의 설계도(가중치)**를 들여다보고, "내 설계도를 어떻게 수정해야 더 똑똑해질까?"를 **스스로 예측**하게 만들자고 제안했습니다. 마치 AI가 자기 자신을 성찰하고 개선하는 것과 같습니다.
    *   **방법**: 당시에는 경사 하강법으로 이런 복잡한 구조를 훈련하기 어려웠기 때문에, 다윈의 진화론처럼 좋은 해결책은 살아남고 나쁜 해결책은 도태되는 **진화 알고리즘**을 사용하자고 제안했습니다.

*   **1990년대 초, 벤지오와 스런의 발전**:
    *   **벤지오**: "실제 뇌가 학습하는 방식과 유사한, **생물학적으로 그럴듯한 학습 규칙**을 AI가 스스로 배우게 만들 수는 없을까?"라는 방향으로 연구를 확장했습니다.
    *   **스런**: 'learning-to-learn'이 무엇인지 개념적으로 명확하게 다듬고, "이것이 왜 이론적으로 타당한가?"에 대한 근거를 제시하며 연구의 기틀을 다졌습니다.

*   **1991년, 경사 하강법의 도입**:
    *   "진화 알고리즘 말고, 우리가 신경망 훈련에 흔히 쓰는 **경사 하강법과 역전파**를 메타러닝에도 적용해보자!"라는 제안이 처음 등장합니다. 이것이 현대 메타러닝 방법론의 시초가 됩니다. (예: MAML 같은 알고리즘의 먼 조상)

*   **1995년, 강화 학습과의 만남**:
    *   메타러닝은 지도 학습뿐만 아니라, 로봇 제어나 게임처럼 시행착오를 통해 학습하는 **강화 학습(RL)** 분야에도 적용되기 시작하며 그 활용 범위를 넓혔습니다.

## 관련 분야

여기서는 메타러닝과 자주 혼동을 일으키는 관련 분야들과 비교하여 메타러닝의 위상을 정립합니다.

### 전이 학습 (Transfer Learning, TL)

전이 학습(TL)은 소스 과제(source task)의 과거 경험을 사용하여 타겟 과제(target task)의 학습(속도, 데이터 효율성, 정확도)을 향상시키는 것을 목표로 합니다. TL은 이러한 문제 영역과 해결책 계열을 모두 지칭하며, 가장 흔한 방법은 파라미터 전이 후 선택적으로 미세 조정(fine tuning)을 하는 것입니다 (물론 다른 수많은 접근법도 있습니다[34]).

> [34] survey: S. J. Pan and Q. Yang, “A Survey On Transfer Learning,” IEEE TKDE, 2010.

반면, 메타러닝은 다른 문제들뿐만 아니라 TL을 개선하는 데 사용될 수 있는 패러다임입니다. TL에서는 **메타-목적(meta-objective)을 사용하지 않고** 소스 과제에 대한 일반적인(vanilla) 학습을 통해 사전 지식(prior)을 추출합니다. 

메타러닝에서는, MAML[16]에서 보여주듯이, 

+ 새로운 과제를 학습할 때 사전 지식의 이점을 평가하는 **외부 최적화(outer optimization)**를 통해 해당 사전 지식이 정의됩니다. 
+ 더 일반적으로, 메타러닝은 단지 모델 파라미터뿐만 아니라 훨씬 더 넓은 범위의 meta-representation을 다룹니다 (섹션 4.1 참조).

> [16] MAML: C.Finn,P. Abbeel, and S. Levine, “Model-Agnostic Meta-learning For Fast Adaptation Of Deep Networks,” in ICML, 2017.

**comments**

*   **전이 학습(TL)이란?**: 어떤 분야에서 얻은 지식을 다른 분야에 써먹는 것입니다.
*   **메타러닝과의 핵심 차이**: **'어떻게' 지식을 옮길지를 최적화하는가?**
    *   **전이 학습**: 일단 A라는 과제를 열심히 공부해서 지식을 쌓고 (예: 이미지넷으로 모델 훈련), 그 지식을 B라는 과제에 가져가서 약간 수정해서(fine-tuning) 씁니다. '어떻게 하면 B에 더 잘 써먹을 수 있을까?'를 고민하며 A를 공부하지는 않습니다.
    *   **메타러닝**: "나중에 B, C, D 같은 과제들을 **가장 빠르고 쉽게 배울 수 있도록**, 지금 A를 **어떤 방식으로** 공부해 둬야 할까?"라는 **'학습 전략 자체'를 최적화**합니다.

### 도메인 적응(Domain Adaptation, DA) 및 도메인 일반화(Domain Generalization, DG)

**도메인 이동(Domain-shift)**

+ 소스 문제와 타겟 문제가 동일한 목표를 공유하지만, 타겟 문제의 입력 데이터 분포가 소스 과제와 달라 모델 성능이 저하되는 상황을 말합니다[34], [58]. 

**DA**

+ 타겟 도메인의 희소하거나 레이블이 없는 데이터를 사용하여 소스에서 훈련된 모델을 조정함으로써 이 문제를 완화하려는 전이 학습의 한 변형입니다. 

**DG**

+ 추가적인 적응 없이 이러한 도메인 이동에 강건한 소스 모델을 훈련시키는 방법을 말합니다.

TL과 마찬가지로, 일반적인(vanilla) DA와 DG는 여러 도메인에 걸쳐 '학습하는 방법'을 최적화하기 위한 메타-목적을 사용하지 않습니다. 반면, 메타러닝 방법은 DA[59]와 DG[42]를 모두 수행하는 데 사용될 수 있습니다 (섹션 5.8 참조).

**comments**

*   **DA/DG란?**: 같은 문제인데 데이터의 '스타일'이 다른 상황에 대처하는 것입니다. (예: 낮에 찍은 사진으로 학습한 자율주행차가 밤에도 운전해야 하는 상황)
*   **메타러닝과의 핵심 차이**: **'스타일 변화에 대한 강건함'을 직접적인 목표로 최적화하는가?**
    *   **DA/DG**: 특정 기술(예: 밤 사진을 낮 사진처럼 보이게 변환)을 사용하여 스타일 차이를 줄이려고 노력합니다. '어떻게 훈련해야 어떤 스타일 변화에도 강해질까?'를 최적화하지는 않습니다.
    *   **메타러닝**: 훈련 단계에서 일부러 **다양한 스타일의 데이터(낮, 밤, 비 오는 날, 흐린 날)를 경험**하게 하고, "어떤 스타일의 데이터가 들어와도 성능이 떨어지지 않는 **'강건한 학습법'**" 자체를 찾도록 최적화합니다.

### 연속 학습 (Continual Learning, CL)

연속 또는 평생 학습(lifelong learning)[60]-[62]은 잠재적으로 비정상(non-stationary) 분포에서 추출된 일련의 과제들을 학습하는 능력을 말하며, 특히 새로운 과제를 더 빠르게 배우면서 오래된 과제는 잊지 않는 것을 목표로 합니다. 

메타러닝과 유사하게 과제 분포가 고려되며, 목표의 일부는 타겟 과제의 학습을 가속화하는 것입니다. 그러나 대부분의 연속 학습 방법론은 이 메타-목표를 명시적으로 풀지 않기 때문에 메타러닝 방법론이 아닙니다. 반면에, 메타러닝은 연속 학습을 발전시키기 위한 잠재적인 프레임워크를 제공하며, 최근 몇몇 연구들은 연속 학습 성능을 인코딩하는 메타-목표를 개발함으로써 이를 시도하기 시작했습니다[63]–[65].

**comments**

*   **연속 학습(CL)이란?**: 과제 A를 배운 뒤, 과제 B를 배우고, 이어서 과제 C를 배울 때, 이전에 배운 A와 B를 잊어버리지 않는(catastrophic forgetting 방지) 능력입니다.
*   **메타러닝과의 핵심 차이**: **'연속 학습 자체'를 하나의 학습 목표로 삼는가?**
    *   **연속 학습**: 특정 기술(예: 중요한 지식은 얼려서 보존)을 사용하여 과거의 지식을 잊지 않으려고 노력합니다.
    *   **메타러닝**: "어떻게 훈련해야 **'새로운 것을 배우면서도 옛것을 잊지 않는 능력'** 자체를 극대화할 수 있을까?"라는 메타-목표를 설정하고, 이 목표를 달성하는 학습 전략을 찾습니다.


### 다중과제 학습 (Multi-Task Learning, MTL)

다중과제 학습(MTL)은 여러 관련 과제를 동시에 학습하여, 파라미터 공유와 그로 인한 공유 표현의 다양성 덕분에 정규화 효과를 보고[66]–[68], 컴퓨팅/메모리 절약 효과를 얻는 것을 목표로 합니다. TL, DA, CL과 마찬가지로, 전통적인 MTL은 메타-목적이 없는 단일-수준 최적화입니다. 

더욱이, MTL의 목표는 **이미 알려진 고정된 수의 과제**를 푸는 것인 반면, 메타러닝의 핵심은 종종 **미래에 보게 될 미지의 과제**를 푸는 것입니다. 그럼에도 불구하고, 메타러닝은 MTL에 이점을 가져다줄 수 있습니다. 예를 들어, 과제 간의 관련성을 학습하거나[69] 여러 과제 간의 우선순위를 정하는 방법을 배우는 것[70]입니다.

**comments**

*   **다중과제 학습(MTL)이란?**: 여러 과제를 **동시에** 학습시켜서 서로 도움을 주고받게 하는 것입니다.
*   **메타러닝과의 핵심 차이**: **'알려진 과제' vs '미지의 과제'**
    *   **다중과제 학습**: 목표는 **"주어진 A, B, C 과제 모두"**에서 최고의 성능을 내는 것입니다. 미래에 D라는 과제가 나타날 것은 고려하지 않습니다.
    *   **메타러닝**: A, B, C 과제를 학습하는 이유는 **"미래에 나타날 미지의 과제 D"**를 더 잘 풀기 위함입니다. A, B, C 자체의 성능보다는 일반화 능력이 더 중요합니다.

### 하이퍼파라미터 최적화 (Hyperparameter Optimization, HO)

하이퍼파라미터 최적화(HO)는 학습률이나 정규화 강도와 같은 하이퍼파라미터가 '학습하는 방법'을 기술한다는 점에서 메타러닝의 영역에 속합니다. 여기서 우리는 경사 하강법 기반 하이퍼파라미터 학습[69], [71] 및 신경망 구조 탐색[18]과 같이 신경망으로 종단간(end-to-end) 훈련되는 메타-목표를 정의하는 HO 과제들을 포함합니다. 그러나 우리는 무작위 탐색[72]이나 베이지안 하이퍼파라미터 최적화[73]와 같이, 메타러닝으로 거의 간주되지 않는 다른 접근법들은 제외합니다.

**comments**

*   **하이퍼파라미터 최적화(HO)란?**: 모델 학습에 영향을 주는 설정값(학습률, 층의 개수 등)들을 사람이 아니라 기계가 자동으로 찾게 하는 것입니다.
*   **메타러닝과의 관계**: '학습하는 방법($\omega$)'을 찾는다는 점에서 메타러닝과 매우 유사합니다.
*   **이 논문에서의 구분 기준**: **'신경망'과 '종단간 최적화'를 사용하는가?**
    *   **메타러닝으로 간주**: 학습률이나 모델 구조 자체를 또 다른 신경망을 이용해, 전체 학습 과정에 대한 경사 하강법으로 최적화하는 방식. (과정이 매끄럽게 연결됨)
    *   **메타러닝에서 제외**: 단순히 여러 하이퍼파라미터 조합을 무작위로 시도해 보거나(무작위 탐색), 통계적 추정(베이지안 최적화)을 통해 최적값을 탐색하는 전통적인 방식. (학습 과정과 분리되어 있음)

### 계층적 베이즈 모델 (Hierarchical Bayesian Models, HBM)

계층적 베이즈 모델(HBM)은 사전 확률 $p(\theta|\omega)$ 하에서 파라미터 $\theta$의 베이지안 학습을 포함합니다. 

이 사전 확률은 자체적인 사전 확률 $p(\omega)$를 갖는 다른 변수 $\omega$에 대한 조건부 밀도 함수로 작성됩니다. 

계층적 베이즈 모델은 $D = \{D_i | i = 1, 2, ..., M\}$과 같이 그룹화된 데이터 모델로 강력하게 사용되며, 각 그룹 $i$는 자체적인 $\theta_i$를 가집니다. 

전체 모델은 아래와 같습니다.

$$[\prod_{i=1}^{M} P(D_i|\theta_i) p(\theta_i|\omega)] p(\omega)$$

계층 구조는 더 확장될 수 있으며, 특히 $\omega$ 자체가 매개변수화되어 $p(\omega)$가 학습될 수 있습니다. 

학습은 보통 전체 파이프라인에 걸쳐 이루어지지만, 베이지안 주변화(Bayesian marginalisation)의 한 형태를 사용하여 $\omega$에 대한 사후 확률을 계산하기도 합니다.

$$P(\omega|D) \propto p(\omega) \prod_{i=1}^{M} \int d\theta_i p(D_i|\theta_i)p(\theta_i|\omega)$$

주변화를 수행하는 용이성은 모델에 따라 다릅니다. 

+ 어떤 모델(예: 잠재 디리클레 할당[74])에서는 켤레 지수 모델(conjugate exponential models) 선택 덕분에 주변화가 정확하지만, 
+ 다른 모델(예: [75])에서는 확률적 변분 추론(stochastic variational approach)을 사용하여 근사적인 사후 확률을 계산하고, 이를 통해 주변 우도(marginal likelihood)의 하한(lower bound)을 계산합니다.

베이지안 계층 모델은 메타러닝 과정을 이해하기 위한 알고리즘적 프레임워크보다는 모델링 프레임워크를 제공함으로써, 메타러닝에 대한 가치 있는 관점을 제공합니다. 실제로는, HBM에 대한 이전 연구들은 주로 다루기 쉬운 간단한 모델 $\theta$를 학습하는 데 초점을 맞춘 반면, 대부분의 메타러닝 연구는 여러 번의 반복을 포함하는 복잡한 내부-루프 학습 과정을 고려합니다. 그럼에도 불구하고, MAML[16]과 같은 일부 메타러닝 방법은 HBM의 렌즈를 통해 이해될 수 있습니다[76].

**comments**

*   **계층적 베이즈 모델(HBM)이란?**: "개별 그룹(과제)의 특성($\theta_i$)은 전체 그룹(과제 분포)의 공통적인 특성($\omega$)으로부터 나온다"는 계층적 가정을 사용하는 통계 모델입니다.
*   **메타러닝과의 관계**: **구조가 매우 유사합니다.**
    *   `전체 그룹 특성(ω)` ↔ `메타 지식(ω)`
    *   `개별 그룹 특성(θ_i)` ↔ `개별 과제 모델(θ_i)`
*   **핵심 차이점**: **'관점'과 '복잡성'**
    *   **관점**: HBM은 "이 데이터가 어떻게 생성되었을까?"를 확률적으로 **모델링**하는 데 초점을 맞춥니다. 반면, 메타러닝은 "어떻게 하면 성능을 최적화할까?"라는 **알고리즘적** 관점에 더 가깝습니다.
    *   **복잡성**: 전통적인 HBM은 수학적으로 다루기 쉬운 간단한 모델에 주로 사용된 반면, 현대 메타러닝은 수백만 개의 파라미터를 가진 복잡한 딥러닝 모델의 학습 과정을 다룹니다.
*   **결론**: HBM은 메타러닝의 철학적, 구조적 배경을 이해하는 데 훌륭한 이론적 틀을 제공하지만, 오늘날 딥러닝에서 다루는 문제의 규모와 복잡성에는 직접 적용하기 어렵습니다.


### AutoML

AutoML[31]-[33]은 데이터 준비, 알고리즘 선택, 하이퍼파라미터 튜닝, 구조 탐색과 같이 일반적으로 수동으로 이루어지는 **머신러닝 과정의 일부를 자동화하려는 접근법들을 포괄하는 다소 넓은 분야를 통칭하는 말**입니다. 

AutoML은 종종 여기서 정의한 메타러닝의 범위를 벗어나는 수많은 휴리스틱을 사용하며, 데이터 정제와 같이 메타러닝에서는 덜 중심적인 과제에 초점을 맞춥니다. 

하지만, AutoML은 때때로 메타-목표의 종단간 최적화를 사용하기도 하므로, 메타러닝은 **AutoML의 한 전문 분야(specialization)**로 볼 수 있습니다.

**comments**

*   **AutoML이란?**: 머신러닝의 A부터 Z까지(데이터 정제, 모델 선택, 튜닝 등) 모든 과정을 자동화하려는 기술 분야입니다.
*   **메타러닝과의 관계**:
    *   **AutoML이 더 넓은 개념**: AutoML은 '자동화'라는 목표를 위해 메타러닝뿐만 아니라 온갖 종류의 기법(휴리스틱, 탐색 알고리즘 등)을 모두 사용합니다.
    *   **메타러닝은 AutoML의 한 도구**: AutoML이 여러 단계를 자동화할 때, 특히 '최적의 학습 전략을 찾는' 부분에서 메타러닝의 아이디어(메타-목표의 종단간 최적화)를 강력한 도구로 사용할 수 있습니다.
*   **결론**: 메타러닝은 **AutoML이라는 거대한 목표를 달성하기 위한, 특히 '학습 원리'에 초점을 맞춘 정교하고 전문화된 방법론 중 하나**라고 볼 수 있습니다.

# Taxonomy


## 이전의 분류 체계들

이전의[77], [78] 메타러닝 방법론 분류는 주로 

(1) **최적화 기반(optimization-based)** 방법, 
(2) **모델 기반(model-based)** (또는 블랙박스) 방법, 
(3) **측정 기반(metric-based)** (또는 비모수적) 방법

과 같은 세 갈래의 분류 체계를 따르는 경향이 있었습니다.

### 최적화 (Optimization)

$$
\omega^* = \arg\min_{\omega} \sum_{i=1}^{M} \mathcal{L}^{\text{meta}}(\theta^{*(i)}(\omega), \omega, D_{\text{source}}^{\text{val }(i)}) \quad (5)
$$

$$
\text{s.t. } \theta^{*(i)}(\omega) = \arg\min_{\theta} \mathcal{L}^{\text{task}}(\theta, \omega, D^{\text{train }(i)}_{\text{source}}) \quad (6)
$$

최적화 기반 방법은 내부 수준의 과제(식 6)가 말 그대로 최적화 문제로 해결되며, 최적화 성능을 향상시키는 데 필요한 메타 지식 $\omega$를 추출하는 데 초점을 맞춥니다. 

+ 유명한 예시인 MAML[16]은 초기화 $\omega = \theta_0$를 학습하여, 적은 수의 내부 스텝만으로도 검증 데이터에서 좋은 성능을 내는 분류기를 만들도록 하는 것을 목표로 합니다. 

이 과정은 기반 모델의 업데이트 과정을 미분함으로써 경사 하강법으로 수행됩니다. 더 정교한 대안들은 스텝 사이즈(step sizes)[79], [80]를 학습하거나, gradient로부터 스텝을 예측하는 순환 신경망(recurrent networks)을 훈련시키기도 합니다[19], [39], [81]. 긴 내부 최적화 과정에 대한 gradient를 이용한 메타-최적화는 여러 계산 및 메모리 문제를 야기하며, 이는 섹션 6에서 논의됩니다. 
기존의 많은 방법들을 일반화된 내부 루프 메타러닝 프레임워크의 특수한 경우로 표현하는, gradient 기반 메타러닝에 대한 통합된 관점이 제안된 바 있습니다[82].



### 블랙박스 / 모델 기반 (Black Box / Model-based)

$$
\theta^{*(i)} = \arg\max_{\theta} \log p(\theta|\omega^*, D_{\text{target}}^{\text{train }(i)}) \quad (4)
$$

$$
\text{s.t. } \theta^{*(i)}(\omega) = \arg\min_{\theta} \mathcal{L}^{\text{task}}(\theta, \omega, D^{\text{train }(i)}_{\text{source}}) \quad (6)
$$

모델 기반(또는 블랙박스) 방법에서는 내부 학습 스텝(식 6, 식 4)이 단일 모델의 피드-포워드(feed-forward) 패스 안에 전부 포함되며, 이는 식 (7)에서 설명된 바와 같습니다. 

$$\min_{\omega} \underset{(\mathcal{D}^{tr}, \mathcal{D}^{val}) \in \mathcal{T}}{\mathbb{E}_{\mathcal{T} \sim p(\mathcal{T})}} \sum_{(\mathbf{x}, y) \in \mathcal{D}^{val}} \left[ (\mathbf{x}^T \mathbf{g}_{\omega}(\mathcal{D}^{tr}) - y)^2 \right] \quad (7)$$

이 모델은 현재 데이터셋 $D$를 활성화 상태(activation state)로 임베딩하고, 이 상태를 기반으로 테스트 데이터에 대한 예측을 수행합니다. 

+ 대표적인 구조로는 훈련 인스턴스와 레이블을 임베딩하여 테스트 샘플에 대한 예측기를 정의하는 순환 신경망[39], [51], 합성곱 신경망[38] 또는 하이퍼네트워크[83], [84]가 있습니다. 
+ 이 경우, 모든 내부 수준의 학습은 모델의 활성화 상태에 포함되며 전적으로 피드-포워드 방식입니다. 
+ 외부 수준의 학습은 CNN, RNN 또는 하이퍼네트워크 파라미터를 포함하는 $\omega$로 수행됩니다. 
+ 외부와 내부 수준의 최적화는 $\omega$와 $D$가 직접적으로 $\theta$를 명시하기 때문에 긴밀하게 결합되어 있습니다. 

메모리 증강 신경망(Memory-augmented neural networks)[85]은 명시적인 저장 버퍼를 사용하며, 모델 기반 방법의 한 종류로 볼 수 있습니다[86], [87]. 최적화 기반 접근법과 비교할 때, 이들은 2차 미분(second-order gradients)을 요구하지 않는 더 간단한 최적화를 즐깁니다. 

그러나 **모델 기반 접근법은 최적화 기반 방법보다 분포를 벗어난(out-of-distribution) 과제에 대해 일반화하는 능력이 보통 더 떨어진다**고 관찰되었습니다[88]. 

더욱이, 이들은 데이터 효율적인 퓨샷 학습에는 매우 능숙하지만, 풍부한 기반 모델에 대규모 훈련 세트를 임베딩하는 데 어려움을 겪기 때문에 점근적으로는(asymptotically) 더 약하다는 비판을 받아왔습니다[88].

### 측정 학습 (Metric-Learning)

측정 학습 또는 비모수적 알고리즘은 지금까지 메타러닝의 인기 있지만 특수한 응용 분야인 퓨샷(Section 5.1.1)에 주로 국한되어 왔습니다. 
아이디어는 내부 (과제) 수준에서 검증 포인트를 훈련 포인트와 단순히 비교하고, 일치하는 훈련 포인트의 레이블을 예측함으로써 비모수적인 '학습'을 수행하는 것입니다. 

시간 순서대로, 이는 샴(siamese)[89], 매칭(matching)[90], 프로토타입(prototypical)[20], 관계(relation)[91], 그래프[92] 신경망으로 달성되었습니다. 

여기서 외부 수준의 학습은 데이터를 비교에 적합하게 표현하는 특징 추출기 $\omega$를 찾는 측정 학습에 해당합니다. 이전과 마찬가지로 $\omega$는 소스 과제에서 학습되고, 타겟 과제에 사용됩니다.


### comments

과거에는 메타러닝 방법들을 크게 세 가지 스타일로 나누어 설명했습니다.

#### 1. 최적화 기반: "가장 좋은 출발점/방법을 찾아라!"

*   **핵심 아이디어**: 새로운 문제를 만났을 때, **가장 빨리 정답에 도달할 수 있는 '최적화 과정' 자체를 학습**합니다.
*   **대표 주자 (MAML)**: "어떤 **출발점(초기 가중치 $\theta_0$)**에서 시작해야, 경사 하강법을 몇 걸음만 가도 바로 정답 근처에 도달할까?"를 학습합니다. 마치 산 정상으로 가는 가장 좋은 베이스캠프 위치를 찾는 것과 같습니다.
*   **장점**: 원리가 명확하고, 다양한 문제에 적용할 수 있는 일반성이 높습니다.
*   **단점**: 학습 과정(최적화 과정)을 통째로 미분해야 해서 **계산이 매우 복잡하고 느립니다.** (미분의 미분을 계산해야 할 수도 있음)

#### 2. 모델 기반 (블랙박스): "문제를 척 보면 답이 나오는 만능 함수를 만들어라!"

*   **핵심 아이디어**: 느린 최적화 과정을 없애고, **문제 데이터 자체를 입력받으면 모델 파라미터나 예측값을 한 방에 출력하는 거대한 신경망 하나**를 만듭니다.
*   **작동 방식**: RNN이나 CNN 같은 모델이 문제의 예시 데이터들을 '읽고' 그 정보를 자신의 메모리(활성화 상태)에 저장한 뒤, 새로운 질문이 들어오면 메모리를 참고해서 즉시 답을 내놓습니다.
*   **장점**: 새로운 문제에 대한 **반응 속도가 매우 빠릅니다.** (피드-포워드 한 번이면 끝) 계산도 훨씬 간단합니다.
*   **단점**:
    *   훈련 때 본 적 없는 **생소한 유형의 문제(out-of-distribution)에는 매우 취약**합니다. (마치 암기 과목만 잘하는 학생 같음)
    *   데이터가 아주 많아지면, 그 많은 정보를 작은 메모리에 다 담기 어려워져서 성능 한계에 부딪힐 수 있습니다.

#### **3. 측정 기반: "비슷한 놈들끼리 묶어라!"**

*   **핵심 아이디어**: "학습"이란 결국 **데이터 간의 '거리'나 '유사도'를 잘 재는 방법을 배우는 것**과 같다는 철학입니다.
*   **작동 방식**:
    1.  신경망을 이용해 모든 데이터를 **'특징 공간'이라는 지도**에 점으로 표시합니다.
    2.  이때, 같은 클래스(예: 고양이)의 데이터는 서로 가깝게, 다른 클래스(예: 개)의 데이터는 서로 멀리 떨어지도록 신경망(특징 추출기 $\omega$)을 훈련시킵니다.
    3.  새로운 데이터가 들어오면, 지도 위에서 **가장 가까운 이웃이 누구인지 보고** 그 이웃의 클래스를 따라갑니다.
*   **장점**: 개념이 직관적이고, 특히 **퓨샷 학습(few-shot learning)**처럼 데이터가 매우 적을 때 강력한 성능을 보입니다.
*   **단점**: 퓨샷 분류라는 특정 문제 외의 다른 다양한 메타러닝 문제에는 적용하기 어렵습니다.

## 제안하는 분류 체계

우리는 세 개의 독립적인 축을 따라 새로운 분류를 도입합니다. 각 축에 대해, 우리는 현재 메타러닝의 지형을 반영하는 분류 체계를 제공합니다.

### 메타-표현 ("무엇을?" / "What?")
첫 번째 축은 메타-학습할 **메타 지식 $\omega$의 선택**입니다. 이는 초기 모델 파라미터[16]부터 프로그램 귀납(program induction)의 경우 읽을 수 있는 코드[93]에 이르기까지 무엇이든 될 수 있습니다.

### 메타-optimzer ("어떻게?" / "How?")
두 번째 축은 메타-훈련 중에 외부 수준에서 사용할 **optimzer의 선택**입니다 (식 5 참조). $\omega$에 대한 외부 수준 optimzer는 경사 하강법[16]부터 강화 학습[93] 및 진화 탐색[23]에 이르기까지 다양한 형태를 취할 수 있습니다.

### 메타-목적 ("왜?" / "Why?")
세 번째 축은 **메타-목적 함수 $\mathcal{L}_{\text{meta}}$(식 5), 과제 분포 $p(\mathcal{T})$, 그리고 두 수준 간의 데이터 흐름**의 선택에 의해 결정되는 메타러닝의 목표입니다. 이들을 종합하여 샘플 효율적인 퓨샷 학습[16], [38], 빠른 다중샷(many-shot) 최적화[93], [94], 도메인 이동에 대한 강건성[42], [95], 레이블 노이즈[96], 그리고 적대적 공격[97]과 같은 다양한 목적을 위해 메타러닝을 맞춤화할 수 있습니다.

이 축들은 함께 새로운 알고리즘의 개발과 특정 응용 분야에 대한 맞춤화를 이끌 수 있는 메타러닝 방법론의 **설계 공간(design-space)**을 제공합니다. 기반 모델 표현 $\theta$는 이 분류 체계에 포함되지 않는데, 이는 그것이 당면한 응용 분야에 특화된 방식으로 결정되고 최적화되기 때문입니다.

### comments

이 논문의 저자들은 기존의 3가지 분류(최적화/모델/측정 기반)가 너무 단순해서 현대의 복잡한 메타러닝 연구들을 제대로 설명하지 못한다고 생각했습니다. 그래서 **"메타러닝 알고리즘을 설계할 때 우리가 내려야 하는 3가지 핵심 결정"**이라는 새로운 기준을 제시합니다.

#### **1. 메타-표현 (Meta-Representation): "무엇을" 학습할 것인가?**

*   **핵심 질문**: 우리가 '학습하는 법을 배운다'고 할 때, 그 '법'의 **실체($\omega$)가 무엇인가?**
*   **선택지 (예시)**:
    *   **초기 파라미터**: 가장 좋은 '출발점'을 배운다. (MAML)
    *   **optimzer**: 가장 효율적인 '업데이트 규칙'을 배운다.
    *   **특징 추출기**: 데이터의 핵심을 가장 잘 꿰뚫어 보는 '눈'을 배운다. (측정 기반 학습)
    *   **모델 생성기**: 문제를 보면 바로 모델을 '뚝딱' 만들어내는 함수를 배운다. (모델 기반 학습)
    *   **프로그램 코드**: 심지어 사람이 읽을 수 있는 파이썬 코드 자체를 생성하도록 학습할 수도 있다.

#### **2. 메타-optimzer (Meta-Optimizer): "어떻게" 학습시킬 것인가?**

*   **핵심 질문**: 위에서 정한 '무엇($\omega$)'을 **어떤 방법으로 최적화**할 것인가? (외부 루프의 최적화 방식)
*   **선택지 (예시)**:
    *   **경사 하강법**: 가장 일반적인 방법. 미분을 통해 점진적으로 개선한다.
    *   **강화 학습**: 미분이 불가능할 때, 여러 시도를 해보고 '보상'이 가장 큰 방향으로 학습한다.
    *   **진화 알고리즘**: 여러 후보($\omega$)를 만들어 경쟁시키고, 가장 좋은 놈만 살아남게 하는 '적자생존' 방식으로 학습한다.

#### **3. 메타-목적 (Meta-Objective): "왜" 학습하는가? (최종 목표는 무엇인가?)**

*   **핵심 질문**: 이 메타러닝을 통해 **궁극적으로 달성하고 싶은 목표**는 무엇인가?
*   **선택지 (예시)**:
    *   **퓨샷 학습**: 아주 적은 데이터만으로도 빠르게 학습하는 능력.
    *   **빠른 수렴**: 데이터가 많더라도, 최대한 빨리 최적의 성능에 도달하는 능력.
    *   **강건성 (Robustness)**: 데이터에 노이즈가 있거나, 훈련 환경과 테스트 환경이 달라도 성능이 떨어지지 않는 '맷집'.
    *   **적대적 방어**: 누군가 악의적으로 데이터를 변조해도 속지 않는 능력.

이 세 가지 축을 어떻게 조합하느냐에 따라 무수히 많은 종류의 새로운 메타러닝 알고리즘을 설계할 수 있다는 것이 이 논문이 제안하는 새로운 관점입니다.

# SURVEY: METHODOLOGIES 

이 섹션에서는 우리가 제안한 새로운 방법론적 분류 체계에 따라 기존 문헌을 세분화하여 분석합니다.

## 메타-표현 (Meta-Representation)

메타러닝 방법들은 메타 지식 $\omega$가 무엇이어야 하는지, 즉 학습 전략의 어떤 측면을 학습하고, (배제함으로써) 어떤 측면을 고정된 것으로 간주해야 하는지에 대해 서로 다른 선택을 합니다.

### 파라미터 초기화 (Parameter Initialization)

여기서 $\omega$는 내부 최적화에서 사용될 신경망의 **초기 파라미터**에 해당하며, MAML이 가장 대표적인 예시입니다[16], [98], [99]. 

좋은 초기화는 과제 분포 $p(\mathcal{T})$에서 추출된 어떤 과제 $T$의 해답으로부터 단 몇 번의 경사 하강 스텝만으로 도달할 수 있는 위치에 있으며, 퓨샷 학습에서 과적합 없이 학습하는 데 도움을 줄 수 있습니다. 

+ 이 접근법의 핵심적인 난제는 외부 최적화가 내부 최적화만큼이나 많은 파라미터(대형 CNN의 경우 잠재적으로 수억 개)를 풀어야 한다는 점입니다. 
+ 이로 인해 부분 공간(subspace)[78], [100], 레이어별[83], [100], [101], 또는 스케일과 시프트 분리[102] 등을 통해 메타-학습할 파라미터의 일부를 분리하려는 연구 흐름이 생겨났습니다. 
+ 또 다른 우려는 단일 초기 조건이 광범위한 잠재적 과제에 대해 빠른 학습을 제공하기에 충분한지, 아니면 좁은 분포 $p(\mathcal{T})$에 국한되는지에 대한 것입니다. 이는 여러 초기 조건에 대한 혼합(mixtures)을 모델링하는 변형 연구들로 이어졌습니다[100], [103], [104].

**comments**

*   **무엇을 배우는가?**: **"최고의 출발점"**을 배웁니다.
*   **비유**: 산 정상으로 가는 가장 빠른 길을 찾기 위해, 모든 등산로에서 가장 접근하기 좋은 **'만능 베이스캠프'**의 위치($\omega = \theta_0$)를 찾는 것과 같습니다.
*   **문제점**: 베이스캠프의 위치를 정하는 것(외부 최적화)이 등산 자체(내부 최적화)만큼이나 어렵고 복잡합니다. (파라미터 수가 너무 많음)
*   **해결책**:
    1.  **일부만 배우기**: 전체 파라미터가 아니라, 가장 중요한 일부 레이어나 특정 부분만 학습하자.
    2.  **여러 개 배우기**: '만능 베이스캠프' 하나 대신, '한라산용 베이스캠프', '설악산용 베이스캠프' 등 여러 개의 좋은 출발점을 배우자.



### Optimizer

위의 파라미터 중심 방법들은 보통 새로운 과제가 주어졌을 때 초기화를 개선하기 위해 모멘텀이 있는 SGD나 Adam[105]과 같은 기존의 optimzer에 의존합니다. 

대신, optimzer 중심 접근법[19], [39], [81], [94]은 $\theta$와 
$\nabla_{\theta}\mathcal{L}_{\text{task}}$
와 같은 최적화 상태를 입력으로 받아 각 기반 학습 반복에 대한 최적화 스텝을 생성하는 함수를 훈련함으로써 **내부 optimzer를 학습**하는 데 초점을 맞춥니다. 

훈련 가능한 구성 요소 $\omega$는 고정된 스텝 사이즈[79], [80]와 같은 간단한 하이퍼파라미터부터 더 정교한 사전 조건화 행렬(pre-conditioning matrices)[106], [107]까지 다양할 수 있습니다. 

궁극적으로 $\omega$는 입력 gradient와 다른 메타데이터의 복잡한 비선형 변환을 통해 완전한 gradient 기반 optimzer를 정의하는 데 사용될 수 있습니다[19], [39], [93], [94]. 만약 optimzer가 가중치별로 좌표 단위(coordinate-wise)로 적용된다면 여기서 학습할 파라미터는 적을 수 있습니다[19]. 

초기화 중심 방법과 optimzer 중심 방법은 함께 학습함으로써 병합될 수 있는데, 즉 전자가 후자의 초기 조건을 학습하는 방식입니다[39], [79]. 
optimzer 학습 방법은 퓨샷 학습[39]과 다중샷 학습의 가속 및 개선[19], [93], [94] 모두에 적용되었습니다. 

마지막으로, gradient와 같은 optimzer 상태 대신 $\mathcal{L}_{\text{task}}$의 평가값만 요구하는 0차 optimzer(zeroth-order optimizers)[108]를 메타-학습할 수도 있습니다. 이들은 기존의 베이지안 최적화[73] 대안들과 경쟁력 있음이 보여졌습니다[108].

**comments**

*   **무엇을 배우는가?**: **"가장 효율적인 등산 방법(최적화 규칙)"** 자체를 배웁니다.
*   **비유**: '최고의 베이스캠프'를 찾는 대신, 어떤 지점에서든 가장 빨리 정상으로 갈 수 있는 **'네비게이션 앱'($\omega$)**을 만드는 것과 같습니다. 이 앱은 현재 위치(파라미터 $\theta$)와 gradient($\nabla\mathcal{L}$)를 입력받아 "다음 발걸음은 이쪽으로 이만큼 가세요"라고 알려줍니다.
*   **다양한 수준**:
    *   **단순한 수준**: "보폭(learning rate)은 항상 0.5m로 하세요."
    *   **복잡한 수준**: 지형(gradient)에 따라 보폭과 방향을 동적으로 조절하는 정교한 규칙을 배웁니다.
*   **장점**: 초기화 학습과 결합할 수 있으며, 퓨샷(빠른 적응)과 다중샷(최종 성능) 모두에 효과적입니다.

### 피드-포워드 모델 (FFMs. aka, Black-Box, Amortized)

또 다른 계열의 모델들은 gradient 기반의 반복적인 $\theta$ 최적화에 의존하는 대신, 서포트셋에서 테스트 인스턴스를 분류하는 데 필요한 파라미터로 직접 매핑하는 피드-포워드 매핑을 제공하는 학습기 $\omega$를 훈련시킵니다. 
즉, $\theta = g_{\omega}(D^{\text{train}})$입니다. 

이들은 기존 분류 체계의 블랙박스 모델 기반 학습(섹션 3.1)에 해당하며, 고전적인 방법[109]부터 CNAPs[110]와 같이 도전적인 교차-도메인 퓨샷 벤치마크[111]에서 강력한 성능을 보이는 최근 접근법까지 다양합니다.

이러한 방법들은 어떤 임베딩에 따라 다른 신경망의 가중치를 생성하는 하이퍼네트워크(Hypernetworks)[112], [113]와 연결되며, 종종 압축이나 다중과제 학습에 사용됩니다. 여기서 $\omega$는 하이퍼네트워크이며, 소스 데이터셋이 주어지면 피드-포워드 패스를 통해 $\theta$를 합성합니다[100], [114]. 서포트셋의 임베딩은 종종 순환 신경망[51], [115], [116], 합성곱[38], 또는 집합 임베딩(set embeddings)[45], [110]을 통해 달성됩니다. 

여기서의 연구는 종종 과제-임베딩 네트워크로 분류기를 매개변수화하는 구조를 탐구합니다.

(i) 어떤 파라미터를 모든 과제에 걸쳐 전역적으로 공유하고, 대조적으로 하이퍼네트워크에 의해 과제별로 합성할 것인가 (예: 특징 추출기는 공유하고 분류기는 합성[83], [117])   
(ii) $\omega$에 필요한 파라미터 수를 제한하기 위해 하이퍼네트워크를 어떻게 매개변수화할 것인가 (예: 특징 추출기 내의 경량 어댑터 레이어만 합성[110]하거나, 클래스별 분류기 가중치 합성[45]).

일부 FFM은 확률 모델에서의 상각 추론(amortized inference)[45], [109]의 관점에서 우아하게 이해될 수 있으며, 테스트 데이터 $x$에 대한 예측을 다음과 같이 합니다:

$$
q_{\omega}(y|x, D^{tr}) = \int p(y|x, \theta) q_{\omega}(\theta|D^{tr}) d\theta \quad (8)
$$

여기서 메타-표현 $\omega$는 훈련 데이터 $D^{tr}$로 과제를 해결하는 파라미터 $\theta$에 대한 다루기 힘든 베이지안 추론을 근사하는 네트워크 $q_{\omega}(\cdot)$이며, 적분은 정확하게 계산되거나[109], 샘플링[45] 또는 점 추정[110]으로 근사될 수 있습니다. 그 후 모델 $\omega$는 훈련 과제 분포에 대한 검증 손실을 최소화하도록 훈련됩니다 (식 7 참조).

마지막으로, 오래된 데이터를 기억하고 새로운 데이터를 빠르게 동화시키는 능력을 가진 메모리 증강 신경망도 일반적으로 FFM 범주에 속합니다[86], [87].

**comments**

*   **무엇을 배우는가?**: **"문제지를 보면 바로 정답지를 만들어내는 기계($\omega$)"**를 배웁니다.
*   **비유**: 학생이 문제를 푸는 것이 아니라, 문제지($D^{\text{train}}$)를 스캔하면 그 문제에 맞는 **맞춤형 해설지($\theta$)**를 즉시 출력해주는 '마법 복사기'($g_\omega$)를 만드는 것과 같습니다.
*   **핵심 아이디어**: 느린 반복 최적화 과정을 완전히 생략하고, 함수 한 번 통과시키는 것으로 학습을 끝냅니다.
*   **주요 연구 주제**:
    1.  **어디까지 복사할까?**: 해설지의 모든 내용을 새로 만들까(분류기 합성), 아니면 기존 교과서(특징 추출기)는 그대로 두고 핵심 공식만 새로 만들어줄까?
    2.  **어떻게 효율적으로 복사할까?**: 복사기 자체($\omega$)가 너무 크고 복잡하지 않도록 어떻게 설계할까?
*   **이론적 배경 (상각 추론)**: 이 방식은 복잡한 베이지안 확률 계산을 "미리 학습된 신경망 한 번 통과시키는" 간단한 과정으로 근사하는 것과 같다는 깊은 이론적 해석이 가능합니다.


### 임베딩 함수 (Embedding Functions, Metric Learning)

여기서 메타-최적화 과정은 원시 입력(raw inputs)을 쿼리와 서포트 인스턴스 간의 간단한 유사도 비교를 통해 인식하기에 적합한 표현으로 변환하는 **임베딩 네트워크 $\omega$**를 학습합니다[20], [83], [90], [117] (예: 코사인 유사도 또는 유클리드 거리 사용). 

이러한 방법들은 기존 분류 체계(섹션 3.1)에서 측정 학습으로 분류되지만, 위의 피드-포워드 블랙박스 모델의 특수한 경우로도 볼 수 있습니다. 이는 서포트와 쿼리 이미지 $x_s$와 $x_q$의 임베딩 내적에 기반하여 로짓(logits)을 생성하는 방법, 즉 $g_{w}(x_q)^T g_{w}(x_s)$[83], [117]을 보면 쉽게 알 수 있습니다. 

+ 여기서 서포트 이미지는 쿼리 예제를 해석하기 위한 '가중치'를 생성하며, 이는 '하이퍼네트워크'가 쿼리셋에 대한 선형 분류기를 생성하는 FFM의 특수한 경우로 만듭니다. 

이 계열의 기본 방법들은 임베딩을 과제-조건부(task-conditional)로 만들거나[101], [118], 더 정교한 비교 측정 기준을 학습하거나[91], [92], 또는 확률적 정규화기와 같은 다른 하이퍼파라미터를 훈련시키기 위해 gradient 기반 메타러닝과 결합함으로써[119] 더욱 향상되었습니다.

**comments**

*   **무엇을 배우는가?**: **"데이터를 분류하기 좋은 지도(임베딩 공간)"**를 만드는 방법을 배웁니다.
*   **비유**: 도서관 사서가 책을 정리하는 것과 같습니다. 좋은 사서($\omega$)는 관련 있는 책(같은 클래스)은 같은 서가에 모아두고, 관련 없는 책(다른 클래스)은 멀리 떨어진 서가에 둡니다.
*   **작동 방식**:
    1.  새로운 책(쿼리 데이터)이 들어오면,
    2.  그 책과 가장 가까운 곳에 있는 책들(서포트 데이터)을 보고,
    3.  "아, 이 책은 과학 섹션에 있으니 과학 책이구나!"라고 판단합니다.
*   **FFM과의 관계**: 이것은 FFM의 매우 **특수한 형태**로 볼 수 있습니다. '서포트 데이터'가 일종의 '분류 기준(가중치)' 역할을 하여 '쿼리 데이터'를 판단하기 때문입니다. '마법 복사기'가 아주 단순한 형태의 '선형 분류기'만 출력하는 경우와 같습니다.

### 손실 및 보조 과제 (Losses and Auxiliary Tasks)

optimzer 설계에 대한 메타러닝 접근법과 유사하게, 
이들은 기반 모델을 위한 **내부 과제-손실 $\mathcal{L}^{\text{task}}_{\omega}(\cdot)$을 학습**하는 것을 목표로 합니다. 
손실-학습 접근법은 일반적으로 손실과 관련된 양(예: 예측, 특징, 또는 모델 파라미터)을 입력받아 스칼라 값을 출력하는 작은 신경망을 정의하며, 이 출력은 내부 (과제) optimzer에 의해 손실로 취급됩니다. 

+ 이는 일반적으로 사용되는 손실 함수들보다 최적화하기 더 쉬운(예: 지역 최솟값이 적은) 학습된 손실로 이어지거나[23], [120], [121], 
+ 개선된 일반화와 함께 더 빠른 학습으로 이어지거나[43], [122]-[124], 
+ 또는 그 최솟값이 도메인 이동에 더 강건한 모델에 해당하는 손실[42]로 이어지는 등의 잠재적 이점을 가집니다. 

손실 학습 방법은 또한 레이블이 없는 인스턴스로부터 학습하는 법을 배우거나[101], [125], 또는 정밀도-재현율 곡선 아래 면적과 같이 미분 불가능한 실제 과제 손실에 대한 미분 가능한 근사치로서 $\mathcal{L}^{\text{task}}(\cdot)$를 학습하는 데[126], [127] 사용되었습니다.

손실 학습은 자기-지도(self-supervised)[128] 또는 보조 과제(auxiliary task)[129] 학습의 일반화에서도 발생합니다. 
이러한 문제들에서는 주 과제에 대한 표현을 향상시킬 목적으로 비지도 예측 과제(예: 비전 분야의 픽셀 색칠하기[128] 또는 RL 분야의 단순히 픽셀 바꾸기[129])가 정의되고 최적화됩니다. 이 경우, 사용할 최상의 보조 과제(손실)를 미리 예측하기 어려울 수 있으므로, 메타러닝을 사용하여 주 과제 학습 개선에 미치는 영향에 따라 여러 보조 손실 중에서 선택할 수 있습니다. 즉, $\omega$는 보조 과제별 가중치입니다[70]. 더 일반적으로는, 예시에 보조 레이블을 달아주는 보조 과제 생성기를 메타-학습할 수도 있습니다[130].

**comments**

*   **무엇을 배우는가?**: **"가장 이상적인 채점 기준(손실 함수)"**을 배웁니다.
*   **비유**: 학생을 평가할 때, 단순히 '정답 개수'로만 점수를 매기는 것이 최선이 아닐 수 있습니다. "어떤 방식으로 채점($\mathcal{L}^{\text{task}}$)해야 학생이 가장 빠르고 깊이 있게 성장할까?"라는 **'최고의 채점 방식'($\omega$)** 자체를 학습하는 것입니다.
*   **목표**:
    1.  **더 쉬운 최적화**: 정답으로 가는 길이 울퉁불퉁하지 않고 매끄러운 손실 함수를 만듭니다.
    2.  **더 나은 일반화**: 최종 시험에서 더 높은 점수를 받게 유도하는 손실 함수를 만듭니다.
    3.  **보조 과제 선택**: 주 과제(예: 자율주행)에 도움이 되는 최적의 보조 과제(예: '도로선 예측하기', '신호등 색깔 맞추기')의 중요도를 자동으로 학습합니다.

### 구조 (Architectures)

구조 발견은 신경망에서 항상 중요한 영역이었으며[37], [131], 간단한 전체 탐색(exhaustive search)이 불가능한 분야입니다. 메타러닝은 구조를 학습함으로써 이 매우 비용이 많이 드는 과정을 자동화하는 데 사용될 수 있습니다. 

+ 초기 시도들은 진화 알고리즘을 사용하여 LSTM 셀의 토폴로지를 학습했으며[132], 
+ 이후 접근법들은 좋은 CNN 구조에 대한 설명을 생성하기 위해 RL을 활용했습니다[26]. 
+ 진화 알고리즘[25]은 그래프로 모델링된 구조 내의 블록을 학습할 수 있으며, 이 그래프를 편집하여 돌연변이를 일으킬 수 있습니다. 
+ DARTS[18] 형태의 경사도 기반 구조 표현도 탐구되었는데, 여기서 훈련 중 순전파는 주어진 블록 내의 모든 가능한 레이어의 출력에 대한 소프트맥스로 구성되며, 이는 메타-학습될 계수(즉, $\omega$)에 의해 가중치가 부여됩니다. 메타-테스트 중에는 가장 높은 계수에 해당하는 레이어만 유지하여 구조가 이산화(discretized)됩니다. 
+ DARTS를 개선하기 위한 최근 노력들은 더 효율적인 미분 가능한 근사[133], 이산화 단계의 강건성 향상[134], 적응하기 쉬운 초기화 학습[135], 또는 구조 사전 확률(priors) 학습[136]에 초점을 맞추었습니다. 자세한 내용은 섹션 5.4를 참조하십시오.

**comments**

*   **무엇을 배우는가?**: **"가장 성능이 좋은 모델의 설계도(구조)"**를 배웁니다.
*   **비유**: 최고의 자동차를 만들기 위해, 엔진, 바퀴, 차체를 어떻게 조합하고 연결해야 하는지에 대한 **'최적의 설계도'($\omega$)**를 컴퓨터가 자동으로 찾게 하는 것입니다.
*   **어떻게?**:
    1.  **진화/강화학습**: 여러 설계도를 무작위로 만들어보고, 성능 테스트를 거쳐 가장 좋은 설계도만 살아남게 하거나, 좋은 설계도를 만드는 '설계 에이전트'를 훈련시킵니다.
    2.  **경사도 기반 (DARTS)**: 모든 가능한 부품(레이어)을 일단 전부 연결해두고, 각 연결의 '중요도($\omega$)'를 학습합니다. 학습이 끝난 뒤, 중요도가 낮은 연결은 끊어버리고 가장 중요한 연결만 남겨 최종 설계도를 완성합니다.


### 어텐션 모듈 (Attention Modules)

어텐션 모듈은 측정 기반 메타-학습기에서 비교기로 사용되거나[137], 퓨샷 연속 학습에서 치명적 망각(catastrophic forgetting)을 방지하거나[138], 텍스트 분류 과제들의 분포를 요약하는 데[139] 사용되었습니다.

**comments**

*   **무엇을 배우는가?**: **"어디에 집중해야 하는지"**를 배웁니다.
*   **비유**: 학생이 문제를 풀 때, 문제의 어느 부분(키워드, 조건 등)에 집중해야 정답을 찾을 수 있는지를 알려주는 **'형광펜'($\omega$)**의 역할을 학습합니다.
*   **용도**:
    *   두 이미지를 비교할 때 어느 부분이 유사한지 집중합니다.
    *   새로운 것을 배울 때, 이전에 배운 지식 중 어느 부분을 건드리지 말아야 할지 집중합니다.

### 모듈 (Modules)

모듈러 메타러닝[140], [141]은 과제에 구애받지 않는 지식 $\omega$가 **모듈 집합**을 정의하고, 이 모듈들이 마주치는 각 과제를 해결하기 위해 $\theta$에 의해 정의된 과제-특화 방식으로 재구성된다고 가정합니다. 
이러한 전략들은 다중과제 및 전이 학습에서 잘 연구된 지식 공유에 대한 일반적인 구조적 접근법의 메타러닝 일반화로 볼 수 있으며[67], [68], [142], 궁극적으로는 구성적 학습(compositional learning)[143]의 기반이 될 수 있습니다.

**comments**

*   **무엇을 배우는가?**: **"재사용 가능한 레고 블록(모듈) 세트"**를 배웁니다.
*   **비유**: 자동차, 비행기, 배를 각각 처음부터 설계하는 대신, '바퀴', '날개', '엔진', '조종석'과 같은 **'표준 부품'($\omega$)** 세트를 미리 만들어 둡니다.
*   **작동 방식**: 새로운 과제(예: '잠수함 만들기')가 주어지면, 이 표준 부품들을 **어떻게 조합($\theta$)**해야 할지만 빠르게 학습합니다. 이는 훨씬 효율적입니다.


### 하이퍼파라미터 (Hyper-parameters)

여기서 $\omega$는 정규화 강도[17], [71], 파라미터별 정규화[95], 다중과제 학습에서의 과제-관련성[69], 또는 데이터 정제에서의 희소성 강도[69]와 같은 기반 학습기의 하이퍼파라미터를 나타냅니다. 스텝 사이즈[71], [79], [80]와 같은 하이퍼파라미터는 optimzer의 일부로 볼 수 있어, 하이퍼파라미터와 optimzer 학습 범주 간에 중첩이 발생합니다.

**comments**

*   **무엇을 배우는가?**: **"학습 과정을 조절하는 각종 설정값"**을 배웁니다.
*   **비유**: 요리를 할 때 '불의 세기', '조리 시간', '소금의 양'($\omega$)과 같은 **'최적의 레시피 설정값'**을 학습하는 것입니다. 이 설정값들이 최종 요리의 맛(성능)을 결정합니다.


### 데이터 증강 (Data Augmentation)

지도 학습에서는 기존 데이터에 레이블을 보존하는 변환을 가하여 더 많은 훈련 데이터를 합성함으로써 일반화를 개선하는 것이 일반적입니다. 데이터 증강 연산은 내부 문제(식 6)의 최적화 단계에 포함되며, 전통적으로는 사람이 직접 설계합니다. 그러나 $\omega$가 데이터 증강 전략을 정의할 때, 이는 검증 성능을 최대화하기 위해 외부 최적화(식 5)에 의해 학습될 수 있습니다[144]. 

증강 연산은 일반적으로 미분 불가능하기 때문에, 이는 강화 학습[144], discrete gradient-estimators[145], 또는 진화[146] 방법을 필요로 합니다. 강력한 GAN 기반 데이터 증강 방법[147]이 내부-수준 학습에 사용되고 외부-수준 학습에서 최적화될 수 있는지 여부는 아직 미해결 질문입니다.

**comments**

*   **무엇을 배우는가?**: **"가장 효과적인 데이터 뻥튀기 방법"**을 배웁니다.
*   **비유**: 고양이 사진이 10장밖에 없을 때, 이 사진들을 '좌우 반전', '회전', '밝기 조절' 등을 해서 1000장처럼 만드는 것이 데이터 증강입니다. 메타러닝은 "어떤 종류의 '뻥튀기'($\omega$)를 해야 모델 성능이 가장 많이 오를까?"라는 **'최고의 뻥튀기 전략'**을 학습합니다.

### 미니배치 선택, 샘플 가중치, 및 커리큘럼 학습 (Minibatch Selection, Sample Weights, and Curriculum Learning)

기반 알고리즘이 미니배치 기반 확률적 경사 하강법일 때, 학습 전략의 설계 파라미터는 배치 선택 과정입니다. 무작위로 샘플링된 미니배치를 개선하기 위해 사람이 설계한 다양한 방법[148]이 존재합니다. 
메타러닝 접근법은 $\omega$를 인스턴스 선택 확률[149] 또는 미니배치에 포함할 인스턴스를 선택하는 신경망[150]으로 정의할 수 있습니다. 미니배치 선택 정책과 관련된 방법으로는 훈련 세트에 대한 샘플별 손실 가중치 $\omega$를 학습하는 방법이 있습니다[151], [152]. 이는 노이즈가 있는 샘플의 가중치를 줄이거나[151], [152], 이상치의 가중치를 줄이거나[69], 또는 클래스 불균형을 보정하는 데[151] 사용될 수 있습니다.

더 일반적으로, 커리큘럼[153]은 항목을 무작위 순서로 학습하는 것보다 더 나은 성능을 내는 데이터 또는 개념의 학습 순서를 의미합니다. 

+ 예를 들어, 너무 어렵거나 너무 쉬운(이미 학습된) 인스턴스는 거부하고 적절한 난이도의 인스턴스에 집중하는 것입니다. 
+ 커리큘럼을 사람이 직접 정의하는 대신[154], 메타러닝은 그 과정을 자동화하고, 메타 지식으로서 가르치는 정책을 정의하고 학생의 발전을 최적화하도록 훈련함으로써 적절한 난이도의 예시를 선택할 수 있습니다[150], [155].

**comments**

*   **무엇을 배우는가?**: **"어떤 데이터를, 어떤 순서로, 얼마나 중요하게"** 가르칠지를 배웁니다.
*   **비유**: 최고의 교사는 학생의 수준에 맞춰 **'맞춤형 교육 계획'($\omega$)**을 세웁니다.
    1.  **미니배치 선택**: "지금 단계에서는 이 유형의 문제를 집중적으로 풀어보는 게 좋겠어." (가장 도움이 될 데이터 선택)
    2.  **샘플 가중치**: "이 문제는 아주 중요하니 별표 다섯 개 치고, 저 문제는 오타가 있으니 무시해." (데이터의 중요도 조절)
    3.  **커리큘럼 학습**: "처음에는 쉬운 덧셈부터 가르치고, 그 다음에 뺄셈, 마지막에 곱셈을 가르쳐야 해." (가장 효율적인 학습 순서 결정)
*   메타러닝은 이러한 **'최고의 교수법'**을 데이터로부터 자동으로 학습합니다.

### 데이터셋, 레이블 및 환경 (Datasets, Labels and Environments)

또 다른 메타-표현은 **서포트 데이터셋 그 자체**입니다. 

+ 이는 소스 데이터셋이 고정되어 있다고 간주했던 우리의 초기 정형화(섹션 2.1, 식 2-3)에서 벗어납니다. 
+ 그러나 이는 식 5-6의 이중 최적화 관점에서는 쉽게 이해될 수 있습니다. 
  + 만약 상위 최적화의 검증 세트가 실제이고 고정되어 있고, 하위 최적화의 훈련 세트가 $\omega$로 매개변수화된다면, 훈련 데이터셋은 검증 성능을 최적화하기 위해 메타-학습에 의해 조정될 수 있습니다.

**데이터셋 증류(Dataset distillation)**[156], [157]에서는, 서포트 이미지 자체가 학습되어, 그 이미지들에 대해 몇 스텝만 학습해도 실제 쿼리 이미지에 대한 좋은 일반화를 가능하게 합니다. 이는 대규모 데이터셋을 소수의 이미지로 요약하는 데 사용될 수 있으며, 스트리밍 데이터셋을 저장할 수 없는 연속 학습에서의 리플레이에 유용합니다.

고정된 레이블 $y$에 대해 입력 이미지 $x$를 학습하는 대신, 고정된 이미지 $x$에 대해 **입력 레이블 $y$를 학습**할 수도 있습니다. 이는 데이터셋 증류에서처럼 핵심 세트(core sets)[158]를 증류하거나, 또는 예를 들어 검증 세트 성능을 최적화하기 위해 레이블이 없는 세트의 레이블을 직접 학습하는 준지도 학습(semi-supervised learning)에 사용될 수 있습니다[159], [160].

컴퓨터 비전이나 강화 학습에서의 **sim2real 학습**[161]의 경우, 훈련 데이터를 생성하기 위해 환경 시뮬레이터를 사용합니다. 이 경우, 섹션 5.3에서 자세히 설명하겠지만, 그 환경 시뮬레이터에서 생성된 데이터로 훈련한 후 다운스트림 모델의 실제 데이터 (검증) 성능을 최적화하기 위해 **그래픽 엔진[162]이나 시뮬레이터[163]를 훈련**시킬 수도 있습니다.

**comments**

*   **무엇을 배우는가?**: **"가장 이상적인 교과서/문제집($\omega$)"** 자체를 배웁니다.
*   **비유**: 학생에게 기존의 교과서를 주는 것이 아니라, "어떤 내용으로 구성된 교과서를 만들어줘야 학생이 가장 빠르고 깊이 있게 배울까?"를 고민하여 **'맞춤형 교과서'를 직접 저술**하는 것과 같습니다.
*   **다양한 형태**:
    1.  **데이터셋 증류**: 1000페이지짜리 교과서의 모든 핵심 내용을 단 10페이지로 요약한 **'초압축 요약본'**을 만듭니다. 이 요약본만 봐도 전체 내용을 이해할 수 있습니다.
    2.  **레이블 학습**: 문제(이미지)는 그대로 두고, **가장 이상적인 정답(레이블)**을 만들어냅니다. (준지도 학습)
    3.  **환경 학습 (Sim2Real)**: 자율주행차를 훈련시킬 가상현실(시뮬레이터)이 있을 때, 이 가상현실을 **실제 세계와 최대한 비슷하게** 만들도록 시뮬레이터 자체를 학습시킵니다.

### 논의: 추론적 표현과 방법 (Discussion: Transductive Representations and Methods)

위에서 논의된 대부분의 표현 $\omega$는 데이터를 처리하거나 생성하는 함수의 파라미터 벡터입니다. 그러나 언급된 표현 중 일부는 $\omega$가 말 그대로 데이터 포인트[156], 레이블[159], 또는 샘플별 가중치[152]에 해당하는 **추론적(transductive)**인 의미를 가집니다. 따라서 메타-학습할 $\omega$의 파라미터 수는 데이터셋의 크기에 따라 확장됩니다. 이러한 방법들의 성공은 현대 메타러닝의 역량을 증명하지만[157], 이 속성은 궁극적으로 그들의 확장성을 제한할 수 있습니다.

추론적 표현과는 별개로, 서포트 인스턴스뿐만 아니라 쿼리 인스턴스에 대해서도 작동하는 **추론적인 방법**들이 있습니다[101], [130].

**comments**

*   **추론적(Transductive)이란?**: 훈련 데이터뿐만 아니라, **미리 주어진 테스트 데이터(쿼리셋)의 정보까지 활용**하여 학습하는 방식입니다. (반대: 귀납적(Inductive)은 훈련 데이터만 보고 학습)
*   **추론적 표현**: $\omega$가 모델의 가중치 같은 일반적인 규칙이 아니라, **데이터 자체(이미지 픽셀, 레이블 값 등)**인 경우를 말합니다.
    *   **장점**: 특정 데이터셋에 매우 강력하게 최적화될 수 있습니다.
    *   **단점**: 데이터가 많아지면 학습해야 할 $\omega$의 크기도 무한정 커져서 **확장성이 떨어집니다.**
*   **추론적 방법**: $\omega$는 일반적인 규칙이지만, 학습 과정에서 테스트 데이터(쿼리셋)의 특징을 참고하는 방법입니다.

### 논의: 해석 가능한 기호적 표현 (Discussion: Interpretable Symbolic Representations)

위에서 논의된 많은 메타-표현들을 가로지르는 또 다른 구분은 **해석 불가능한 (비기호적) 표현**과 **인간이 해석 가능한 (기호적) 표현** 사이의 구분입니다. $\omega$가 신경망을 매개변수화할 때와 같은 비기호적 표현[19]이 더 일반적이며, 위에서 인용된 연구의 대다수를 차지합니다.

그러나 기호적 표현을 사용한 메타러닝도 가능하며, 여기서 $\omega$는 최적화 프로그램 코드[93]와 같이 인간이 읽을 수 있는 기호적 함수를 나타냅니다. 신경망 손실 함수[42] 대신, 교차 엔트로피와 유사한 표현식으로 정의된 기호적 손실 $\omega$를 훈련시킬 수 있습니다[123]. ReLU와 같은 표준 활성화 함수를 능가하는 새로운 기호적 활성화 함수를 메타-학습할 수도 있습니다[164]. 이러한 메타-표현은 매끄럽지 않기 때문에, 메타-목적은 미분 불가능하며 최적화하기 더 어렵습니다 (섹션 4.2 참조). 그래서 $\omega$에 대한 상위 최적화는 일반적으로 RL[93]이나 진화 알고리즘[123]을 사용합니다. 그러나 기호적 표현은 과제군 전반에 걸쳐 일반화하는 능력에서 이점을 가질 수 있습니다[93], [123], [164]. 즉, 메타-훈련 중 단일 $\omega$로 더 넓은 분포 $p(\mathcal{T})$를 포괄하거나, 또는 학습된 $\omega$가 메타-테스트 중 분포를 벗어난 과제에 대해 일반화하는 것입니다 (섹션 6 참조).

**comments**

*   **비기호적(Sub-symbolic) 표현**: 우리가 이해할 수 없는 **숫자의 나열** (예: 신경망의 가중치 행렬). 대부분의 메타러닝이 여기에 해당합니다.
*   **기호적(Symbolic) 표현**: **인간이 읽고 이해할 수 있는 형태** (예: `loss = -y * log(p)`)
*   **무엇을 배우는가?**: 신경망 가중치 대신, **수학 공식이나 프로그램 코드 자체**를 배웁니다.
*   **장점**:
    1.  **해석 가능**: AI가 무엇을 배웠는지 우리가 이해할 수 있습니다.
    2.  **뛰어난 일반화**: 특정 숫자에 얽매이지 않고 추상적인 규칙을 배우기 때문에, 훈련 때 본 적 없는 매우 생소한 문제에도 잘 적응할 수 있는 잠재력이 있습니다.
*   **단점**: 미분이 불가능해서 경사 하강법으로 학습시키기 어렵습니다. (그래서 RL이나 진화 알고리즘을 사용)

### 논의: 상각 (Discussion: Amortization)

논의된 표현 중 일부를 연관 짓는 한 가지 방법은 수반되는 **학습 상각(learning amortization)**의 정도입니다[45]. 즉, 메타-테스트 중에 얼마나 많은 과제-특화 최적화가 수행되는가 대 메타-훈련 중에 얼마나 많은 학습이 상각되는가 입니다. 처음부터 훈련하거나 전통적인 미세 조정[57]은 메타-테스트에서 완전한 과제-특화 최적화를 수행하며, 상각은 없습니다. MAML[16]은 초기 조건을 맞춤으로써 제한된 상각을 제공하여, 몇 스텝의 미세 조정으로 새로운 과제를 학습할 수 있게 합니다. 순수한 FFM[20], [90], [110]은 과제-특화 최적화 없이 완전히 상각되어, 새로운 과제를 가장 빠르게 학습할 수 있게 합니다. 한편, 일부 하이브리드 접근법[100], [101], [111], [165]은 단일 프레임워크 내에서 피드-포워드와 최적화 기반 메타러닝 모두를 활용하여 준-상각(semi-amortized) 학습을 구현합니다.

**comments**

*   **상각(Amortization)이란?**: "비싼 계산 비용을 언제 치를 것인가?"의 문제입니다.
*   **비유**: 레스토랑에서 요리하기
    *   **상각 없음 (전통적 학습)**: 손님이 주문할 때마다(새로운 과제), 처음부터 재료를 손질하고 요리를 시작합니다. (시간이 오래 걸림)
    *   **완전 상각 (FFM)**: 영업 시작 전에(메타-훈련), 모든 재료를 미리 손질해두고(밀키트 제작), 소스도 다 만들어 둡니다. 손님이 주문하면(새로운 과제), 그냥 데워서 내기만 하면 됩니다. (매우 빠름)
    *   **제한된 상각 (MAML)**: 재료 손질(초기화)까지는 미리 해두지만, 소스를 만들고 볶는 과정(몇 스텝의 최적화)은 주문이 들어오면 합니다. (중간 정도의 속도)
    *   **준-상각 (하이브리드)**: 밀키트(FFM)를 사용하면서, 손님의 특별 요청에 따라 약간의 추가 조리(최적화)를 하는 방식입니다.

## 메타-최적화기 (Meta-Optimizer)

학습 전략의 어떤 측면을 최적화할지 선택한 후, 메타-학습기 설계의 다음 축은 $\omega$를 훈련시키는 데 사용할 실제 **외부 (메타) 최적화 전략**입니다.

### 경사도 (Gradient)

상당히 많은 방법들이 메타 파라미터 $\omega$에 대해 **경사 하강법**을 사용합니다[16], [39], [42], [69]. 이는 외부 목적 함수의 도함수 $d\mathcal{L}^{\text{meta}}/d\omega$를 계산해야 하며, 이는 일반적으로 연쇄 법칙(chain rule)을 통해 모델 파라미터 $\theta$와 연결됩니다.

$$\dfrac{d\mathcal{L}^{\text{meta}}}{d\omega} = (\dfrac{d\mathcal{L}^{\text{meta}}}{d\theta})(\dfrac{d\theta}{d\omega})$$

이러한 방법들은 $\omega$의 해석적 경사도(analytical gradients)를 활용하기 때문에 잠재적으로 가장 효율적입니다. 그러나 주요 난제는 다음과 같습니다: 

(i) 예를 들어 신중하게 설계된 미분 알고리즘[17], [71], [193]과 암시적 미분[157], [167], [194]을 통해, 그리고 요구되는 2차 경사도를 다루기 쉽게 처리함으로써[195], **내부 최적화의 여러 스텝을 효율적으로 미분**하는 것.    
(ii) 내부 루프 최적화 스텝의 수가 증가함에 따라 심각성이 커지는 **필연적인 경사도 저하(gradient degradation) 문제를 줄이는 것**.   
(iii) 기반 학습기, $\omega$, 또는 $\mathcal{L}^{\text{task}}$가 이산적이거나 미분 불가능한 연산을 포함할 때 **경사도를 계산**하는 것.

**comments**

*   **어떻게 학습하는가?**: **미분(경사도)**을 이용한 가장 표준적이고 효율적인 최적화 방법인 **경사 하강법**을 사용합니다.
*   **비유**: 산에서 가장 빨리 내려가기 위해, 현재 위치에서 **가장 가파른 내리막길 방향(경사도)**으로 한 걸음씩 계속 나아가는 것과 같습니다.
*   **어려움 (난제)**:
    1.  **효율적인 미분**: 내부 학습 과정 전체(수십~수백 스텝)를 하나의 긴 함수로 보고 미분해야 하는데, 이는 **계산이 매우 복잡하고 메모리를 많이 차지**합니다.
    2.  **경사도 저하**: 미분 과정이 너무 길어지면, 초반부의 정보가 뒤로 갈수록 희미해져서 **경사도가 0에 가까워지거나(소실) 무한대로 커지는(폭주) 문제**가 발생합니다.
    3.  **미분 불가능**: 학습 과정에 미분이 불가능한 단계(예: 'A 또는 B 중 하나를 선택')가 포함되어 있으면 경사 하강법을 쓸 수 없습니다.


### 강화 학습 (Reinforcement Learning)

Base Learner가 미분 불가능한 스텝을 포함하거나[144], 또는 메타-목적 $\mathcal{L}^{\text{meta}}$ 자체가 미분 불가능할 때[126], 많은 방법들이 외부 목적 함수 식 (5)를 최적화하기 위해 **강화 학습(RL)**에 의존합니다[22]. 

$$
\omega^* = \arg\min_{\omega} \sum_{i=1}^{M} \mathcal{L}^{\text{meta}}(\theta^{*(i)}(\omega), \omega, D_{\text{source}}^{\text{val }(i)}) \quad (5)
$$

이는 일반적으로 정책 경사도 정리(policy gradient theorem)를 사용하여 경사도 $\nabla_{\omega}\mathcal{L}^{\text{meta}}$를 추정합니다. 그러나 이러한 방식으로 미분 가능성 요구 사항을 완화하는 것은 일반적으로 **비용이 매우 많이 듭니다**. $\nabla_{\omega}\mathcal{L}^{\text{meta}}$에 대한 **분산이 큰(high-variance)** 정책-경사도 추정치는 수렴하는 데 많은 외부-수준 최적화 스텝이 필요하며, 이 스텝들 각각은 그 안에 과제-모델 최적화를 포함하고 있기 때문에 그 자체로 비용이 많이 듭니다.

**comments**

*   **어떻게 학습하는가?**: 미분이 불가능할 때 사용하는 **강화 학습**을 이용합니다.
*   **비유**: 산에서 내려갈 때 경사도를 계산하는 대신, **여러 방향으로 직접 한 걸음씩 가보고 그중 가장 많이 내려간 방향으로 계속 진행**하는 '시행착오' 방식입니다.
*   **어려움 (난제)**:
    *   **비용이 매우 비쌈**: 어떤 방향이 좋은지 알려면 수많은 시도를 해봐야 합니다. 이 과정에서 얻는 정보의 '운 편차(분산)'가 매우 커서, 수많은 시도를 해야만 겨우 올바른 방향을 찾을 수 있습니다. 즉, **학습 속도가 매우 느립니다.**

### 진화 (Evolution)

메타-목적을 최적화하는 또 다른 접근법은 **진화 알고리즘(EA)**입니다[14], [131], [196]. 많은 진화 알고리즘은 강화 학습 알고리즘과 강력한 연관성을 가집니다[197]. 그러나 그들의 성능은 RL처럼 내부 최적화의 길이나 보상 희소성(reward sparsity)에 의존하지 않습니다.

진화 알고리즘은 여러 가지 이유로 매력적입니다[196]: 

(i) **미분 가능성 제약 없이** 어떤 기반 모델과 메타-목적도 최적화할 수 있습니다. 
(ii) 역전파에 의존하지 않음으로써 **경사도 저하 문제와 기존 경사도 기반 방법의 고차 경사도 계산 비용을 모두 피할 수 있습니다.** 
(iii) **확장성을 위해 매우 병렬화하기 좋습니다.** 
(iv) **다양한 해의 집단을 유지**함으로써, 경사도 기반 방법이 빠지기 쉬운 **지역 최솟값을 피할 수 있습니다**[131]. 

그러나 여러 단점도 있습니다: 
(i) 학습할 파라미터의 수에 따라 **필요한 집단의 크기가 급격히 증가**합니다. 
(ii) **돌연변이 전략에 민감**할 수 있으며 신중한 하이퍼파라미터 최적화가 필요할 수 있습니다. 
(iii) 특히 CNN과 같은 대규모 모델의 경우, **적합 능력(fitting ability)이 일반적으로 경사도 기반 방법보다 열등**합니다.

진화 알고리즘은 (모델이 일반적으로 더 작고, 내부 최적화가 길고 미분 불가능한) RL 응용 분야[23], [172]에서 비교적 더 흔하게 적용됩니다. 그러나 학습 규칙[198], 최적화기[199], 구조[25], [131] 및 지도 학습에서의 데이터 증강 전략[146]을 학습하는 데에도 적용되었습니다. 또한 인간이 해석 가능한 기호적 메타-표현을 학습하는 데 특히 중요합니다[123].

**comments**

*   **어떻게 학습하는가?**: **자연의 진화(적자생존)** 원리를 모방한 **진화 알고리즘**을 사용합니다.
*   **비유**:
    1.  다양한 '학습 전략($\omega$)' 후보들을 무작위로 많이 만듭니다 (개체군 생성).
    2.  각 후보에게 문제를 풀게 해서 성능을 평가합니다 (적합도 평가).
    3.  성능이 좋은 후보들을 선택하고, 이들을 서로 섞거나(교차) 약간 변형하여(돌연변이) 새로운 자손 세대를 만듭니다.
    4.  이 과정을 수 세대 반복하면 결국 가장 뛰어난 전략만 살아남습니다.
*   **장점**:
    *   미분이 전혀 필요 없습니다.
    *   경사도 저하 문제가 없습니다.
    *   여러 후보를 동시에 평가할 수 있어 병렬 처리에 유리합니다.
    *   다양성 덕분에 최적의 해(전역 최솟값)를 찾을 가능성이 더 높습니다.
*   **단점**:
    *   파라미터가 많아지면(예: 대형 CNN) 엄청나게 많은 후보가 필요해서 **비효율적**입니다.
    *   성능이 경사 하강법보다 떨어지는 경우가 많습니다.
*   **주요 용도**: 모델이 비교적 작거나, 미분 불가능하며, 해석 가능한 해(코드, 수식)를 찾아야 할 때 유용합니다.

## 메타-목적 및 에피소드 설계 (Meta-Objective and Episode Design)

마지막 구성 요소는 **메타-목적 함수 $\mathcal{L}^{\text{meta}}$의 선택**과, 내부 루프 에피소드와 외부 최적화 간의 **관련 데이터 흐름**을 통해 메타러닝의 목표를 정의하는 것입니다. 
대부분의 방법은 과제 모델을 $\omega$로 업데이트한 후, 검증 세트에서 계산된 성능 지표를 사용하여 메타-목적을 정의합니다. 
이는 하이퍼파라미터 및 모델 선택에 대한 고전적인 검증 세트 접근 방식과 일맥상통합니다. 그러나 이 프레임워크 내에서도 몇 가지 설계 옵션이 있습니다:

### 다중샷 vs 퓨샷 에피소드 설계 (Many vs Few-Shot Episode Design)

목표가 퓨샷 성능을 향상시키는 것인지, 아니면 다중샷 성능을 향상시키는 것인지에 따라, 내부 루프 학습 에피소드는 과제당 **많은**[69], [93], [94] 또는 **적은**[16], [39] 예제로 정의될 수 있습니다.

**comments**

*   **핵심 질문**: "내부 루프(학생)에게 문제를 몇 개나 주고 공부시킬 것인가?"
*   **퓨샷(Few-shot)**: 예제를 **아주 조금**(1개~5개)만 주고, 그 적은 정보로 얼마나 빨리 적응하는지를 평가.
*   **다중샷(Many-shot)**: 예제를 **충분히 많이** 주고, 최종적으로 얼마나 높은 성능에 도달하는지를 평가.


### 빠른 적응 vs 점근적 성능 (Fast Adaptation vs Asymptotic Performance)

검증 손실이 내부 학습 에피소드의 **끝에서** 계산될 때, 메타-훈련은 기반 과제의 더 나은 **최종 성능**을 장려합니다. 만약 검증 손실이 **각 내부 최적화 스텝 이후의 손실 합**으로 계산된다면, 메타-훈련은 기반 과제에서의 **더 빠른 학습** 또한 장려합니다[80], [93], [94]. 대부분의 RL 응용 분야도 후자의 설정을 사용합니다.

**comments**

*   **핵심 질문**: "학생의 성적을 언제, 어떻게 채점할 것인가?"
*   **점근적 성능 (Asymptotic Performance)**: 공부가 **다 끝난 후** 치르는 '기말고사' 성적만 보고 평가. 최종적으로 얼마나 잘하는지가 중요.
*   **빠른 적응 (Fast Adaptation)**: 공부하는 **매 순간**(매 스텝) 치르는 '쪽지시험' 성적을 모두 합산하여 평가. 얼마나 빨리 배우는지가 중요.


### 다중 과제 vs 단일 과제 (Multi vs Single-Task)

목표가 주어진 과제군에서 추출된 어떤 과제든 더 잘 풀도록 학습기를 조정하는 것이라면, 내부 루프 학습 에피소드는 $p(\mathcal{T})$에서 무작위로 추출된 과제에 해당합니다[16], [20], [42]. 
목표가 단지 하나의 특정 과제를 더 잘 풀도록 학습기를 조정하는 것이라면, 내부 루프 학습 에피소드는 모두 동일한 기저 과제(underlying task)에서 데이터를 추출합니다[19], [69], [175], [183], [184], [200].

이 두 메타-목표는 서로 다른 가정과 가치 제안을 갖는 경향이 있다는 점은 주목할 가치가 있습니다. 다중 과제 목표는 당연히 함께 작업할 과제군 $p(\mathcal{T})$를 필요로 하지만, 단일 과제는 그렇지 않습니다. 한편, 다중 과제의 경우 메타-훈련의 데이터 및 컴퓨팅 비용은 메타-테스트 중에 여러 타겟 과제의 성능을 잠재적으로 향상시킴으로써 상각될 수 있습니다. 그러나 단일 과제는 —상각할 새로운 과제가 없으므로— 현재 과제의 최종 해법이나 점근적 성능을 향상시키거나, 또는 온라인으로 작동할 만큼 충분히 빠르게 메타-학습을 해야 합니다.

**comments**

*   **핵심 질문**: "이 메타러닝으로 궁극적으로 풀고 싶은 문제가 무엇인가?"
*   **다중 과제 (Multi-Task)**: **"어떤 종류의 문제가 나와도 잘 푸는 범용 학습법"**을 배우는 것이 목표. (예: 어떤 동물이든 잘 분류하는 법)
*   **단일 과제 (Single-Task)**: **"지금 당면한 이 하나의 어려운 문제"**를 더 잘 풀기 위한 최적의 학습법을 찾는 것이 목표. (예: 알파고가 오직 '바둑'만 더 잘 두는 법)
*   **가치 차이**:
    *   **다중 과제**: 한 번 비싼 돈 들여 '만능 공부법'을 배우면, 나중에 여러 문제에 써먹으며 **본전을 뽑을 수 있음 (상각 가능)**.
    *   **단일 과제**: 오직 한 문제에만 적용되므로, **'최종 성능'을 확실히 높여주거나, 실시간으로 학습 전략을 수정할 만큼 빨라야** 의미가 있음.

### 온라인 vs 오프라인 (Online vs Offline)

고전적인 메타러닝 파이프라인은 메타-최적화를 내부 기반 학습기의 외부-루프로 정의하지만[16], [19], 일부 연구들은 단일 기반 학습 에피소드 내에서 **온라인으로 메타-최적화를 수행**하려고 시도했습니다[42], [183], [200], [201]. 
이 경우, 기반 모델 $\theta$와 학습기 $\omega$는 단일 에피소드 동안 함께 진화합니다. 이제 상각할 소스 과제 집합이 없으므로, 메타러닝은 샘플 또는 컴퓨팅 효율성에서 이점을 얻기 위해 기반 모델 학습에 비해 빨라야 합니다.

**comments**

*   **핵심 질문**: "언제 학습 전략($\omega$)을 업데이트할 것인가?"
*   **오프라인 (Offline)**: 학생(내부 루프)의 공부가 **완전히 끝난 후**에, 선생님(외부 루프)이 그 결과를 보고 학습 전략을 업데이트. (학습과 메타-학습이 분리됨)
*   **온라인 (Online)**: 학생이 **공부하는 실시간으로** 선생님이 계속해서 학습 전략을 수정하고 피드백을 줌. (학습과 메타-학습이 동시에 진행됨)
*   **어려움**: 온라인 방식은 여러 과제 경험을 쌓을 시간이 없으므로, **매우 빠르게 학습 전략을 수정할 수 있어야** 효과가 있습니다.


### 기타 에피소드 설계 요소 (Other Episode Design Factors)

특정 응용 분야를 위해 메타러닝을 맞춤화하기 위해 에피소드 생성 파이프라인에 다른 연산자를 삽입할 수 있습니다. 예를 들어, 훈련과 검증 사이에 도메인 이동을 

+ 시뮬레이션하여 도메인 이동 하에서의 좋은 성능을 위해 메타-최적화하거나[42], [59], [95]
+ 훈련과 검증 사이에 양자화(quantization)와 같은 네트워크 압축을 시뮬레이션하여 네트워크 압축성을 위해 메타-최적화하거나[202]
+ 메타-훈련 중에 노이즈가 있는 레이블을 제공하여 레이블-노이즈 강건성을 위해 최적화하거나[96]
+ 또는 적대적 방어를 위해 메타-최적화하기 위해 적대적 검증 세트를 생성할 수 있습니다[97]. 

이러한 기회들은 다음 섹션에서 더 자세히 탐구됩니다.

**comments**

*   **핵심 아이디어**: "메타러닝의 최종 목표에 맞춰 **'맞춤형 훈련 환경'**을 설계하자."
*   **예시**:
    *   **강건성 목표**: 훈련은 깨끗한 데이터로 시키고, 평가는 일부러 노이즈를 섞은 데이터로 해서 **'노이즈에 강한 학습법'**을 배우게 함.
    *   **모델 압축 목표**: 훈련은 고품질 모델로 시키고, 평가는 용량을 줄인 저품질 모델로 해서 **'압축해도 성능이 잘 유지되는 학습법'**을 배우게 함.
    *   **보안 목표**: 훈련은 정상 데이터로 시키고, 평가는 해커의 공격을 받은 데이터로 해서 **'해킹 공격에 강한 학습법'**을 배우게 함.

# 응용 분야 (APPLICATIONS)

이 섹션에서는 컴퓨터 비전, 강화 학습, 구조 탐색 등에서 메타러닝이 어떻게 활용되어 왔는지를 간략하게 검토합니다.

## 컴퓨터 비전 및 그래픽스 (Computer Vision and Graphics)

컴퓨터 비전은 메타러닝 기술의 주요 소비 도메인이며, 특히 비전에서 인식해야 할 개념들의 롱테일(long-tail) 분포로 인해 발생하는 문제를 해결할 가능성을 지닌 **퓨샷 학습(few-shot learning)**에 미치는 영향 때문에 주목받고 있습니다.

### 퓨샷 학습 방법론 (Few-Shot Learning Methods)

퓨샷 학습(FSL)은 특히 대규모 신경망[1], [13]에게 매우 어려운 과제입니다. 대규모 신경망에서는 데이터의 양이 종종 성능을 좌우하는 지배적인 요인이 되며[203], 작은 데이터셋으로 큰 모델을 훈련시키면 과적합이나 비수렴으로 이어지기 때문입니다. 
메타러닝 기반 접근법은 많은 비전 문제에서 작은 데이터셋으로도 강력한 CNN을 훈련시킬 수 있는 능력을 점점 더 보여주고 있습니다. 다음과 같이 비-총망라적인 대표적 요약을 제공합니다.

#### 분류 (Classification)
메타러닝의 가장 일반적인 응용 분야는 **퓨샷 다중 클래스 이미지 인식**이며, 여기서 내부 및 외부 손실 함수는 일반적으로 각각 훈련 및 검증 데이터에 대한 교차 엔트로피입니다[20], [39], [77], [79], [80], [90], [92], [100], [101], [104], [107], [204]–[207]. 최적화기-중심[16], 블랙박스[38], [83] 및 측정 학습[90]-[92] 모델이 모두 고려되었습니다.

이러한 연구 흐름은 초기 방법들[16], [89], [90]에 비해 꾸준한 성능 향상을 이끌어냈습니다. 그러나 성능은 여전히 완전 지도 학습 방법의 성능에 훨씬 뒤처져 있어 더 많은 연구가 필요합니다. 현재의 연구 이슈에는 **교차-도메인 일반화** 개선[119], 메타-훈련 및 메타-테스트 클래스에 의해 정의된 **결합된 레이블 공간 내에서의 인식**[84], 그리고 **새로운 퓨샷 클래스의 점진적 추가**[138], [178]가 포함됩니다.

**comments**

*   **무엇을 하는가?**: 이미지를 보고 어떤 클래스(예: 고양이, 개, 자동차)에 속하는지 맞추는 가장 기본적인 문제입니다.
*   **어떻게 하는가?**: 이 논문에서 소개된 거의 모든 종류의 메타러닝 방법(최적화, 블랙박스, 측정 기반)이 이 문제에 적용되었습니다.
*   **현재 과제**:
    1.  **교차-도메인 일반화**: 일반 사진으로 학습한 뒤, 만화 그림이나 스케치 같은 완전히 다른 스타일의 이미지를 맞추는 능력.
    2.  **결합된 레이블 공간**: 훈련 때 배운 '개'와 테스트 때 새로 배운 '늑대'를 동시에 구분하는 능력.
    3.  **점진적 학습**: '개', '고양이'를 배운 모델에, 나중에 '새'를 추가로 가르칠 때 이전 지식을 잊지 않고 새로운 지식만 효율적으로 추가하는 능력.


#### 객체 탐지 (Object Detection)

퓨샷 분류의 발전에 힘입어, **퓨샷 객체 탐지**[178], [208]가 시연되었습니다. 이는 종종 서포트셋 이미지를 임베딩하고 기반 모델의 최종 레이어 분류 가중치를 합성하기 위해 피드-포워드 하이퍼네트워크 기반 접근법을 사용합니다.

**comments**

*   **무엇을 하는가?**: 이미지에서 단순히 무엇이 있는지 맞추는 것을 넘어, **어디에 있는지(bounding box)**까지 찾는 문제입니다.
*   **어떻게 하는가?**: "이런 모양이 '고양이'야"라고 알려주는 서포트셋 이미지를 보고, 그 정보를 바탕으로 최종 분류기(이미지 조각을 보고 '고양이'인지 아닌지 판단하는 부분)를 **즉석에서 생성**하는 하이퍼네트워크 방식이 주로 사용됩니다.


#### 랜드마크 예측 (Landmark Prediction)

사람의 관절이나 로봇의 관절과 같은 이미지 내의 **핵심 포인트들의 골격(skeleton)을 찾는 것**을 목표로 합니다. 이는 일반적으로 이미지-조건부 회귀 문제로 정형화됩니다. 예를 들어, MAML 기반 모델이 인간 포즈 추정[209]에 효과가 있음이 보여졌고, 모듈러 메타러닝은 로보틱스에 성공적으로 적용되었으며[140], 하이퍼네트워크 기반 모델은 새로운 패션 아이템에 대한 퓨샷 옷 피팅에 적용되었습니다[178].

**comments**

*   **무엇을 하는가?**: 사람 사진을 보고 어깨, 팔꿈치, 손목 등 **주요 관절의 위치(좌표)**를 정확히 예측하는 문제입니다.
*   **어떻게 하는가?**: 분류 문제와 유사하게, MAML(좋은 출발점 찾기), 모듈러(부품 조립), 하이퍼네트워크(분류기 생성) 등 다양한 메타러닝 기법이 적용되어, 몇 개의 예시만 보고도 새로운 사람이나 객체의 자세를 예측하는 데 성공했습니다.


#### 퓨샷 객체 분할 (Few-Shot Object Segmentation)

픽셀 단위로 레이블이 지정된 이미지를 얻는 비용 때문에 중요합니다. 하이퍼네트워크 기반 메타-학습기가 원샷(one-shot) 상황에 적용되었고[210], 이후 프로토타입 네트워크를 변형하여 성능이 개선되었습니다[211]. 다른 모델들은 분할 밀도가 낮은 경우를 다룹니다[212].

**comments**

*   **무엇을 하는가?**: 객체의 위치를 네모 박스로 찾는 것을 넘어, **객체에 해당하는 모든 픽셀을 정확히 색칠**하는, 훨씬 더 정교한 문제입니다.
*   **어려움**: 픽셀 하나하나에 레이블을 다는 것은 매우 비싼 작업이라 데이터가 부족합니다.
*   **어떻게 하는가?**: 객체 탐지와 유사하게, "이런 픽셀들이 '고양이'에 해당해"라는 예시(서포트셋)를 보고, 이미지의 다른 픽셀들이 고양이에 속하는지 아닌지를 판단하는 규칙을 빠르게 학습합니다.


#### 이미지 및 비디오 생성 (Image and Video Generation)

[45]에서는 상각된(amortized) 확률적 메타-학습기를 사용하여 단 하나의 이미지로부터 객체의 여러 뷰를 생성하고, 생성적 쿼리 네트워크(generative query networks)[213]는 새로운 뷰에서 장면을 렌더링하며, 말하는 얼굴은 빠른 적응을 위해 적대적 모델의 초기화를 학습함으로써 적은 데이터로 생성됩니다[214]. 비디오 도메인에서는, [215]가 몇 개의 예시 이미지를 단서로 받아 비디오를 합성하는 가중치 생성기를 메타-학습합니다.

**comments**

*   **무엇을 하는가?**: 기존 이미지를 분석하는 것을 넘어, **세상에 없는 새로운 이미지나 비디오를 만들어내는** 문제입니다.
*   **어떻게 하는가?**:
    *   **이미지 생성**: 자동차 옆모습 사진 한 장만 보고 앞모습, 뒷모습 등 다양한 각도의 이미지를 생성하는 법을 배웁니다.
    *   **비디오 생성**: "이런 스타일로 움직여"라는 몇 장의 예시 이미지를 보고, 그 스타일을 따라가는 짧은 비디오를 생성하는 법을 배웁니다.


#### 생성 모델 및 밀도 추정 (Generative Models and Density Estimation)

이미지를 생성할 수 있는 밀도 추정기는 일반적으로 많은 파라미터를 필요로 하며, 따라서 퓨샷 상황에서는 과적합됩니다. PixelCNN 생성기의 경사도 기반 메타러닝이 그들의 퓨샷 학습을 가능하게 함이 보여졌습니다[216].

**comments**

*   **무엇을 하는가?**: 이미지 생성 모델(GAN, VAE 등)은 보통 수많은 이미지를 보고 데이터의 복잡한 확률 분포를 학습해야 합니다. 이 과정을 몇 장의 이미지로도 가능하게 만드는 문제입니다.
*   **어떻게 하는가?**: MAML과 유사하게, "어떤 초기 상태에서 시작해야, 몇 장의 이미지만 보고도 그 이미지들의 스타일을 빠르게 따라 하는 생성 모델을 만들 수 있을까?"를 학습합니다.

### 퓨샷 학습 벤치마크 (Few-Shot Learning Benchmarks)

AI와 머신러닝의 발전은 종종 잘 설계된 벤치마크[217]에 의해 측정되고 촉진됩니다. 전통적인 ML 벤치마크는 모델이 본 인스턴스에서 보지 못한 인스턴스로 일반화해야 하는 하나의 과제와 데이터셋을 정의합니다. 메타러닝에서는 벤치마크 설계가 더 복잡한데, 이는 우리가 종종 본 과제에서 보지 못한 과제로 일반화해야 하는 학습기를 다루기 때문입니다. 따라서 벤치마크 설계는 메타-훈련 및 메타-테스트 과제를 추출할 수 있는 **과제군(families of tasks)**을 정의해야 합니다. 확립된 FSL 벤치마크에는 **miniImageNet**[39], [90], **Tiered-ImageNet**[218], **SlimageNet**[219], **Omniglot**[90] 및 **Meta-Dataset**[111]이 포함됩니다.

**comments**

*   **벤치마크란?**: AI 모델의 성능을 공정하게 비교하고 측정하기 위한 **'표준 시험 문제지'**와 같습니다.
*   **메타러닝 벤치마크의 특징**:
    *   **기존 벤치마크**: '수능 영어 시험'처럼, 정해진 훈련 데이터로 공부하고 정해진 테스트 데이터로 시험을 봅니다.
    *   **메타러닝 벤치마크**: '모의고사 출제용 문제 은행'과 같습니다. 이 문제 은행에서 **훈련용 모의고사(메타-훈련 과제)**와 **실전 수능(메타-테스트 과제)**을 뽑아서 사용합니다. 중요한 것은 훈련 때 풀어본 문제 유형이 실전 시험에는 절대 나오지 않는다는 점입니다.
*   **대표적인 벤치마크**:
    *   **Omniglot**: 전 세계의 다양한 문자(알파벳, 한글, 상형문자 등)를 분류하는 벤치마크. 클래스 수가 매우 많고 클래스당 샘플이 적어 퓨샷 학습의 고전적인 테스트베드입니다.
    *   **miniImageNet / Tiered-ImageNet**: 거대한 ImageNet 데이터셋에서 일부 클래스를 추출하여 만든, 더 현실적인 이미지 분류 벤치마크입니다.



#### 데이터셋 다양성, 편향 및 일반화 (Dataset Diversity, Bias and Generalization)

표준 벤치마크들은 훈련과 평가를 위한 과제를 제공하지만, **다양성이 부족(좁은 $p(\mathcal{T})$)**하다는 문제로 어려움을 겪습니다. 이로 인해 이러한 벤치마크에서의 성능이 실제 세계의 퓨샷 과제에서의 성능을 반영하지 못하게 됩니다. 예를 들어, miniImageNet에서 다른 종류의 동물 사진 간에 전환하는 것은 일반화에 대한 강력한 테스트가 아닙니다. 이상적으로 우리는 더 다양한 범주와 유형의 이미지(위성, 의료, 농업, 수중 등)를 포괄하고, 심지어 메타-훈련과 메타-테스트 과제 간의 **도메인 이동(domain-shifts)**에도 강건해야 합니다.

여기에는 아직 해야 할 일이 남아있는데, 다중샷(many-shot) 설정에서조차 매우 넓은 분포의 데이터에 딥 모델을 맞추는 것 자체가 간단하지 않으며[220], 샘플 범위를 벗어난 데이터로 일반화하는 것도 마찬가지이기 때문입니다[42], [95]. 유사하게, 메타-학습기의 성능은 소스 과제와 타겟 과제 분포 사이에 도메인 이동이 도입될 때 종종 급격히 떨어집니다[117]. 이는 최근의 **Meta-Dataset**[111]과 **CVPR 교차-도메인 퓨샷 챌린지**[221]의 동기가 되었습니다. Meta-Dataset은 넓은 과제 분포에 적응하고 도메인 이동을 넘어 일반화하는 능력을 평가하기 위해 여러 개별 인식 벤치마크를 통합하여 더 넓은 과제 분포 $p(\mathcal{T})$를 제공합니다. 한편, [221]은 일상적인 ImageNet 이미지에서 의료, 위성 및 농업 이미지로 일반화하는 방법들에 도전합니다. 최근 연구는 샘플 효율성뿐만 아니라 도메인 이동 강건성을 위해 메타-훈련함으로써 이러한 문제를 해결하기 시작했습니다[119]. 일반화 문제는 저대표성 국가(under-represented countries)의 데이터에 모델을 적용할 때도 발생합니다[222].

**comments**

*   **현재 벤치마크의 문제점**:
    *   **다양성 부족**: miniImageNet은 결국 '동물'이나 '사물' 사진이라는 한정된 범주 안에서만 문제를 냅니다. 여기서 1등을 했다고 해서, 갑자기 '의료 영상'이나 '위성 사진' 같은 생소한 문제를 잘 풀 것이라고 보장할 수 없습니다.
    *   **쉬운 일반화**: '개 vs 고양이'를 배우고 '사자 vs 호랑이'를 푸는 것은 너무 쉽다는 것입니다. 진정한 일반화 능력을 측정하기 어렵습니다.
*   **새로운 벤치마크의 방향**:
    *   **Meta-Dataset**: 여러 종류의 데이터셋(ImageNet, Omniglot, 항공 사진 등)을 하나로 합쳐, 훨씬 더 **다양하고 어려운 문제 은행**을 만들었습니다.
    *   **교차-도메인 챌린지**: 훈련은 **'일상 사진'**으로 시키고, 테스트는 **'의료 영상', '위성 사진'** 같은 완전히 다른 도메인으로 하여, 극한의 일반화 성능을 측정합니다.
*   **연구 동향**: 이러한 어려운 벤치마크가 등장함에 따라, 단순히 퓨샷 학습만 잘하는 것을 넘어 **'도메인 이동에 강건한' 메타러닝** 방법론에 대한 연구가 활발해지고 있습니다.

## 메타 강화 학습 및 로보틱스 (Meta Reinforcement Learning and Robotics)

강화 학습은 일반적으로 에이전트가 환경 내에서 순차적인 행동 과제를 수행한 후 높은 보상을 얻을 수 있도록 하는 제어 정책을 학습하는 것과 관련이 있습니다. RL은 일반적으로 희소한 보상, 탐험의 필요성, 그리고 최적화 알고리즘의 높은 분산[223]으로 인해 극심한 샘플 비효율성으로 어려움을 겪습니다. 그러나 응용 분야는 종종 메타러닝이 활용할 수 있는 과제군을 자연스럽게 수반합니다. 예를 들어, 다른 위치로 이동하거나 도달하기[188], 다른 환경 내에서 탐색하기[38], 다른 지형 횡단하기[65], 다른 자동차 운전하기[187], 다른 경쟁자 에이전트와 경쟁하기[63], 그리고 개별 로봇 팔다리의 고장과 같은 다른 핸디캡 처리하기[65] 등입니다. 따라서 RL은 과제 분포에 대한 메타러닝이 표준 RL 알고리즘에 비해 샘플 효율성을 향상시키는 데 상당한 성공을 거둔 비옥한 응용 분야를 제공합니다. 이러한 방법의 효과를 직관적으로 이해할 수 있습니다. 예를 들어, 미로 레이아웃에 대한 메타 지식은 그 미로 내에서 탐색해야 하는 모든 과제에 대해 전이 가능합니다.

**comments**

*   **강화학습(RL)이란?**: 로봇이나 게임 캐릭터(에이전트)가 수많은 **시행착오**를 통해 보상을 최대로 받는 행동 규칙(정책)을 배우는 것입니다.
*   **RL의 고질적인 문제**: **학습이 너무너무 느립니다 (샘플 비효율성)**. 보상을 어쩌다 한 번 받기 때문에(희소한 보상), 무엇이 좋은 행동인지 배우는 데 엄청난 시간이 걸립니다.
*   **메타 RL이 필요한 이유**: RL 문제들은 자연스럽게 **'과제군'**의 형태를 띱니다.
    *   '길 찾기' 문제군: (A 지점으로 가기, B 지점으로 가기, C 지점으로 가기...)
    *   '운전' 문제군: (A 자동차 운전하기, B 자동차 운전하기...)
    *   '장애물 피하기' 문제군: (다리 하나 고장났을 때 걷기, 팔 하나 고장났을 때 걷기...)
*   **메타 RL의 효과**: 이런 비슷한 과제들을 여러 번 경험하게 하면, 에이전트는 **"길 찾기의 일반적인 원리"**나 **"운전의 기본 원리"**와 같은 **메타 지식**을 터득하게 됩니다. 덕분에 새로운 목적지나 새로운 자동차가 주어져도 훨씬 빠르게 학습할 수 있습니다.



### 방법론 (Methods)

우리가 이미 살펴본 여러 메타-표현들이 RL에서 탐구되었습니다. 여기에는 초기 조건[16], [173], 하이퍼파라미터[173], [177], 스텝 방향[79] 및 스텝 사이즈[176] 학습이 포함되며, 이는 더 적은 환경 상호작용으로 신경망 정책을 훈련시키는 경사도 기반 학습을 가능하게 합니다. 또한, 주어진 환경의 경험을 임베딩하여 정책을 합성하는 빠른 합성곱[38] 또는 순환[22], [116] 블랙박스 모델 훈련도 있습니다. 최근 연구는 이러한 과제들을 위한 개선된 메타-최적화 알고리즘을 개발했으며[169], [170], [172], 메타-RL에 대한 이론적 보장을 제공했습니다[224].

**comments**

*   지도 학습에서 사용된 메타러닝 기법(MAML, 블랙박스 모델 등)들이 RL에도 거의 그대로 적용되었습니다.
*   목표는 결국 **"최소한의 시행착오(환경 상호작용)만으로 최적의 행동 규칙을 배우는 것"**입니다.



#### 탐험 (Exploration)

RL에 다소 독특한 메타-표현은 **탐험 정책**입니다. RL은 데이터 분포가 고정되어 있지 않고 에이전트의 행동에 따라 변한다는 사실 때문에 복잡합니다. 더욱이, 희소한 보상은 에이전트가 학습을 안내하는 데 사용할 수 있는 보상을 얻기 전에 많은 행동을 취해야 함을 의미할 수 있습니다. 따라서, 학습을 위한 데이터를 어떻게 탐험하고 획득하는지는 모든 RL 알고리즘에서 중요한 요소입니다. 전통적으로 탐험은 무작위 행동을 샘플링하거나[225] 사람이 만든 휴리스틱[226]에 기반합니다. 대신 여러 메타-RL 연구들은 **탐험 전략이나 호기심 함수를 메타 지식 $\omega$로 명시적으로 취급**하고, 그 획득을 메타러닝 문제로 모델링했습니다[24], [186], [187], [227]. 이는 **'탐험하는 법을 학습'**함으로써 샘플 효율성 향상으로 이어졌습니다.

**comments**

*   **탐험이란?**: 아직 가보지 않은 길을 가보는 것. RL 에이전트가 똑똑해지려면, 당장의 보상이 없더라도 세상을 적극적으로 돌아다니며 정보를 수집해야 합니다.
*   **기존 방식**: 그냥 **'랜덤'**하게 움직이거나, 사람이 **'대충'** 규칙을 정해줬습니다. (예: 10% 확률로 무작위 행동)
*   **메타 RL 방식**: "어떻게 돌아다녀야 **가장 효율적으로 새로운 정보를 얻을 수 있을까?**"라는 **'최고의 탐험 전략'($\omega$)** 자체를 학습합니다. 마치 최고의 탐험가를 훈련시키는 것과 같습니다.



#### 최적화 (Optimization)

RL은 학습된 정책이 '훈련 세트' 에피소드에서조차 최적과는 거리가 먼 어려운 최적화 문제입니다. 이는 메타-SL(지도학습)과 대조적으로, 메타-RL 방법이 샘플 효율성뿐만 아니라 **점근적 성능(asymptotic performance)을 높이기 위해** 더 흔하게 사용됨을 의미합니다[23], [177], [183]. 최적화의 어려움은 또한 RL 에이전트가 전통적인 희소한 보상 목표 대신 또는 추가적으로 최적화해야 하는 **손실(또는 보상)을 학습**하는 연구가 비교적 더 많았음을 의미합니다[121], [124], [183], [228]. 이러한 학습된 손실은 실제 목표[23], [228]에 비해 최적화하기 더 쉬울 수 있습니다(더 조밀하고, 더 매끄러움). 이는 또한 보상 학습이 학습 내재적 동기(learning intrinsic motivation)의 메타러닝을 구현하는 것으로 간주될 수 있기 때문에 탐험과도 연결됩니다[184].

**comments**

*   **RL 최적화의 어려움**: RL은 너무 어려워서, 아무리 오래 학습해도 '완벽한' 해법을 찾기가 거의 불가능합니다.
*   **메타 RL의 다른 목표**:
    *   **메타-SL**: 주로 **'학습 속도'**를 높이는 데 관심이 있습니다.
    *   **메타-RL**: '학습 속도'뿐만 아니라, **'최종 도달 성능'** 자체를 높이는 데도 많이 사용됩니다.
*   **보상 함수 학습**: "원래 게임 점수(희소한 보상)는 너무 받기 힘드니, 에이전트가 더 쉽게 학습할 수 있도록 **'중간 목표'에 해당하는 새로운 보상 함수($\omega$)**를 만들어주자!"라는 아이디어입니다.
    *   **예시**: '열쇠를 찾아 문 열기' 게임에서, '문 열기'(+100점)라는 최종 보상만 주는 대신, '열쇠 근처로 가기'(+1점), '열쇠 줍기'(+10점) 같은 **'학습용 가짜 보상'**을 메타러닝으로 학습합니다. 이는 에이전트에게 **'내재적 호기심/동기'**를 부여하는 것과 같습니다.



#### 온라인 메타-RL (Online meta-RL)

메타-RL 연구의 상당 부분은 손실[121], [183], 보상[177], [184], 하이퍼파라미터[175], [176], 또는 탐험 전략[185]과 같은 메타 지식이 단일 과제를 학습하는 동안 기반 정책과 함께 **온라인으로 훈련**되는 단일-과제 설정을 다루었습니다. 따라서 이러한 방법들은 과제군을 필요로 하지 않으며, 각자의 기반 학습기 성능에 직접적인 개선을 제공합니다.

**comments**

*   **온라인 메타-RL이란?**: 여러 과제를 미리 경험하는 대신, **단 하나의 어려운 과제를 플레이하는 실시간으로** '학습 전략'($\omega$)을 계속해서 수정하고 개선해나가는 방식입니다.
*   **장점**: 여러 과제 데이터가 필요 없어서 적용하기 쉽고, 현재 풀고 있는 문제의 성능을 즉각적으로 향상시킬 수 있습니다.



#### On- vs Off-Policy 메타-RL

전통적인 RL의 주요 이분법은 PPO[225]와 같은 on-policy 학습과 SAC[229]와 같은 off-policy 학습 사이에 있습니다. Off-policy 방법은 일반적으로 훨씬 더 샘플 효율적입니다. 그러나 off-policy 방법은 메타-RL로 확장하기 더 어려웠고, 이로 인해 더 많은 메타-RL 방법이 on-policy RL 방법 위에 구축되어 메타-RL의 절대적인 성능을 제한했습니다. Off-policy 메타-RL 방법에 대한 초기 연구는 강력한 결과를 낳았습니다[114], [121], [171], [228]. Off-policy 학습은 또한 메타-RL에서 비용이 많이 들 수 있는 메타-훈련 단계의 효율성을 향상시킵니다[114]. 또한 메타-훈련의 리플레이 버퍼 샘플로부터 메타-테스트를 가속화할 새로운 기회를 제공합니다[171].

**comments**

*   **On-policy vs Off-policy**:
    *   **On-policy**: **'현재 내가 하고 있는 방식'**으로만 데이터를 수집하고 학습합니다. (보수적이지만 안정적)
    *   **Off-policy**: **'과거의 다른 방식'**으로 수집된 데이터까지 모두 활용하여 학습합니다. (데이터 효율이 높지만 불안정할 수 있음)
*   **메타-RL에서의 상황**:
    *   대부분의 메타-RL은 구현이 쉬운 **On-policy** 기반이라, 데이터 효율이 떨어지는 한계가 있었습니다.
    *   최근 **Off-policy** 기반 메타-RL 연구가 활발해지면서, **더 적은 데이터로 더 빠르게 학습**하는 것이 가능해지고 있습니다.



#### 기타 동향 및 과제 (Other Trends and Challenges)

[65]는 실제 물리적 로봇에서 성공적인 메타-RL을 시연했다는 점에서 주목할 만합니다. 로보틱스에서의 지식 전이는 종종 구성적으로(compositionally) 가장 잘 연구됩니다[230]. 예를 들어, 걷기, 탐색하기, 물체 집기/놓기는 방 청소 로봇의 서브루틴이 될 수 있습니다. 그러나 효과적인 구성적 지식 전이를 갖춘 메타-학습기를 개발하는 것은 아직 미해결 질문이며, 모듈러 메타러닝[141]이 하나의 옵션입니다. 비지도 메타-RL 변형은 수동으로 지정된 보상 없이 메타-훈련을 수행하거나[231], 또는 메타-테스트에서 새로운 보상 없이 변경된 환경에 적응하는 것[232]을 목표로 합니다. 지속적인 적응은 에이전트가 하나의 메타-테스트 에피소드 내에서 일련의 과제에 적응할 수 있는 능력을 제공하며[63]–[65], 이는 연속 학습과 유사합니다. 마지막으로, 메타러닝은 모방[115] 및 역 RL[233]에도 적용되었습니다.

**comments**

*   **실제 로봇 적용**: 시뮬레이션을 넘어, 실제 로봇에 메타-RL을 적용하는 성공 사례가 등장하고 있습니다.
*   **구성적 학습 (Compositional Learning)**: '걷기', '잡기' 같은 **기본 동작(레고 블록)**들을 메타-학습한 뒤, 이들을 **조합**하여 '방 청소하기' 같은 복잡한 임무를 수행하게 하는 연구가 중요해지고 있습니다.
*   **비지도 메타-RL**: 사람이 보상을 설계해주지 않아도, AI가 **스스로 흥미로운 목표를 찾아** 학습하는 법을 배웁니다.
*   **지속적인 적응**: 한 번의 실행 중에 환경이 계속 변할 때(예: 갑자기 다리가 고장남), **실시간으로 계속해서 적응**해나가는 능력을 학습합니다.

### 벤치마크 (Benchmarks)

RL을 위한 메타러닝 벤치마크는 일반적으로 에이전트가 '학습하는 법을 배우도록' 훈련하고 평가하기 위해 풀어야 할 **과제군(a family to solve)**을 정의합니다. 이는 달성해야 할 **과제(보상 함수)**일 수도 있고, **도메인(서로 다른 환경 또는 MDPs)**일 수도 있습니다.

**comments**

*   **메타-RL 벤치마크란?**: AI 에이전트의 **'학습 능력'** 자체를 평가하기 위한 표준화된 시험 환경입니다.
*   **두 가지 유형**:
    1.  **과제(Task) 변경**: 환경(맵 구조, 물리 법칙 등)은 똑같은데, **목표(Goal)**만 계속 바뀝니다. (예: 같은 미로 안에서 출구 위치만 계속 바꿈)
    2.  **도메인(Domain) 변경**: 목표는 비슷한데, **환경** 자체가 계속 바뀝니다. (예: '출구 찾기'라는 목표는 같지만, 미로의 구조가 계속 바뀜)



#### 이산 제어 RL (Discrete Control RL)

시각-기반 제어를 위한 초기 메타-RL 벤치마크는 **아케이드 학습 환경(ALE)**[234]으로, 고전 아타리 게임 세트를 메타-훈련과 메타-테스트용으로 분할합니다. 여기서 프로토콜은 메타-테스트 환경에서 고정된 수의 타임스텝 이후의 보상을 평가하는 것입니다. 한 가지 어려운 점은 게임 간의 **엄청난 다양성(넓은 $p(\mathcal{T})$)**으로, 이는 성공적인 메타-훈련을 어렵게 만들고 지식 전이의 이점을 제한합니다[234]. 또 다른 벤치마크[235]는 **소닉-헤지혹** 레벨을 메타-훈련/메타-테스트로 분할하는 것에 기반합니다. 여기서 과제 분포는 더 좁고, 유익한 메타러닝을 달성하기가 비교적 더 쉽습니다. Cobbe 등[236]은 메타-RL 벤치마킹을 위해 특별히 설계된 두 개의 비디오 게임을 제안했습니다. **CoinRun** 게임[236]은 다양한 난이도와 시각적 외형을 가진 $2^{32}$개의 절차적으로 생성된 레벨을 제공합니다. 그들은 새로운 레벨에 안정적으로 일반화하기 위해서는 약 10,000 레벨의 메타-훈련 경험이 필요함을 보여줍니다. CoinRun은 빠른 적응보다는 직접적인 일반화를 테스트하도록 주로 설계되었으며, 과제에 대한 적응을 테스트하기보다는 MDP 환경에 대한 일반화를 테스트하는 분포를 제공하는 것으로 볼 수 있습니다. 더 넓은 과제 분포에서 빠른 학습을 더 잘 테스트하기 위해, **ProcGen**[236]은 CoinRun을 포함하여 16개의 절차적으로 생성된 게임 세트를 제공합니다.

**comments**

*   **이산 제어(Discrete Control)**: 조이스틱처럼 '왼쪽, 오른쪽, 점프' 등 정해진 몇 가지 행동만 할 수 있는 환경입니다. (주로 비디오 게임)
*   **벤치마크 종류**:
    *   **ALE (아타리 게임)**: '퐁', '스페이스 인베이더' 등 수십 개의 **서로 완전히 다른 게임**들로 구성됩니다. 게임 간 편차가 너무 커서, 한 게임에서 배운 지식이 다른 게임에 별 도움이 안 되는 **'지식 전이의 어려움'**을 테스트하기 좋습니다.
    *   **소닉-헤지혹**: **같은 게임의 다른 레벨**들로 구성됩니다. 게임 규칙은 같으므로, '소닉 게임을 잘하는 법'이라는 메타 지식을 배우기 더 쉽습니다.
    *   **CoinRun / ProcGen**: 메타-RL 테스트를 위해 **인공적으로 만들어진 게임**입니다. 레벨이 거의 무한대로 자동 생성되므로, AI가 맵을 외우는 것을 방지하고 진정한 **'일반화 능력'**을 테스트할 수 있습니다. ProcGen은 여기서 더 나아가 게임의 종류 자체도 다양하게 제공합니다.



#### 연속 제어 RL (Continuous Control RL)

gym[237]과 같은 일반적인 벤치마크가 RL 연구에 큰 도움이 되었지만, 메타-RL 벤치마크에 대해서는 합의가 덜 이루어져 있어 기존 연구들을 비교하기 어렵습니다. 대부분의 연속 제어 메타-RL 연구들은 특정 위치나 속도로 이동하기[16], [114] 또는 다른 지형 횡단하기[65]와 같은 특정 과제의 저차원 파라미터 변형인 **자체 제작(home-brewed) 벤치마크**를 제안했습니다. 최근 여러 다중-MDP 벤치마크[238], [239]가 제안되었지만, 이들은 주로 다른 과제보다는 다른 환경적 섭동(perturbation)에 대한 일반화를 테스트합니다. **Meta-World** 벤치마크[240]는 레버 당기기나 문 열기와 같은 간단한 파라미터 변형부터 다양한 50개의 연속 제어 과제 모음을 상태 기반 작동(state-based actuation)으로 제공합니다. 이 벤치마크는 더 비교 가능한 평가와 과제 분포 내/외부로의 일반화 조사를 가능하게 할 것입니다. Meta-World 평가[240]는 기존 메타-RL 방법들이 넓은 과제 분포와 메타-훈련/메타-테스트 간의 차이에 대해 일반화하는 데 어려움을 겪는다는 것을 시사합니다. 이는 우리의 메타-RL 모델이 너무 약하거나, 벤치마크가 효과적인 learning-to-learn을 위해 과제의 수나 범위 측면에서 너무 작기 때문일 수 있습니다. 메타-RL에 적합한 또 다른 최근 벤치마크는 **PHYRE**[241]로, 간단한 행동으로 해결할 수 있지만 효율적으로 다루기 위해서는 모델 기반 추론이 필요할 가능성이 높은 50개의 시각 기반 물리 과제 템플릿을 제공합니다. 이들은 또한 템플릿 내/외부로의 일반화 테스트를 제공합니다.

**comments**

*   **연속 제어(Continuous Control)**: 로봇 팔의 관절 각도처럼 행동이 연속적인 값을 갖는 환경입니다.
*   **문제점**: 표준화된 시험지가 없어서 연구마다 **'자체 제작' 문제**로 성능을 평가해왔습니다. (연구 간 공정한 비교가 어려움)
*   **새로운 벤치마크**:
    *   **Meta-World**: '문 열기', '서랍 닫기', '버튼 누르기' 등 로봇 팔이 수행할 수 있는 **50가지의 표준화된 과제**를 제공합니다. 이를 통해 "우리 모델이 50개 중 몇 개나 풀 수 있다"고 객관적으로 비교할 수 있게 되었습니다.
    *   **PHYRE**: 간단한 도구를 움직여 공을 목표 지점에 도달시키는 **물리 퍼즐 게임**입니다. 단순한 반사 신경이 아니라, "어떻게 움직여야 공이 저렇게 굴러갈까?"를 예측하는 **'물리적 직관'과 '추론 능력'**을 테스트하기 위해 설계되었습니다.



#### 논의 (Discussion)

시각-기반 메타-RL의 한 가지 복잡한 점은 (컴퓨터 비전에서처럼) 시각적 일반화와 더 일반적으로 제어 전략의 빠른 학습을 분리하는 것입니다. 예를 들어, CoinRun[236] 평가는 배치 정규화(batch norm)와 같은 표준 비전 기술로부터 큰 이점을 보였으며, 이는 인지(perception)가 주요 병목 현상임을 시사합니다.

**comments**

*   **핵심 질문**: 시각 기반 RL(예: 게임 화면을 보고 플레이)에서 AI가 게임을 못하는 이유가, **'화면을 제대로 이해하지 못해서(시각적 일반화 실패)'**일까, 아니면 **'게임 규칙을 빨리 배우지 못해서(제어 전략 학습 실패)'**일까?
*   **CoinRun의 교훈**: CoinRun 게임에서는 단순히 이미지 처리 기술(배치 정규화)만 잘 써줘도 성능이 크게 올랐습니다. 이는 AI가 게임 자체를 못했다기보다는, **매번 바뀌는 게임 화면에 적응하는 것을 더 어려워했다**는 것을 의미합니다. 즉, '보는 것(perception)'이 '행동하는 것(control)'보다 더 큰 문제일 수 있다는 점을 시사합니다.

### 환경 학습 및 Sim2Real (Environment Learning and Sim2Real)

Sim2Real에서는 시뮬레이션에서 훈련된 모델이 실제 세계에서도 일반화할 수 있도록 하는 데 관심이 있습니다. 고전적인 **도메인 랜덤화(domain randomization)** 접근법은 도메인/MDP의 넓은 분포에 대해 시뮬레이션하여, 실제 세계에서 성공할 만큼 충분히 강건한 모델을 훈련시키는 것을 목표로 하며, 비전[242]과 RL[161] 모두에서 성공을 거두었습니다. 그럼에도 불구하고, 시뮬레이션 분포를 조정하는 것은 여전히 어려운 과제로 남아있습니다.

이는 메타러닝 설정으로 이어지는데, 여기서 **내부-수준 최적화**는 시뮬레이션에서 모델을 학습하고, **외부-수준 최적화 $\mathcal{L}^{\text{meta}}$**는 실제 세계에서 모델의 성능을 평가하며, **메타-표현 $\omega$**는 시뮬레이션 환경의 파라미터에 해당합니다. 이 패러다임은 RL[163]뿐만 아니라 비전[162], [243]에서도 사용되었습니다. 이 경우, 메타-훈련 과제에 사용되는 소스 과제는 미리 제공된 데이터 분포가 아니라, 오메가($\omega$)에 의해 매개변수화된 $\mathcal{D}_{\text{source}}(\omega)$입니다. 그러나 내부 과제 학습 단계의 긴 그래프를 통한 비용이 많이 드는 역전파와, Sim2Real의 경우 실제 세계에서의 $\mathcal{L}^{\text{meta}}$ 평가 횟수를 최소화해야 하는 측면에서 여전히 어려움이 남아있습니다.

**comments**

*   **Sim2Real이란?**: **시뮬레이션(가상현실)**에서 안전하고 빠르게 AI를 훈련시킨 뒤, **실제 세계(Real world)**에 투입하는 기술입니다. (예: 자율주행차, 로봇 팔)
*   **문제점**: 시뮬레이션과 현실은 미묘하게 다릅니다 (Reality Gap). 그래서 시뮬레이션에서 1등을 해도 현실에서는 제대로 작동하지 않을 수 있습니다.
*   **기존 해결책 (도메인 랜덤화)**: 시뮬레이션 환경을 **아주 다채롭게** 만듭니다. 조명, 색깔, 마찰력 등을 마구잡이로 바꿔서, AI가 "어지간한 변화에는 끄떡없는" 강인한 모델이 되도록 훈련시킵니다.
*   **메타러닝적 해결책 (환경 학습)**:
    *   **아이디어**: "무작정 환경을 바꾸지 말고, **현실과 가장 유사한 시뮬레이션 환경**을 만들도록 **시뮬레이터 자체를 학습**시키자!"
    *   **작동 방식 (메타러닝 구조)**:
        *   **내부 루프**: 현재의 시뮬레이터($\omega$)가 만들어준 가상 환경에서 AI 모델($\theta$)을 훈련시킵니다.
        *   **외부 루프**: 그렇게 훈련된 AI 모델($\theta$)을 **실제 세계**에 가서 테스트합니다. 그 성능($\mathcal{L}^{\text{meta}}$)을 보고, "어떻게 해야 시뮬레이터($\omega$)를 현실과 더 비슷하게 만들 수 있을까?"를 학습하여 시뮬레이터를 업데이트합니다.
    *   **어려움**: 실제 세계에서 테스트하는 것은 **비용이 비싸고 위험**하기 때문에, 외부 루프 평가 횟수를 최소화해야 합니다.



## 신경망 구조 탐색 (Neural Architecture Search, NAS)

구조 탐색[18], [25], [26], [37], [131]은 $\omega$가 신경망의 구조를 명시하는 일종의 하이퍼파라미터 최적화로 볼 수 있습니다. 내부 최적화는 명시된 구조로 네트워크를 훈련시키고, 외부 최적화는 좋은 검증 성능을 가진 구조를 탐색합니다. NAS는 특히 다음과 같은 이유로 어렵습니다: 

(i) 내부 루프를 완전히 평가하는 것은 다중샷(many-shot) 신경망을 끝까지 훈련시켜야 하므로 비용이 많이 듭니다. 이는 훈련 세트의 일부만 사용하거나, 내부 루프를 조기 종료하거나, 온라인 메타러닝에서처럼 $\omega$와 $\theta$에 대한 경사 하강을 번갈아 수행하는[18] 등의 근사로 이어집니다. 
(ii) 탐색 공간이 정의하기 어렵고 최적화하기 어렵습니다. 이는 대부분의 탐색 공간이 광범위하고, 구조의 공간이 자명하게 미분 가능하지 않기 때문입니다. 이는 셀-수준 탐색[18], [26]으로 탐색 공간을 제약하거나, RL[26], 이산적 경사도 추정기[133] 및 진화[25], [131]에 의존하게 만듭니다.

### 최신 이슈 (Topical Issues)

NAS 자체가 하이퍼파라미터 또는 가설-클래스 메타러닝의 한 예로 볼 수 있지만, 다른 형태의 메타러닝과 상호작용할 수도 있습니다. NAS는 비용이 많이 들기 때문에, 발견된 구조가 새로운 문제로 일반화될 수 있는지 여부가 최신 이슈입니다[244]. 여러 데이터셋에 걸쳐 메타-훈련을 하면 구조의 교차-과제 일반화가 향상될 수 있습니다[136]. 마지막으로, 퓨샷 학습에 적합한 구조를 훈련시키기 위해 NAS 메타-목표를 정의할 수도 있습니다[245], [246]. MAML[16]과 같은 빠른 적응 초기 조건 메타러닝 접근법과 유사하게, 특정 과제에 쉽게 적응할 수 있는 좋은 초기 구조[135]나 구조 사전 확률(priors)[136]을 훈련시킬 수 있습니다.

### 벤치마크 (Benchmarks)

NAS는 종종 CIFAR-10에서 평가되지만, 수행 비용이 비싸고 하이퍼파라미터 튜닝과 같은 교란 요인(confounding factors) 때문에 결과를 재현하기 어렵습니다[247]. 재현 가능하고 접근 가능한 연구를 지원하기 위해, **NASbenches**[248]는 수많은 네트워크 구조에 대한 미리 계산된 성능 측정치를 제공합니다.

**comments**

*   **NAS란?**: 최고의 성능을 내는 **신경망의 '설계도(구조)'**를 사람이 아니라 AI가 자동으로 찾게 하는 기술입니다.
*   **메타러닝과의 관계**: 구조($\omega$)를 찾고, 그 구조로 모델($\theta$)을 훈련시켜 성능을 평가하는 **전형적인 이중 루프(내부/외부) 구조**를 가집니다.
*   **어려움**:
    1.  **비싼 평가 비용**: 설계도 하나가 좋은지 평가하려면, 그 설계도로 모델을 처음부터 끝까지 다 훈련시켜봐야 합니다. (시간이 너무 오래 걸림)
    2.  **어려운 탐색**: 탐색해야 할 설계도의 종류가 거의 무한대에 가깝고, 설계도라는 개념은 미분이 불가능해서 경사 하강법을 쓰기 어렵습니다. (그래서 RL이나 진화 알고리즘을 주로 사용)
*   **최신 연구 동향**:
    *   **구조의 일반화**: CIFAR-10 데이터에서 찾은 최고의 구조가, ImageNet이나 다른 문제에서도 효과가 있을까? (교차-과제 일반화)
    *   **메타러닝과의 결합**: "MAML처럼, **'약간만 수정해도 어떤 문제에든 강해지는 만능 초기 구조'**를 찾을 수 있지 않을까?" 라는 연구가 진행 중입니다.
*   **벤치마크 (NASbenches)**: NAS 연구는 너무 비싸서 아무나 하기 어렵습니다. 그래서 "우리가 미리 수많은 구조에 대해 훈련/평가를 다 해놨으니, 여러분은 이 **'성능 데이터베이스'**를 가지고 빠르고 저렴하게 연구하세요"라는 취지로 만들어진 벤치마크입니다.

## 베이지안 메타러닝 (Bayesian Meta-learning)

베이지안 메타러닝 접근법은 베이지안 계층 모델링을 통해 메타러닝을 정형화하고, 파라미터의 직접적인 최적화 대신 베이지안 추론을 학습에 사용합니다. 메타러닝 맥락에서, 베이지안 학습은 일반적으로 다루기 어렵기(intractable) 때문에, 확률적 변분 추론이나 샘플링과 같은 근사법이 사용됩니다.

베이지안 메타러닝은 중요하게도 $\omega$ 파라미터에 대한 **불확실성 측정치**를 제공하며, 따라서 예측 불확실성에 대한 측정치도 제공합니다. 이는 안전이 중요한 응용 분야, RL에서의 탐험, 그리고 능동 학습(active learning)에 중요할 수 있습니다.

여러 저자들이 경쟁력 있는 결과를 가진 복잡한 신경망 모델에 대한 베이지안 접근법을 탐구해왔습니다. 예를 들어, 변분 오토인코더를 확장하여 과제 변수를 명시적으로 모델링하거나[75], 가우시안 프로세스에서 영감을 받았지만 신경망으로 구현된 피드-포워드 베이지안 메타-학습기인 뉴럴 프로세스(Neural Processes)[179]를 정의했습니다. 딥 커널 러닝(Deep kernel learning) 또한 메타러닝 설정에 적용된 활발한 연구 분야이며[249], 종종 가우시안 프로세스와 결합됩니다[250]. [76]에서는 경사도 기반 메타러닝이 계층적 경험적 베이즈 추론 문제(즉, 사전 확률 학습)로 재구성되며, 이는 과제-특화 파라미터 $\theta$의 불확실성을 모델링합니다. 베이지안 MAML[251]은 비-가우시안 사후 확률을 허용하는 베이지안 앙상블 접근법을 사용하여 이 모델을 개선하며, 이후 연구에서는 비용이 많이 드는 앙상블의 필요성을 제거했습니다[45], [252]. 확률적 MAML(Probabilistic MAML)[98]에서는 메타 지식 $\omega$의 불확실성이 모델링되고, $\theta$에 대해서는 MAP 추정치가 사용됩니다. 점점 더, 이러한 베이지안 방법들은 모호한 과제, 능동 학습 및 RL 문제를 다루는 데 효과가 있음이 보여지고 있습니다.

위와는 별개로, [253]에서처럼 베이지안 추론 과정 자체를 돕기 위해 메타러닝이 제안되기도 했는데, 여기서 저자들은 효율적인 적응적 샘플링 방법을 제공하기 위해 베이지안 샘플러를 조정합니다.

**comments**

*   **베이지안 메타러닝이란?**: "최고의 학습법($\omega$)은 **하나의 고정된 값**이 아니라, **확률 분포**다"라고 생각하는 접근법입니다.
*   **핵심 장점: 불확실성 측정**:
    *   "이 예측은 80% 확률로 고양이지만, **20% 정도는 확신할 수 없어**"라고 말할 수 있습니다.
    *   **왜 중요한가?**:
        *   **안전**: 자율주행차가 "저 앞에 있는 물체가 사람인지 아닌지 헷갈린다"고 말할 수 있다면, 일단 멈추는 등 안전한 행동을 할 수 있습니다.
        *   **탐험(RL)**: 에이전트가 "이 행동의 결과는 예측이 잘 안되는데?"라고 생각하면, 새로운 정보를 얻기 위해 그 행동을 시도해볼 수 있습니다(호기심).
        *   **능동 학습**: 모델이 "이 데이터는 정답을 정말 모르겠다"고 말하면, 사람에게 그 데이터의 정답을 물어봄으로써 효율적으로 학습할 수 있습니다.
*   **주요 연구**: MAML과 같은 기존 메타러닝 기법에 **확률적 관점**을 도입하여, 파라미터($\theta$ 또는 $\omega$)를 점이 아닌 분포로 학습하려는 연구가 주를 이룹니다.



## 비지도 메타러닝 (Unsupervised Meta-Learning)

비지도 학습이 내부 루프 또는 외부 루프에서 수행되는지, 그리고 메타-훈련 중인지 메타-테스트 중인지에 따라, 비지도 학습이 메타러닝과 상호작용하는 몇 가지 뚜렷한 방식이 있습니다.

### 지도 학습기의 비지도 학습 (Unsupervised Learning of a Supervised Learner)

여기서의 목표는 (예를 들어, 지도 학습 미세 조정을 위한 MAML[16] 스타일의 초기 조건을 통해) 지도 학습 알고리즘을 배우는 것이지만, 메타-훈련을 위한 대규모 소스 과제 세트의 요구 사항 없이 그렇게 하는 것입니다[254]–[256]. 이를 위해, 클러스터링이나 클래스를 보존하는 데이터 증강을 통해 비지도 방식으로 합성 소스 과제를 구성하고, 이를 사용하여 메타-훈련을 위한 메타-목표를 정의합니다.

**comments**

*   **목표**: MAML을 훈련시키고 싶은데, **훈련용 과제 데이터가 하나도 없을 때** 어떻게 할까?
*   **해결책**: 레이블이 없는 대규모 데이터만 가지고, **'가짜' 훈련 과제를 스스로 만들어냅니다.**
    *   **방법 1 (클러스터링)**: 비슷한 이미지끼리 묶어서 "이 그룹은 클래스 1, 저 그룹은 클래스 2"라고 임시 과제를 만듭니다.
    *   **방법 2 (데이터 증강)**: 고양이 사진 한 장을 여러 방식으로 변형(회전, 반전 등)하고, "이 변형된 이미지들은 모두 같은 고양이 클래스다"라고 과제를 만듭니다.


### 비지도 학습기의 지도 학습 (Supervised Learning of an Unsupervised Learner)

이 방법 계열은 비지도 학습기를 메타-훈련하는 것을 목표로 합니다. 예를 들어, 비지도 알고리즘이 다운스트림 지도 학습 과제에 대해 잘 작동하도록 훈련시킵니다. 비지도 학습 규칙[21]이나 손실[101], [125]을 훈련시켜, 비지도 표현을 지도 과제에 재사용한 후[21] 또는 레이블 없는 데이터에 기반하여 적응한 후[101], [125] 다운스트림 지도 학습 성능이 최적화되도록 할 수 있습니다. 대안으로, 클러스터링과 같은 비지도 과제가 고립된 것이 아니라 하나의 과제군에 존재할 때, 여러 소스 과제에 대해 '클러스터링하는 법'을 배우면 새로운 클러스터링 과제군에서 더 나은 성능을 제공할 수 있습니다[180]-[182], [257], [258]. 피드-포워드 모델을 사용하는 이 그룹의 방법들은 종종 **상각 클러스터링(amortized clustering)**[181], [182]으로 알려져 있는데, 이는 일반적으로 반복적인 클러스터링 알고리즘 계산을 단일 추론 모델 훈련 비용으로 상각하고, 이후에는 단일 피드-포워드 패스를 사용하여 클러스터링을 수행하기 때문입니다.

**comments**

*   **목표**: 클러스터링이나 차원 축소 같은 **비지도 학습 알고리즘 자체를 더 잘하게** 만드는 것입니다.
*   **어떻게?**: **지도 학습 방식**으로 비지도 학습기를 평가하고 훈련시킵니다.
    *   **방법 1 (다운스트림 과제)**: 비지도 학습으로 특징을 추출한 뒤, 그 특징으로 지도 학습(분류)을 시켜봅니다. 최종 분류 성능이 가장 좋게 나오는 비지도 학습법을 찾습니다.
    *   **방법 2 (메타-클러스터링)**: 여러 데이터셋에 대해 클러스터링을 수행해보면서, **"어떻게 하면 새로운 데이터셋도 가장 잘 클러스터링할 수 있을까?"**라는 '클러스터링 노하우'를 배웁니다.



## 연속, 온라인 및 적응 학습 (Continual, Online and Adaptive Learning)

**연속 학습 (Continual Learning)**은 순서대로 제시되는 과제들을 학습하는 인간과 같은 능력을 말합니다. 이상적으로 이는 과거 경험을 활용하여 새로운 과제를 더 잘 배우면서, 이전에 학습한 과제를 잊지 않고, 과거 데이터를 저장할 필요 없이 수행됩니다[62]. 딥러닝 신경망은 이러한 기준을 충족시키는 데 어려움을 겪으며, 특히 이전 과제에서 본 정보를 잊어버리는 경향이 있는데, 이는 **치명적 망각(catastrophic forgetting)**으로 알려진 현상입니다. 메타러닝은 예를 들어, 서포트셋은 하나의 새로운 과제를 포함하지만 쿼리셋은 지금까지 본 모든 과제에서 추출된 예제를 포함하는 학습 에피소드 시퀀스를 정의함으로써, 연속 학습의 요구 사항을 메타-목표에 포함시킬 수 있습니다[107], [174]. 가중치 사전 확률[138], 경사 하강 사전 조건화 행렬[107], 또는 RNN 학습 최적화기[174], 또는 특징 표현[259]과 같은 다양한 메타-표현이 연속 학습 성능을 향상시키기 위해 학습될 수 있습니다. 관련된 아이디어는 간섭 없이 개선을 위한 로컬 편집 업데이트를 지원하기 위해 표현을 메타-훈련하는 것입니다[260].

**comments**

*   **연속 학습이란?**: AI가 과제 A, B, C를 순서대로 배울 때, **이전에 배운 A, B를 잊지 않는 능력**입니다.
*   **메타러닝의 역할**: "어떻게 학습해야 **새로운 지식을 배우면서도 옛 지식을 잊지 않을까?**"라는 **'연속 학습 능력' 자체를 최적화**하는 학습법($\omega$)을 배웁니다.
*   **어떻게?**: 훈련 시나리오를 실제 연속 학습 상황처럼 만듭니다.
    *   **내부 루프**: 새로운 과제 C를 공부합니다.
    *   **외부 루프**: 공부가 끝난 뒤, **이전에 배운 A, B와 새로 배운 C를 모두** 테스트하여 종합 점수를 매깁니다. 이 종합 점수를 최대화하는 학습 전략을 찾습니다.



**온라인 및 적응 학습 (Online and Adaptive Learning)**
또한 스트림으로 들어오는 과제들을 고려하지만, 오래된 과제를 기억하는 것보다는 스트림의 **현재 과제에 효과적으로 적응**하는 능력에 더 관심이 있습니다. 이를 위해, MAML의 온라인 확장이 제안되어[99] 과제 시퀀스 동안 MAML 스타일의 메타-훈련을 온라인으로 수행합니다. 한편, 다른 연구들[63]-[65]은 메타-테스트에서 타겟 과제 시퀀스에 대한 적응 능력을 평가하기 전에, 소스 과제에 대해 미리 메타-훈련을 수행하는 설정을 고려합니다.

**comments**

*   **온라인/적응 학습이란?**: 과거는 중요하지 않고, **지금 당장 마주한 새로운 환경/과제에 얼마나 빨리 적응하는가**에 초점을 맞춥니다.
*   **메타러닝의 역할**:
    *   **온라인 MAML**: 과제가 주어지는 실시간으로 MAML 훈련을 계속 진행하여, 변화하는 환경에 맞춰 '최적의 출발점'을 계속 업데이트합니다.
    *   **사전 훈련 후 적응**: 미리 다양한 상황에 대한 '적응 훈련'을 마친 뒤(메타-훈련), 실제 상황(메타-테스트)에서 환경이 계속 변할 때 얼마나 잘 적응하는지를 평가합니다.

## 도메인 적응 및 도메인 일반화 (Domain Adaptation and Domain Generalization)

**도메인 이동(Domain-shift)**은 배포 시 마주치는 데이터의 통계가 훈련에 사용된 데이터와 다른 것을 의미합니다. 수많은 도메인 적응 및 일반화 알고리즘이 지도, 비지도, 준지도 설정에서 이 문제를 해결하기 위해 연구되었습니다[58].

**도메인 일반화 (Domain Generalization)**
도메인 일반화는 종종 훈련 도메인 분포를 활용하여, 훈련-테스트 간 도메인 이동에 대한 모델의 강건성을 높이는 것을 목표로 합니다[261]. 훈련 도메인과 다른 검증 도메인을 사용하여[262], 정규화기[95], 손실[42], 노이즈 증강[119]과 같은 다양한 종류의 메타 지식을 (메타) 학습하여, 훈련-테스트 간 도메인 이동에 대한 학습된 모델의 강건성을 최대화할 수 있습니다.

**comments**

*   **도메인 일반화란?**: 훈련 때 본 적 없는 **완전히 새로운 스타일(도메인)의 데이터**가 들어와도 성능이 떨어지지 않는 강인한 모델을 만드는 것입니다.
*   **메타러닝의 역할**: "어떻게 훈련해야 **어떤 스타일의 데이터가 들어와도 잘 작동하는 모델**을 만들 수 있을까?"라는 **'강건한 학습법'($\omega$)** 자체를 배웁니다.
*   **어떻게?**: 훈련 단계에서 일부러 도메인 이동 상황을 시뮬레이션합니다.
    *   **내부 루프**: '사진' 도메인으로 모델을 훈련시킵니다.
    *   **외부 루프**: 훈련된 모델을 '스케치' 도메인으로 평가합니다. 이 '스케치' 도메인에서의 성능을 최대화하는 방향으로 학습법($\omega$, 예: 특정 손실 함수나 정규화 방법)을 최적화합니다.

**도메인 적응 (Domain Adaptation)**
전통적인 도메인 적응[58]을 개선하기 위해, 메타러닝을 사용하여 기반이 되는 비지도 DA 알고리즘의 성능을 최적화하는 메타-목표를 정의할 수 있습니다[59].

**comments**

*   **도메인 적응이란?**: 타겟 도메인(예: 스케치)의 레이블 없는 데이터를 약간 활용하여, 소스 도메인(예: 사진)에서 훈련된 모델을 타겟 도메인에 맞게 '조정'하는 기술입니다.
*   **메타러닝의 역할**: 수많은 도메인 적응 시나리오를 경험하게 함으로써, **"가장 효과적으로 모델을 조정하는 방법"** 자체를 학습합니다.


**벤치마크 (Benchmarks)**
DA와 DG를 위한 인기 있는 벤치마크는 사진/스케치/만화와 같은 여러 도메인에 걸친 이미지 인식을 고려합니다. **PACS**[263]는 좋은 시작 벤치마크를 제공하며, **Visual Decathlon**[42], [220]과 **Meta-Dataset**[111]은 더 큰 규모의 대안을 제공합니다.



## 하이퍼파라미터 최적화 (Hyper-parameter Optimization)

메타러닝은 $\omega$가 정규화 강도나 학습률과 같은 하이퍼파라미터를 명시하도록 고려할 때 하이퍼파라미터 최적화를 다룹니다. 두 가지 주요 설정이 있습니다: 여러 과제 분포에 걸쳐 훈련을 개선하는 하이퍼파라미터를 학습하거나, 단지 단일 과제에 대해서만 학습할 수 있습니다. 전자의 경우는 보통 최적화 기반 방법에서 특히 퓨샷 응용 분야와 관련이 있습니다. 예를 들어, MAML은 레이어별, 스텝별로 학습률을 학습함으로써 개선될 수 있습니다[80]. 단일 과제에 대한 하이퍼파라미터를 학습하고자 하는 경우는 보통 다중샷(many-shot) 응용 분야와 더 관련이 있으며[71], [157], 여기서 일부 검증 데이터는 섹션 2.1에서 논의된 바와 같이 훈련 데이터셋에서 추출될 수 있습니다. 종단간(End-to-end) 경사도 기반 메타러닝은 이미 (예를 들어 MAML[16]과 데이터셋 증류[156], [157]에서 시연된 바와 같이) 수백만 개의 파라미터에 대한 유망한 확장성을 보여주었는데, 이는 일반적으로 수십 개의 하이퍼파라미터에만 성공적인 고전적인 접근법(그리드나 랜덤 탐색[72]에 의한 교차 검증, 또는 베이지안 최적화[73])과는 대조적입니다.

**comments**

*   **하이퍼파라미터란?**: 학습률, 정규화 강도 등 모델 훈련에 영향을 주는 각종 설정값입니다.
*   **메타러닝의 역할**: **'최고의 설정값($\omega$)'**을 자동으로 찾습니다.
*   **두 가지 시나리오**:
    1.  **다중 과제 (주로 퓨샷)**: "어떤 문제가 주어져도 빠르게 학습할 수 있는 **'범용 최적 설정값'**"을 찾습니다. (예: MAML의 최적 학습률 찾기)
    2.  **단일 과제 (주로 다중샷)**: "지금 풀고 있는 **이 하나의 어려운 문제**에 대한 **'맞춤형 최적 설정값'**"을 찾습니다.
*   **장점**: 전통적인 방법(랜덤 탐색 등)은 몇십 개 정도의 하이퍼파라미터만 다룰 수 있었지만, 경사도 기반 메타러닝은 **수백만 개의 파라미터(예: MAML의 전체 초기 가중치)**까지도 하나의 거대한 하이퍼파라미터로 보고 최적화할 수 있는 **엄청난 확장성**을 보여줍니다.

## 참신하고 생물학적으로 타당한 학습기 (Novel and Biologically Plausible Learners)

기반 모델에 대해 명시적인 (피드-포워드/블랙박스가 아닌) 최적화를 사용하는 대부분의 메타러닝 연구는 역전파에 의한 경사 하강법에 기반합니다. 메타러닝은 $\omega$의 함수 클래스를 정의하여, 비지도 학습[21]이거나 생물학적으로 타당한[46], [264], [265] 새로운 학습 규칙의 발견으로 이어질 수 있으며, 헤비안 업데이트(Hebbian updates)[264]나 신경조절(neuromodulation)[265]과 같이 현대 딥러닝에서는 덜 흔하게 사용되는 아이디어를 활용합니다.

**comments**

*   **문제의식**: 현재 딥러닝의 표준 학습법인 '역전파'는 매우 강력하지만, 실제 뇌가 작동하는 방식과는 거리가 멉니다.
*   **메타러닝의 역할**: "역전파를 대체할 수 있는, **완전히 새로운 학습 규칙($\omega$)**을 AI가 스스로 발견하게 할 수는 없을까?"
*   **목표**:
    1.  **신규 학습기**: 역전파보다 더 효율적이거나 강력한 새로운 학습 규칙을 찾습니다.
    2.  **생물학적 타당성**: 실제 뇌의 뉴런 학습 방식(예: **헤비안 학습** - "함께 활성화되는 뉴런은 서로 강하게 연결된다")과 유사한, 더 그럴듯한 학습 규칙을 찾습니다.
*   **결론**: 메타러닝은 기존의 틀을 넘어서, **학습의 근본 원리 자체를 탐색**하는 도구로도 사용될 수 있는 강력한 패러다임입니다.

## 언어 및 음성 (Language and Speech)

**언어 모델링 (Language Modelling)**

퓨샷 언어 모델링은 메타-학습기의 다재다능함을 점점 더 잘 보여주고 있습니다. 초기 매칭 네트워크(matching networks)는 빠진 단어 채우기와 같은 원샷(one-shot) 과제에서 인상적인 성능을 보여주었습니다[90]. 이후 텍스트 분류[139], 신경망 프로그램 귀납[266] 및 합성[267], 영어-SQL 프로그램 합성[268], 텍스트 기반 관계 그래프 추출기[269], 기계 번역[270], 그리고 대화에서 새로운 페르소나에 빠르게 적응하기[271] 등 훨씬 더 많은 과제들이 다루어졌습니다.

**comments**

*   **언어 분야에서의 메타러닝**: 컴퓨터 비전과 마찬가지로, 언어 분야에도 데이터가 부족한 수많은 '퓨샷' 문제들이 존재합니다.
*   **응용 예시**:
    *   **텍스트 분류**: 몇 개의 예시만 보고 새로운 카테고리(예: '스포츠', '정치')의 뉴스를 분류합니다.
    *   **프로그램 합성**: "사용자 목록을 보여줘"라는 자연어 명령을 보고, `SELECT * FROM users;` 와 같은 SQL 코드를 생성하는 법을 몇 개의 예시만으로 학습합니다.
    *   **기계 번역**: 데이터가 매우 부족한 희귀 언어(low-resource language)에 대해, 몇 개의 번역 예시만으로 번역기를 만듭니다.
    *   **대화형 AI**: 챗봇이 새로운 사용자나 새로운 대화 주제(페르소나)에 빠르게 적응하여 맞춤형 대화를 나누도록 합니다.

**음성 인식 (Speech Recognition)**

딥러닝은 이제 최첨단 자동 음성 인식(ASR)의 지배적인 패러다임입니다. 메타러닝은 ASR 내에서 발생하는 많은 퓨샷 적응 문제를 해결하기 위해 적용되기 시작했습니다. 여기에는 저자원 언어에 대한 훈련 방법 학습[272], 교차 억양 적응[273], 그리고 개별 화자에 대한 모델 최적화[274]가 포함됩니다.

**comments**

*   **음성 분야에서의 메타러닝**: 음성 데이터는 개인의 목소리, 억양, 언어, 주변 소음 등 변수가 매우 많아 '적응'이 매우 중요합니다.
*   **응용 예시**:
    *   **저자원 언어**: 한국어 데이터는 많지만, 소수 민족 언어 데이터는 거의 없습니다. 이런 언어에 대한 음성 인식기를 빠르게 만드는 법을 배웁니다.
    *   **교차 억양 적응**: 표준 서울말로 훈련된 음성 인식기가, 몇 개의 예시만 듣고 부산 사투리나 영어 억양에 빠르게 적응하도록 합니다.
    *   **화자 적응**: 음성 비서가 내 목소리에만 특화되도록, 나의 발음 습관 몇 개를 듣고 빠르게 모델을 최적화합니다.



## 사회적 선을 위한 메타러닝 (Meta-learning for Social Good)

메타러닝은 데이터가 종종 부족한 의료 영상 분류나 신약 개발과 같은 사회적 선을 위한 AI 응용 분야에서 발생하는 다양한 어려운 과제에 적합합니다. 의료 분야에서의 발전은 병리학자의 전 세계적인 부족을 고려할 때 특히 중요합니다[275]. [5]에서는 LSTM과 그래프 신경망을 결합하여 원샷 데이터 상황에서 분자(예: 독성)의 행동을 예측합니다. [276]에서는 MAML이 약지도 유방암 탐지 과제에 적용되며, 과제의 순서는 커리큘럼에 따라 선택됩니다. MAML은 또한 의료 시각적 질문 답변(medical visual question answering)[277]을 위해 노이즈 제거 오토인코더와 결합되며, 서포트 샘플의 가중치를 학습하는 방법[218]은 노이즈가 있는 레이블을 가진 피부 병변 분할 과제에서 픽셀 단위 가중치 부여에 적용됩니다[278].

**comments**

*   **사회적 선(Social Good) 분야의 특징**: 공익을 위한 중요한 문제들이지만, 데이터 수집이 매우 어렵거나 비쌉니다. (예: 희귀병 환자 데이터, 신약 후보 물질 실험 데이터)
*   **메타러닝의 역할**: 이러한 **'데이터 부족'** 문제를 해결하는 데 메타러닝이 핵심적인 역할을 합니다.
*   **응용 예시**:
    *   **신약 개발**: 단 한 번의 실험 결과만으로 새로운 화학 물질의 독성이나 효능을 예측합니다.
    *   **의료 영상 분석**: 의사가 레이블링한 소수의 암 영상 데이터만으로, 새로운 환자의 영상에서 암을 탐지하는 모델을 만듭니다.
    *   **데이터 노이즈 처리**: 여러 의사의 판독 소견이 약간씩 다른(노이즈가 있는) 데이터로부터, 더 정확하게 병변을 찾아내는 방법을 학습합니다.



## 추상적 추론 (Abstract Reasoning)

딥러닝의 장기적인 목표 중 하나는 단순한 인식 과제를 넘어, 레이븐의 점진적 행렬(Raven's Progressive Matrices, RPMs)[279] 형태의 IQ 테스트와 같은 더 추상적인 추론 문제를 다루는 것입니다. RPM을 푸는 것은 컨텍스트 패널로부터 답변 패널로의 퓨샷 일반화를 요구하는 것으로 볼 수 있습니다. RPM을 사용한 추상적 추론에 대한 최근 메타러닝 접근법은 패널에 대한 데이터 생성 분포를 정의하는 교사(teacher)를 메타-학습함으로써 상당한 개선을 달성했습니다[280]. 교사는 학생과 함께 공동으로 훈련되며, 학생의 발전에 의해 보상을 받습니다.

**comments**

*   **추상적 추론이란?**: 이미지 속 객체를 인식하는 것을 넘어, **규칙과 패턴을 발견하고 논리적으로 추론**하는 능력입니다. (예: IQ 테스트의 도형 문제)
*   **메타러닝과의 관계**: IQ 테스트 문제는 보통 몇 개의 예시 도형(컨텍스트)을 보고 규칙을 파악한 뒤, 빈칸에 들어갈 도형(답변)을 찾는 **'퓨샷 추론'** 문제입니다.
*   **메타러닝의 역할**: "어떤 규칙을 가진 문제들을 만들어줘야 학생(추론 모델)이 가장 효과적으로 추론 능력을 기를 수 있을까?"를 고민하는 **'최고의 선생님'($\omega$)**을 메타-학습합니다. 이 선생님은 학생의 성과를 보면서 문제의 난이도와 유형을 동적으로 조절합니다.



## 시스템 (Systems)

**네트워크 압축 (Network Compression)**

현대의 CNN은 임베디드 장치에서는 사용이 금지될 수 있는 많은 양의 메모리를 필요로 합니다. 따라서 양자화(quantization)나 가지치기(pruning)와 같은 다양한 형태의 네트워크 압축은 최신 연구 분야입니다[281]. 메타러닝은 양자화된 네트워크가 훈련될 수 있도록 하는 경사도 생성기 메타-네트워크를 훈련시키거나[202], 양자화된 네트워크가 경사도로 훈련될 수 있도록 하는 가중치 생성기 메타-네트워크를 훈련시키는[282] 등, 이 목표에도 적용되기 시작했습니다.

**comments**

*   **네트워크 압축이란?**: 스마트폰 같은 저사양 기기에서도 딥러닝 모델이 돌아갈 수 있도록 모델의 **용량을 줄이는 기술**입니다.
*   **문제점**: 모델을 압축(양자화 등)하면 미분이 불가능해져서 일반적인 방법으로는 훈련시키기 어렵습니다.
*   **메타러닝의 역할**:
    *   압축된 모델을 훈련시키기 위한 **'가짜 경사도(gradient)'를 생성하는 또 다른 신경망($\omega$)**을 학습합니다.
    *   또는, 압축 과정 자체를 학습 과정에 포함시켜, **"어떻게 압축해야 성능 손실이 가장 적을까?"**를 메타-학습합니다.

---

**통신 (Communications)**

딥러닝은 통신 시스템에 빠르게 영향을 미치고 있습니다. 예를 들어, 실제 채널에 대해 사람이 설계한 최고의 코드를 능가하는 코딩 시스템을 학습함으로써[283] 가능합니다. 퓨샷 메타러닝은 변화하는 채널 특성에 코드를 빠르게 적응시키는 데 사용될 수 있습니다[284].

**comments**

*   **딥러닝 in 통신**: 통신 과정(데이터 인코딩/디코딩) 자체를 딥러닝으로 최적화합니다.
*   **메타러닝의 역할**: 통신 환경(채널 상태)은 시시각각 변합니다. 메타러닝은 "현재 채널 상태가 이러하니, 지금은 이런 방식으로 데이터를 인코딩/디코딩하는 것이 가장 효율적이야"라고 **실시간으로 통신 방식을 최적화**하는 법을 배웁니다.



**능동 학습 (Active Learning, AL)**

AL 방법은 지도 학습을 감싸고, 선택적인 데이터 주석(annotation)을 위한 정책을 정의합니다 – 일반적으로 주석을 순차적으로 얻을 수 있는 설정에서. AL의 목표는 가장 적은 주석으로 다운스트림 지도 학습의 성능을 최대화하기 위해 주석을 달 최적의 데이터 부분집합을 찾는 것입니다. AL은 수많은 사람이 설계한 알고리즘이 있는 잘 연구된 문제입니다[285]. 메타러닝은 다음과 같이 능동 학습 알고리즘 설계를 학습 과제로 매핑할 수 있습니다: (i) 내부-수준 최적화를 지금까지 주석이 달린 데이터셋에 대한 전통적인 지도 학습으로 정의하고, (ii) $\omega$를 주석을 달 최고의 레이블 없는 데이터 포인트를 선택하는 쿼리 정책으로 정의하고, (iii), 메타-목표를 쿼리 정책에 따른 반복적인 학습 및 주석 후의 검증 성능으로 정의하고, (iv) 최적의 주석 쿼리 정책을 훈련시키기 위해 외부-수준 최적화를 수행합니다[190]–[192]. 그러나 레이블이 AL 알고리즘을 훈련시키는 데 사용된다면, 그들의 훈련 비용을 상각하기 위해 여러 과제에 걸쳐 일반화될 필요가 있습니다[192].

**comments**

*   **능동 학습이란?**: 모델이 **"어떤 데이터에 레이블을 달아야 가장 효율적으로 똑똑해질까?"**를 스스로 판단하여 사람에게 질문하는 기술입니다. (라벨링 비용 절약)
*   **메타러닝의 역할**: "어떤 기준으로 질문($\omega$)해야 가장 적은 질문으로 최고 성능에 도달할까?"라는 **'최고의 질문 전략'** 자체를 메타-학습합니다.
*   **작동 방식 (메타러닝 구조)**:
    *   **내부 루프**: 현재까지 레이블이 달린 데이터로 모델을 학습합니다.
    *   **외부 루프**: 어떤 '질문 전략($\omega$)'을 썼을 때, 최종 모델의 검증 성능이 가장 높아지는지를 평가하여, 최고의 질문 전략을 찾습니다.

**레이블 노이즈와 함께 학습하기 (Learning with Label Noise)**

웹 스크래핑이나 크라우드소싱으로 대규모 데이터셋을 수집할 때 흔히 발생합니다. 이 상황을 위해 사람이 설계한 많은 알고리즘이 있지만, 최근 메타러닝 방법들은 레이블 노이즈 문제를 다루어왔습니다. 예를 들어, 노이즈가 있는 샘플의 가중치를 낮추기 위해 샘플별 가중치를 추론적으로(transductively) 학습하거나[151], 노이즈가 있는 레이블 훈련에 강건한 초기 조건을 학습하는 것입니다[96].

**comments**

*   **문제 상황**: 인터넷에서 수집한 데이터에는 잘못된 레이블(예: 고양이 사진에 '개'라고 태그된 경우)이 섞여 있기 마련입니다.
*   **메타러닝의 역할**: "어떻게 훈련해야 **잘못된 정답지에 속지 않고** 진짜 지식만 배울 수 있을까?"라는 **'노이즈 면역 학습법'($\omega$)**을 배웁니다.
*   **어떻게?**:
    1.  **가중치 조절**: "이 데이터는 정답이 좀 이상한 것 같으니, 학습할 때 비중을 낮추자"라고 판단하여 데이터 샘플별 **중요도($\omega$)**를 학습합니다.
    2.  **강건한 초기 조건**: MAML처럼, "어떤 출발점에서 학습을 시작해야, 중간에 이상한 데이터를 만나도 길을 잃지 않고 올바른 방향으로 갈 수 있을까?"라는 **'튼튼한 초기값'($\omega$)**을 학습합니다.

**적대적 공격 및 방어 (Adversarial Attacks and Defenses)**
딥러닝 신경망은 사람이 쉽게 인식할 수 있는 데이터 포인트에, 사람이 감지할 수 없는 교란(perturbation)을 교묘하게 추가하여 잘못 분류하도록 속일 수 있습니다[286]. 최근 몇 년간 수많은 공격 및 방어 방법이 발표되었으며, 방어 전략은 보통 신중하게 손수 설계된 구조나 훈련 알고리즘으로 구성됩니다. 도메인 이동의 경우와 유사하게, 적대적 공격 하에서의 성능을 메타-손실로 정의함으로써 강건성을 위한 학습 알고리즘을 훈련시킬 수 있습니다[97], [287].

**comments**

*   **적대적 공격이란?**: 이미지에 사람이 볼 수 없는 미세한 노이즈를 추가하여 AI 모델이 이미지를 완전히 다른 것으로 착각하게 만드는 해킹 기술입니다.
*   **메타러닝의 역할**: "어떻게 훈련해야 **이런 해킹 공격에 속지 않는 '철벽 방어 모델'**을 만들 수 있을까?"라는 **'보안 강화 학습법'($\omega$)**을 배웁니다.
*   **어떻게?**: 훈련 과정에서 일부러 '가상 해킹' 상황을 시뮬레이션합니다.
    *   **내부 루프**: 정상 데이터로 모델을 훈련시킵니다.
    *   **외부 루프**: 훈련된 모델을 **'가상 해커'가 공격**하게 한 뒤, 그 공격을 얼마나 잘 막아내는지를 평가합니다. 이 방어 성능을 최대화하는 방향으로 학습 알고리즘($\omega$)을 최적화합니다.


**추천 시스템 (Recommendation Systems)**
추천 시스템은 상업 분야에서 머신러닝의 성숙한 소비처입니다. 그러나 과거 상호작용 데이터가 거의 없는 신규 사용자나 추천할 새로운 아이템에 대한 추천을 시작하는 것은 **콜드 스타트(cold-start) 문제**로 알려진 어려운 과제로 남아있습니다. 메타러닝은 아이템 콜드 스타트에 블랙박스 모델을 적용하고[288], 사용자 콜드 스타트에 경사도 기반 방법을 적용해왔습니다[289].

**comments**

*   **콜드 스타트 문제란?**: 쇼핑몰에 새로 가입한 사용자(활동 기록 없음)나 새로 등록된 상품(리뷰 없음)에 대해 무엇을 추천해야 할지 막막한 상황입니다.
*   **메타러닝의 역할**: "어떻게 하면 **몇 안 되는 초기 정보만으로도 사용자의 취향이나 상품의 특성을 빠르게 파악**할 수 있을까?"라는 **'빠른 파악 능력'($\omega$)**을 학습합니다.
*   **어떻게?**: 수많은 '신규 사용자'와 '신규 상품' 케이스를 학습 데이터로 삼습니다.
    *   **내부 루프**: 신규 사용자의 몇 안 되는 클릭 기록으로 추천 모델을 잠시 학습시킵니다.
    *   **외부 루프**: 그 추천이 얼마나 정확했는지를 평가하여, "신규 사용자의 취향을 가장 빨리 파악할 수 있는 **'초기 추천 모델'($\omega$)**"을 학습합니다.